# Python 2

In [1]:
import numpy as np

from LHCO_reader import LHCO_reader

import math
import os

# Plotting library
import matplotlib.pyplot as plt

import pandas as pd

In [2]:
!ls ../data/Signal/lhco/*.lhco

../data/Signal/lhco/BP0.lhco
../data/Signal/lhco/BP10_01.lhco
../data/Signal/lhco/BP10_02.lhco
../data/Signal/lhco/BP11_01.lhco
../data/Signal/lhco/BP11_02.lhco
../data/Signal/lhco/BP12_01.lhco
../data/Signal/lhco/BP12_02.lhco
../data/Signal/lhco/BP13_01.lhco
../data/Signal/lhco/BP13_02.lhco
../data/Signal/lhco/BP14_01.lhco
../data/Signal/lhco/BP14_02.lhco
../data/Signal/lhco/BP15_01.lhco
../data/Signal/lhco/BP15_02.lhco
../data/Signal/lhco/BP1.lhco
../data/Signal/lhco/BP1_run100k_10.lhco
../data/Signal/lhco/BP1_run100k_11.lhco
../data/Signal/lhco/BP1_run100k_12.lhco
../data/Signal/lhco/BP1_run100k_13.lhco
../data/Signal/lhco/BP1_run100k_14.lhco
../data/Signal/lhco/BP1_run100k_15.lhco
../data/Signal/lhco/BP1_run100k_16.lhco
../data/Signal/lhco/BP1_run100k_17.lhco
../data/Signal/lhco/BP1_run100k_18.lhco
../data/Signal/lhco/BP1_run100k_19.lhco
../data/Signal/lhco/BP1_run100k_1.lhco
../data/Signal/lhco/BP1_run100k_20.lhco
../data/Signal/lhco/BP1_run100k_2.lhco
../data/Signal/lhco/BP1_run1

## Signal

In [3]:
signal_NLO  = 1.25
luminosidad = 100

cross_section_bps    = np.asarray([190, 189, 189, 92, 92, 92, 49, 48, 48, 27, 27, 27, 16, 16, 16])
branching_ratios_bps = np.asarray([0.14, 0.37, 0.73, 0.15, 0.36, 0.67, 0.16, 0.36, 0.62, 0.17, 0.35, 0.58, 0.16, 0.32, 0.52])

signal_cross = cross_section_bps * branching_ratios_bps *  0.2134 # fb

In [13]:
# PARA BPs de RO. 1,2,3,4

Folder    = '../data/Signal/lhco/'
datFolder = '../data/Signal/'


signal_photon  = []
signal_jet     = []
signal_lepton  = []
signal_MET     = []
signal_hadinfo = []

signal_cut_ev  = 0
signal_init_ev = 0

counter_bad_jets = 0
contador_tau     = 0

bp = 4
if bp == 3:
    signal_runs = 26 # 21 for BP1-2-4 y 26 for BP3
else:
    signal_runs = 21
    
for ii in range(1, signal_runs):
    print(ii) 
    inputevents = LHCO_reader.Events(f_name = Folder + "BP" + str(bp) + "_run100k_" + str(ii) + ".lhco")

    signal_init_ev += len(inputevents)
    print('Esta run tuvo ' + str(len(inputevents)) + ' eventos en el lhco')

    for i in range(len(inputevents)):

        # at least one: photon, jet and lepton (electron or muon)
        if inputevents[i].number()["photon"] > 0 and inputevents[i].number()["jet"] > 0 and (inputevents[i].number()["electron"] > 0 or inputevents[i].number()["muon"] > 0):
            if inputevents[i]["jet"][0]["PT"] > 100 and -4.5 < inputevents[i]["jet"][0]["eta"] < 4.5:
                if inputevents[i]["photon"][0]["PT"] > 10 and -2.37 < inputevents[i]["photon"][0]["eta"] < 2.37:
                    if inputevents[i]["MET"][0]["PT"] > 200:
                        
                        # lepton info
                        if (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] > 0):
                            if inputevents[i]["electron"][0]["PT"] > inputevents[i]["muon"][0]["PT"]: 
                                flag = 'electron'
                                eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                            else: 
                                flag = 'muon'
                                eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron
                        elif (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] == 0): 
                            flag = 'electron'
                            eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                        elif (inputevents[i].number()["muon"] > 0) and (inputevents[i].number()["electron"] == 0): 
                            flag = 'muon'
                            eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron

                        if (-2.47 < eta_leading_lepton < -1.52) or (-1.37 < eta_leading_lepton < 1.37) or (1.52 < eta_leading_lepton < 2.47): 
                            
                            # how many jets (total) and total hadronic pT
                            flag_bad_jets = 0                            
                            num_jets = 0
                            HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
                            HT = 0 # total HT, defined as the scalar sum of the pT of all reconstructed particles

                            for j in range(inputevents[i].number()["jet"]):
                                if (inputevents[i]["jet"][j]["PT"] > 20) and (-4.5 < inputevents[i]["jet"][j]["eta"] < 4.5):
                                    num_jets += 1
                                    HT_had += inputevents[i]["jet"][j]["PT"]
                                    HT += inputevents[i]["jet"][j]["PT"]
                                if (np.abs(inputevents[i]["jet"][j]["eta"]) > 4.5) or (inputevents[i]["jet"][j]["PT"] < 20):
                                    flag_bad_jets = 1
                            
                            # how many photons                         
                            num_photons = 0
                            for j in range(inputevents[i].number()["photon"]):
                                if inputevents[i]["photon"][j]["PT"] > 10 and -2.37 < inputevents[i]["photon"][j]["eta"] < 2.37:
                                    num_photons = num_photons + 1
                                    HT += inputevents[i]["photon"][j]["PT"]
                                    
                            # how many muons                        
                            num_muons = 0
                            for j in range(inputevents[i].number()["muon"]):
                                if inputevents[i]["muon"][j]["PT"] > 10 and -2.7 < inputevents[i]["muon"][j]["eta"] < 2.7:
                                    num_muons = num_muons + 1
                                    HT += inputevents[i]["muon"][j]["PT"]
                                    
                            # how many electrons                        
                            num_electrons = 0
                            for j in range(inputevents[i].number()["electron"]):
                                if inputevents[i]["electron"][j]["PT"] > 10:
                                    eta = inputevents[i]["electron"][j]["eta"]
                                    if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                        num_electrons = num_electrons + 1
                                        HT += inputevents[i]["electron"][j]["PT"]
                                    
                            if flag_bad_jets == 1: counter_bad_jets += 1 
                            if flag_bad_jets == 0:
                                num_taus = 0
                                if inputevents[i].number()["tau"] > 0: 
                                    contador_tau = contador_tau + 1
                                    # how many taus                    
                                    for j in range(inputevents[i].number()["tau"]):
                                        if inputevents[i]["tau"][j]["PT"] > 20:
                                            eta = inputevents[i]["tau"][j]["eta"]
                                            if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                                num_taus = num_taus + 1
                                                HT += inputevents[i]["tau"][j]["PT"]

                                # photon info:
                                with open(datFolder + 'BP' + str(bp) + '_photon_6.dat', 'a') as f:
                                    f.write('%f %f %f %d\n' % (inputevents[i]["photon"][0]["PT"],inputevents[i]["photon"][0]["eta"],inputevents[i]["photon"][0]["phi"], num_photons))

                                # jet info:
                                with open(datFolder + 'BP' + str(bp) + '_jet_6.dat', 'a') as f:
                                    f.write('%f %f %f\n' % (inputevents[i]["jet"][0]["PT"],inputevents[i]["jet"][0]["eta"],inputevents[i]["jet"][0]["phi"]))

                                # lepton info
                                num_leptons = num_electrons + num_muons
                                if flag == 'electron':
                                    with open(datFolder + 'BP' + str(bp) + '_lepton_6.dat', 'a') as f:
                                        f.write('%f %f %f %d %d\n' % (inputevents[i]["electron"][0]["PT"],inputevents[i]["electron"][0]["eta"],inputevents[i]["electron"][0]["phi"], num_leptons, num_taus))
                                else:
                                    with open(datFolder + 'BP' + str(bp) + '_lepton_6.dat', 'a') as f:
                                        f.write('%f %f %f %d %d\n' % (inputevents[i]["muon"][0]["PT"],inputevents[i]["muon"][0]["eta"],inputevents[i]["muon"][0]["phi"], num_leptons, num_taus))

                                # MET (eta=0 by def, but I save phi because I have many jets)
                                with open(datFolder + 'BP' + str(bp) + '_MET_6.dat', 'a') as f:
                                    f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))

                                # hadronic info
                                with open(datFolder + 'BP' + str(bp) + '_hadinfo_6.dat', 'a') as f:
                                    f.write('%f %f %d\n' % (HT, HT_had, num_jets))

                                # total events that passed the cuts
                                signal_cut_ev += 1

print('Num initial events: ', signal_init_ev)
print('Total pasaron los cortes: ', signal_cut_ev)
print('Hay: ', contador_tau, ' taus')
print('Hay: ', counter_bad_jets, ' eventos con al menos un jet malo')


cross       = signal_cross[bp-1] * signal_NLO
aceptancia  = signal_cut_ev / (1. * signal_init_ev)

signal_fidcross = cross * aceptancia
signal_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', signal_fidcross)
print('Events expected: ', signal_expected)

1
Esta run tuvo 100000 eventos en el lhco
2
Esta run tuvo 100000 eventos en el lhco
3
Esta run tuvo 100000 eventos en el lhco
4
Esta run tuvo 100000 eventos en el lhco
5
Esta run tuvo 100000 eventos en el lhco
6
Esta run tuvo 100000 eventos en el lhco
7
Esta run tuvo 100000 eventos en el lhco
8
Esta run tuvo 100000 eventos en el lhco
9
Esta run tuvo 100000 eventos en el lhco
10
Esta run tuvo 100000 eventos en el lhco
11
Esta run tuvo 100000 eventos en el lhco
12
Esta run tuvo 100000 eventos en el lhco
13
Esta run tuvo 100000 eventos en el lhco
14
Esta run tuvo 100000 eventos en el lhco
15
Esta run tuvo 100000 eventos en el lhco
16
Esta run tuvo 100000 eventos en el lhco
17
Esta run tuvo 100000 eventos en el lhco
18
Esta run tuvo 100000 eventos en el lhco
19
Esta run tuvo 100000 eventos en el lhco
20
Esta run tuvo 100000 eventos en el lhco
('Num initial events: ', 2000000)
('Total pasaron los cortes: ', 268160)
('Hay: ', 4405, ' taus')
('Hay: ', 2758, ' eventos con al menos un jet malo'

In [11]:
# PARA BPs de ERNEST
Folder    = '../data/Signal/lhco/'
datFolder = '../data/Signal/'


signal_photon  = []
signal_jet     = []
signal_lepton  = []
signal_MET     = []
signal_hadinfo = []

signal_bps = 15
for ii in range(5, signal_bps + 1):
    print(ii)
    counter_bad_jets = 0
    contador_tau = 0
    if ii not in [5]: # BPs que ya fueron analizados
        signal_cut_ev  = 0
        signal_init_ev = 0
        for jj in range(2):
            inputevents = LHCO_reader.Events(f_name = Folder + "BP" + str(ii) + "_0" + str(jj + 1) + ".lhco")

            signal_init_ev = len(inputevents) + signal_init_ev
            print('Esta run tuvo ' + str(len(inputevents)) + ' eventos en el lhco')

            for i in range(len(inputevents)):

                # at least one: photon, jet and lepton (electron or muon)
                if inputevents[i].number()["photon"] > 0 and inputevents[i].number()["jet"] > 0 and (inputevents[i].number()["electron"] > 0 or inputevents[i].number()["muon"] > 0):
                    if inputevents[i]["jet"][0]["PT"] > 100 and -4.5 < inputevents[i]["jet"][0]["eta"] < 4.5:
                        if inputevents[i]["photon"][0]["PT"] > 10 and -2.37 < inputevents[i]["photon"][0]["eta"] < 2.37:
                            if inputevents[i]["MET"][0]["PT"] > 200:
                                # lepton info
                                if (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] > 0):
                                    if inputevents[i]["electron"][0]["PT"] > inputevents[i]["muon"][0]["PT"]: 
                                        flag = 'electron'
                                        eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                                    else: 
                                        flag = 'muon'
                                        eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron
                                elif (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] == 0): 
                                    flag = 'electron'
                                    eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                                elif (inputevents[i].number()["muon"] > 0) and (inputevents[i].number()["electron"] == 0): 
                                    flag = 'muon'
                                    eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron

                                if (-2.47 < eta_leading_lepton < -1.52) or (-1.37 < eta_leading_lepton < 1.37) or (1.52 < eta_leading_lepton < 2.47): 

                                    # how many jets (total) and total hadronic pT
                                    flag_bad_jets = 0                            
                                    num_jets = 0
                                    HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
                                    HT = 0 # total HT, defined as the scalar sum of the pT of all reconstructed particles

                                    for j in range(inputevents[i].number()["jet"]):
                                        if (inputevents[i]["jet"][j]["PT"] > 20) and (-4.5 < inputevents[i]["jet"][j]["eta"] < 4.5):
                                            num_jets += 1
                                            HT_had += inputevents[i]["jet"][j]["PT"]
                                            HT += inputevents[i]["jet"][j]["PT"]
                                        if (np.abs(inputevents[i]["jet"][j]["eta"]) > 4.5) or (inputevents[i]["jet"][j]["PT"] < 20):
                                            flag_bad_jets = 1

                                    # how many photons                         
                                    num_photons = 0
                                    for j in range(inputevents[i].number()["photon"]):
                                        if inputevents[i]["photon"][j]["PT"] > 10 and -2.37 < inputevents[i]["photon"][j]["eta"] < 2.37:
                                            num_photons = num_photons + 1
                                            HT += inputevents[i]["photon"][j]["PT"]

                                    # how many muons                        
                                    num_muons = 0
                                    for j in range(inputevents[i].number()["muon"]):
                                        if inputevents[i]["muon"][j]["PT"] > 10 and -2.7 < inputevents[i]["muon"][j]["eta"] < 2.7:
                                            num_muons = num_muons + 1
                                            HT += inputevents[i]["muon"][j]["PT"]

                                    # how many electrons                        
                                    num_electrons = 0
                                    for j in range(inputevents[i].number()["electron"]):
                                        if inputevents[i]["electron"][j]["PT"] > 10:
                                            eta = inputevents[i]["electron"][j]["eta"]
                                            if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                                num_electrons = num_electrons + 1
                                                HT += inputevents[i]["electron"][j]["PT"]

                                    if flag_bad_jets == 1: counter_bad_jets += 1 
                                    if flag_bad_jets == 0:
                                        num_taus = 0
                                        if inputevents[i].number()["tau"] > 0: 
                                            contador_tau = contador_tau + 1
                                            # how many taus                    
                                            for j in range(inputevents[i].number()["tau"]):
                                                if inputevents[i]["tau"][j]["PT"] > 20:
                                                    eta = inputevents[i]["tau"][j]["eta"]
                                                    if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                                        num_taus = num_taus + 1
                                                        HT += inputevents[i]["tau"][j]["PT"]

                                        # photon info:
                                        with open(datFolder + "BP" + str(ii) + '_photon_6.dat', 'a') as f:
                                            f.write('%f %f %f %d\n' % (inputevents[i]["photon"][0]["PT"],inputevents[i]["photon"][0]["eta"],inputevents[i]["photon"][0]["phi"], num_photons))

                                        # jet info:
                                        with open(datFolder + "BP" + str(ii) + '_jet_6.dat', 'a') as f:
                                            f.write('%f %f %f\n' % (inputevents[i]["jet"][0]["PT"],inputevents[i]["jet"][0]["eta"],inputevents[i]["jet"][0]["phi"]))

                                        # lepton info
                                        num_leptons = num_electrons + num_muons
                                        if flag == "electron":
                                            with open(datFolder + "BP" + str(ii) + '_lepton_6.dat', 'a') as f:
                                                f.write('%f %f %f %d %d\n' % (inputevents[i]["electron"][0]["PT"],inputevents[i]["electron"][0]["eta"],inputevents[i]["electron"][0]["phi"], num_leptons,num_taus))
                                        else:
                                            with open(datFolder + "BP" + str(ii) + '_lepton_6.dat', 'a') as f:
                                                f.write('%f %f %f %d %d\n' % (inputevents[i]["muon"][0]["PT"],inputevents[i]["muon"][0]["eta"],inputevents[i]["muon"][0]["phi"], num_leptons,num_taus))

                                        # MET (eta=0 by def, but I save phi because I have many jets)
                                        with open(datFolder + "BP" + str(ii) + '_MET_6.dat', 'a') as f:
                                            f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))

                                        # hadronic info
                                        with open(datFolder + "BP" + str(ii) + '_hadinfo_6.dat', 'a') as f:
                                            f.write('%f %f %d\n' % (HT, HT_had, num_jets))

                                        # total events that passed the cuts
                                        signal_cut_ev += 1
        print('End of BP' + str(ii) + ' analysis')
        inputevents = 0 # Just to clean the memory

        print('Num initial events: ', signal_init_ev)
        print('Total pasaron los cortes: ', signal_cut_ev)
        print('Hay: ', contador_tau, ' taus')
        print('Hay: ', counter_bad_jets, ' eventos con al menos un jet malo')
        cross       = signal_cross[ii-1] * signal_NLO
        aceptancia  = signal_cut_ev / (1. * signal_init_ev)

        signal_fidcross = cross * aceptancia
        signal_expected = cross * aceptancia * luminosidad

        print('fiducial cross section: ', signal_fidcross)
        print('Events expected: ', signal_expected)

5
6
Esta run tuvo 1000000 eventos en el lhco
Esta run tuvo 1000000 eventos en el lhco
End of BP6 analysis
('Num initial events: ', 2000000)
('Total pasaron los cortes: ', 218307)
('Hay: ', 3741, ' taus')
('Hay: ', 2185, ' eventos con al menos un jet malo')
('fiducial cross section: ', 1.794753149145)
('Events expected: ', 179.4753149145)
7
Esta run tuvo 1000000 eventos en el lhco
Esta run tuvo 1000000 eventos en el lhco
End of BP7 analysis
('Num initial events: ', 2000000)
('Total pasaron los cortes: ', 302978)
('Hay: ', 5042, ' taus')
('Hay: ', 3063, ' eventos con al menos un jet malo')
('fiducial cross section: ', 0.31681197548000006)
('Events expected: ', 31.681197548000007)
8
Esta run tuvo 1000000 eventos en el lhco
Esta run tuvo 1000000 eventos en el lhco
End of BP8 analysis
('Num initial events: ', 2000000)
('Total pasaron los cortes: ', 279921)
('Hay: ', 4616, ' taus')
('Hay: ', 2831, ' eventos con al menos un jet malo')
('fiducial cross section: ', 0.64513952712)
('Events expec

In [20]:
signal_cross = 48 * 0.36 *  0.2134 # fb
signal_init_sim_eve = 1000000
signal_cut_ev       = 293092
signal_NLO          = 1.25

cross       = signal_cross * signal_NLO
aceptancia  = signal_cut_ev / (1. * signal_init_sim_eve)
luminosidad = 100

signal_fidcross = cross * aceptancia
signal_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', signal_fidcross)
print('Events expected: ', signal_expected)

('fiducial cross section: ', 1.35098998848)
('Events expected: ', 135.098998848)


## Backgrounds

#### tt (sin jets)
- [x] Done
    - se esperan 9872.695 con met > 100
    - se esperan 1079.33 con met > 200

In [20]:
Folder    = '../data/wagner-bkgs-varios/'
datFolder = '../data/wagner-bkgs-varios/'

tt_runs  = 1

tt_init_ev = 0
tt_cut_ev  = 0

tt_photon  = []
tt_jet     = []
tt_lepton  = []
tt_MET     = []
tt_hadinfo = []


counter_bad_jets = 0
contador_tau = 0
contador_btag = 0
for ii in range(1, 1 + tt_runs):
    print(ii)
    if ii not in [0]:
        inputevents = LHCO_reader.Events(f_name = Folder + 'tt-tests.lhco')

        tt_init_ev += len(inputevents)
        print('Esta run tuvo ' + str(len(inputevents)) + ' eventos en el lhco')
        
        for i in range(len(inputevents)):

            num_jets = 0
    #        num_btag = 0
            HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets


            # at least one: photon, jet and lepton (electron or muon)
            if inputevents[i].number()["photon"] > 0 and inputevents[i].number()["jet"] > 0 and (inputevents[i].number()["electron"] > 0 or inputevents[i].number()["muon"] > 0):
                if inputevents[i]["jet"][0]["PT"] > 100 and -4.5 < inputevents[i]["jet"][0]["eta"] < 4.5:
                    if inputevents[i]["photon"][0]["PT"] > 10 and -2.37 < inputevents[i]["photon"][0]["eta"] < 2.37:
                        if inputevents[i]["MET"][0]["PT"] > 100:
                            # lepton info
                            if (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] > 0):
                                if inputevents[i]["electron"][0]["PT"] > inputevents[i]["muon"][0]["PT"]: 
                                    flag = 'electron'
                                    eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                                else: 
                                    flag = 'muon'
                                    eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron
                            elif (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] == 0): 
                                flag = 'electron'
                                eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                            elif (inputevents[i].number()["muon"] > 0) and (inputevents[i].number()["electron"] == 0): 
                                flag = 'muon'
                                eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron

                            if (-2.47 < eta_leading_lepton < -1.52) or (-1.37 < eta_leading_lepton < 1.37) or (1.52 < eta_leading_lepton < 2.47): 

                                # how many jets (total) and total hadronic pT
                                flag_bad_jets = 0                            
                                num_jets = 0
                                HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
                                HT = 0 # total HT, defined as the scalar sum of the pT of all reconstructed particles

                                for j in range(inputevents[i].number()["jet"]):
                                    if (inputevents[i]["jet"][j]["PT"] > 20) and (-4.5 < inputevents[i]["jet"][j]["eta"] < 4.5):
                                        num_jets += 1
                                        HT_had += inputevents[i]["jet"][j]["PT"]
                                        HT += inputevents[i]["jet"][j]["PT"]
                                    if (np.abs(inputevents[i]["jet"][j]["eta"]) > 4.5) or (inputevents[i]["jet"][j]["PT"] < 20):
                                        flag_bad_jets = 1

                                # how many photons                         
                                num_photons = 0
                                for j in range(inputevents[i].number()["photon"]):
                                    if inputevents[i]["photon"][j]["PT"] > 10 and -2.37 < inputevents[i]["photon"][j]["eta"] < 2.37:
                                        num_photons = num_photons + 1
                                        HT += inputevents[i]["photon"][j]["PT"]

                                # how many muons                        
                                num_muons = 0
                                for j in range(inputevents[i].number()["muon"]):
                                    if inputevents[i]["muon"][j]["PT"] > 10 and -2.7 < inputevents[i]["muon"][j]["eta"] < 2.7:
                                        num_muons = num_muons + 1
                                        HT += inputevents[i]["muon"][j]["PT"]

                                # how many electrons                        
                                num_electrons = 0
                                for j in range(inputevents[i].number()["electron"]):
                                    if inputevents[i]["electron"][j]["PT"] > 10:
                                        eta = inputevents[i]["electron"][j]["eta"]
                                        if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                            num_electrons = num_electrons + 1
                                            HT += inputevents[i]["electron"][j]["PT"]

                                if flag_bad_jets == 1: counter_bad_jets += 1 
                                if flag_bad_jets == 0:
                                    num_taus = 0
                                    if inputevents[i].number()["tau"] > 0: 
                                        contador_tau = contador_tau + 1
                                        # how many taus                    
                                        for j in range(inputevents[i].number()["tau"]):
                                            if inputevents[i]["tau"][j]["PT"] > 20:
                                                eta = inputevents[i]["tau"][j]["eta"]
                                                if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                                    num_taus = num_taus + 1
                                                    HT += inputevents[i]["tau"][j]["PT"]
                                                    
                                    if inputevents[i]["jet"][0]['btag'] == 1: contador_btag = contador_btag + 1

                                    # photon info:
                                    with open(datFolder + 'photon_bu.dat', 'a') as f:
                                        f.write('%f %f %f %d\n' % (inputevents[i]["photon"][0]["PT"],inputevents[i]["photon"][0]["eta"],inputevents[i]["photon"][0]["phi"], num_photons))

                                    # jet info:
                                    with open(datFolder + 'jet_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["jet"][0]["PT"],inputevents[i]["jet"][0]["eta"],inputevents[i]["jet"][0]["phi"]))

                                    # lepton info
                                    num_leptons = num_electrons + num_muons
                                    if flag == "electron":
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["electron"][0]["PT"],inputevents[i]["electron"][0]["eta"],inputevents[i]["electron"][0]["phi"], num_leptons, num_taus))
                                    else:
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["muon"][0]["PT"],inputevents[i]["muon"][0]["eta"],inputevents[i]["muon"][0]["phi"], num_leptons, num_taus))

                                    # MET (eta=0 by def, but I save phi because I have many jets)
                                    with open(datFolder + 'MET_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))

                                    # hadronic info
                                    with open(datFolder + 'hadinfo_bu.dat', 'a') as f:
                                        f.write('%f %f %d\n' % (HT, HT_had, num_jets))

                                    # total events that passed the cuts
                                    tt_cut_ev += 1

print('Num initial events: ', tt_init_ev)
print('Total pasaron los cortes: ', tt_cut_ev)
print('Hay: ', contador_tau, ' taus')
print('Hay: ', counter_bad_jets, ' eventos con al menos un jet malo')
print('Hay: ', contador_btag, ' con btag en leading jet')

1
Esta run tuvo 200000 eventos en el lhco
('Num initial events: ', 200000)
('Total pasaron los cortes: ', 311)
('Hay: ', 21, ' taus')
('Hay: ', 7, ' eventos con al menos un jet malo')
('Hay: ', 161, ' con btag en leading jet')


In [21]:
tt_cross = 63.49 #[pb] integrated cross-section without matching
tt_NLO   = 1. # k-factor [14 TeV] CHECK THIS
tt_init_sim_eve = 200000 * tt_runs

cross       = tt_cross * 1000 * tt_NLO
aceptancia  = tt_cut_ev / (1. * tt_init_sim_eve)
luminosidad = 100

tt_fidcross = cross * aceptancia
tt_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', tt_fidcross)
print('Events expected: ', tt_expected)

('fiducial cross section: ', 98.72695)
('Events expected: ', 9872.695)


### [x] tZj 
- [x] Done
    - Se esperan 24.33216 con MET > 100
    - Se esperan 5.27022 con MET > 200

In [10]:
Folder    = '../data/wagner-bkgs-varios/'
datFolder = '../data/wagner-bkgs-varios/'

tzj_runs  = 1

tzj_init_ev = 0
tzj_cut_ev  = 0

tzj_photon  = []
tzj_jet     = []
tzj_lepton  = []
tzj_MET     = []
tzj_hadinfo = []


counter_bad_jets = 0
contador_tau = 0
contador_btag = 0
for ii in range(1, 1 + tzj_runs):
    print(ii)
    if ii not in [0]:
        inputevents = LHCO_reader.Events(f_name = Folder + 'tZj-tests.lhco')

        tzj_init_ev += len(inputevents)
        print('Esta run tuvo ' + str(len(inputevents)) + ' eventos en el lhco')
        
        for i in range(len(inputevents)):

            num_jets = 0
    #        num_btag = 0
            HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets


            # at least one: photon, jet and lepton (electron or muon)
            if inputevents[i].number()["photon"] > 0 and inputevents[i].number()["jet"] > 0 and (inputevents[i].number()["electron"] > 0 or inputevents[i].number()["muon"] > 0):
                if inputevents[i]["jet"][0]["PT"] > 100 and -4.5 < inputevents[i]["jet"][0]["eta"] < 4.5:
                    if inputevents[i]["photon"][0]["PT"] > 10 and -2.37 < inputevents[i]["photon"][0]["eta"] < 2.37:
                        if inputevents[i]["MET"][0]["PT"] > 200:
                            # lepton info
                            if (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] > 0):
                                if inputevents[i]["electron"][0]["PT"] > inputevents[i]["muon"][0]["PT"]: 
                                    flag = 'electron'
                                    eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                                else: 
                                    flag = 'muon'
                                    eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron
                            elif (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] == 0): 
                                flag = 'electron'
                                eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                            elif (inputevents[i].number()["muon"] > 0) and (inputevents[i].number()["electron"] == 0): 
                                flag = 'muon'
                                eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron

                            if (-2.47 < eta_leading_lepton < -1.52) or (-1.37 < eta_leading_lepton < 1.37) or (1.52 < eta_leading_lepton < 2.47): 

                                # how many jets (total) and total hadronic pT
                                flag_bad_jets = 0                            
                                num_jets = 0
                                HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
                                HT = 0 # total HT, defined as the scalar sum of the pT of all reconstructed particles

                                for j in range(inputevents[i].number()["jet"]):
                                    if (inputevents[i]["jet"][j]["PT"] > 20) and (-4.5 < inputevents[i]["jet"][j]["eta"] < 4.5):
                                        num_jets += 1
                                        HT_had += inputevents[i]["jet"][j]["PT"]
                                        HT += inputevents[i]["jet"][j]["PT"]
                                    if (np.abs(inputevents[i]["jet"][j]["eta"]) > 4.5) or (inputevents[i]["jet"][j]["PT"] < 20):
                                        flag_bad_jets = 1

                                # how many photons                         
                                num_photons = 0
                                for j in range(inputevents[i].number()["photon"]):
                                    if inputevents[i]["photon"][j]["PT"] > 10 and -2.37 < inputevents[i]["photon"][j]["eta"] < 2.37:
                                        num_photons = num_photons + 1
                                        HT += inputevents[i]["photon"][j]["PT"]

                                # how many muons                        
                                num_muons = 0
                                for j in range(inputevents[i].number()["muon"]):
                                    if inputevents[i]["muon"][j]["PT"] > 10 and -2.7 < inputevents[i]["muon"][j]["eta"] < 2.7:
                                        num_muons = num_muons + 1
                                        HT += inputevents[i]["muon"][j]["PT"]

                                # how many electrons                        
                                num_electrons = 0
                                for j in range(inputevents[i].number()["electron"]):
                                    if inputevents[i]["electron"][j]["PT"] > 10:
                                        eta = inputevents[i]["electron"][j]["eta"]
                                        if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                            num_electrons = num_electrons + 1
                                            HT += inputevents[i]["electron"][j]["PT"]

                                if flag_bad_jets == 1: counter_bad_jets += 1 
                                if flag_bad_jets == 0:
                                    num_taus = 0
                                    if inputevents[i].number()["tau"] > 0: 
                                        contador_tau = contador_tau + 1
                                        # how many taus                    
                                        for j in range(inputevents[i].number()["tau"]):
                                            if inputevents[i]["tau"][j]["PT"] > 20:
                                                eta = inputevents[i]["tau"][j]["eta"]
                                                if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                                    num_taus = num_taus + 1
                                                    HT += inputevents[i]["tau"][j]["PT"]
                                                    
                                    if inputevents[i]["jet"][0]['btag'] == 1: contador_btag = contador_btag + 1

                                    # photon info:
                                    with open(datFolder + 'photon_bu.dat', 'a') as f:
                                        f.write('%f %f %f %d\n' % (inputevents[i]["photon"][0]["PT"],inputevents[i]["photon"][0]["eta"],inputevents[i]["photon"][0]["phi"], num_photons))

                                    # jet info:
                                    with open(datFolder + 'jet_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["jet"][0]["PT"],inputevents[i]["jet"][0]["eta"],inputevents[i]["jet"][0]["phi"]))

                                    # lepton info
                                    num_leptons = num_electrons + num_muons
                                    if flag == "electron":
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["electron"][0]["PT"],inputevents[i]["electron"][0]["eta"],inputevents[i]["electron"][0]["phi"], num_leptons, num_taus))
                                    else:
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["muon"][0]["PT"],inputevents[i]["muon"][0]["eta"],inputevents[i]["muon"][0]["phi"], num_leptons, num_taus))

                                    # MET (eta=0 by def, but I save phi because I have many jets)
                                    with open(datFolder + 'MET_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))

                                    # hadronic info
                                    with open(datFolder + 'hadinfo_bu.dat', 'a') as f:
                                        f.write('%f %f %d\n' % (HT, HT_had, num_jets))

                                    # total events that passed the cuts
                                    tzj_cut_ev += 1

print('Num initial events: ', tzj_init_ev)
print('Total pasaron los cortes: ', tzj_cut_ev)
print('Hay: ', contador_tau, ' taus')
print('Hay: ', counter_bad_jets, ' eventos con al menos un jet malo')
print('Hay: ', contador_btag, ' con btag en leading jet')

1
Esta run tuvo 200000 eventos en el lhco
('Num initial events: ', 200000)
('Total pasaron los cortes: ', 201)
('Hay: ', 11, ' taus')
('Hay: ', 3, ' eventos con al menos un jet malo')
('Hay: ', 31, ' con btag en leading jet')


In [11]:
tzj_cross = 0.05244 #[pb] integrated cross-section without matching
tzj_NLO   = 1. # k-factor [14 TeV] CHECK THIS
tzj_init_sim_eve = 200000 * tzj_runs

cross       = tzj_cross * 1000 * tzj_NLO
aceptancia  = tzj_cut_ev / (1. * tzj_init_sim_eve)
luminosidad = 100

tzj_fidcross = cross * aceptancia
tzj_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', tzj_fidcross)
print('Events expected: ', tzj_expected)

('fiducial cross section: ', 0.0527022)
('Events expected: ', 5.27022)


### [x] ZH
- [x] Done. Se esperan 0.8 eventos con MET > 100

In [13]:
Folder    = '../data/wagner-bkgs-varios/'
datFolder = '../data/wagner-bkgs-varios/'

ZH_runs  = 1

ZH_init_ev = 0
ZH_cut_ev  = 0

ZH_photon  = []
ZH_jet     = []
ZH_lepton  = []
ZH_MET     = []
ZH_hadinfo = []


counter_bad_jets = 0
contador_tau = 0
contador_btag = 0
for ii in range(1, 1 + ZH_runs):
    print(ii)
    if ii not in [0]:
        inputevents = LHCO_reader.Events(f_name = Folder + 'ZH-tests.lhco')

        ZH_init_ev += len(inputevents)
        print('Esta run tuvo ' + str(len(inputevents)) + ' eventos en el lhco')
        
        for i in range(len(inputevents)):

            num_jets = 0
    #        num_btag = 0
            HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets


            # at least one: photon, jet and lepton (electron or muon)
            if inputevents[i].number()["photon"] > 0 and inputevents[i].number()["jet"] > 0 and (inputevents[i].number()["electron"] > 0 or inputevents[i].number()["muon"] > 0):
                if inputevents[i]["jet"][0]["PT"] > 100 and -4.5 < inputevents[i]["jet"][0]["eta"] < 4.5:
                    if inputevents[i]["photon"][0]["PT"] > 10 and -2.37 < inputevents[i]["photon"][0]["eta"] < 2.37:
                        if inputevents[i]["MET"][0]["PT"] > 100:
                            # lepton info
                            if (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] > 0):
                                if inputevents[i]["electron"][0]["PT"] > inputevents[i]["muon"][0]["PT"]: 
                                    flag = 'electron'
                                    eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                                else: 
                                    flag = 'muon'
                                    eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron
                            elif (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] == 0): 
                                flag = 'electron'
                                eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                            elif (inputevents[i].number()["muon"] > 0) and (inputevents[i].number()["electron"] == 0): 
                                flag = 'muon'
                                eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron

                            if (-2.47 < eta_leading_lepton < -1.52) or (-1.37 < eta_leading_lepton < 1.37) or (1.52 < eta_leading_lepton < 2.47): 

                                # how many jets (total) and total hadronic pT
                                flag_bad_jets = 0                            
                                num_jets = 0
                                HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
                                HT = 0 # total HT, defined as the scalar sum of the pT of all reconstructed particles

                                for j in range(inputevents[i].number()["jet"]):
                                    if (inputevents[i]["jet"][j]["PT"] > 20) and (-4.5 < inputevents[i]["jet"][j]["eta"] < 4.5):
                                        num_jets += 1
                                        HT_had += inputevents[i]["jet"][j]["PT"]
                                        HT += inputevents[i]["jet"][j]["PT"]
                                    if (np.abs(inputevents[i]["jet"][j]["eta"]) > 4.5) or (inputevents[i]["jet"][j]["PT"] < 20):
                                        flag_bad_jets = 1

                                # how many photons                         
                                num_photons = 0
                                for j in range(inputevents[i].number()["photon"]):
                                    if inputevents[i]["photon"][j]["PT"] > 10 and -2.37 < inputevents[i]["photon"][j]["eta"] < 2.37:
                                        num_photons = num_photons + 1
                                        HT += inputevents[i]["photon"][j]["PT"]

                                # how many muons                        
                                num_muons = 0
                                for j in range(inputevents[i].number()["muon"]):
                                    if inputevents[i]["muon"][j]["PT"] > 10 and -2.7 < inputevents[i]["muon"][j]["eta"] < 2.7:
                                        num_muons = num_muons + 1
                                        HT += inputevents[i]["muon"][j]["PT"]

                                # how many electrons                        
                                num_electrons = 0
                                for j in range(inputevents[i].number()["electron"]):
                                    if inputevents[i]["electron"][j]["PT"] > 10:
                                        eta = inputevents[i]["electron"][j]["eta"]
                                        if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                            num_electrons = num_electrons + 1
                                            HT += inputevents[i]["electron"][j]["PT"]

                                if flag_bad_jets == 1: counter_bad_jets += 1 
                                if flag_bad_jets == 0:
                                    num_taus = 0
                                    if inputevents[i].number()["tau"] > 0: 
                                        contador_tau = contador_tau + 1
                                        # how many taus                    
                                        for j in range(inputevents[i].number()["tau"]):
                                            if inputevents[i]["tau"][j]["PT"] > 20:
                                                eta = inputevents[i]["tau"][j]["eta"]
                                                if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                                    num_taus = num_taus + 1
                                                    HT += inputevents[i]["tau"][j]["PT"]
                                                    
                                    if inputevents[i]["jet"][0]['btag'] == 1: contador_btag = contador_btag + 1

                                    # photon info:
                                    with open(datFolder + 'photon_bu.dat', 'a') as f:
                                        f.write('%f %f %f %d\n' % (inputevents[i]["photon"][0]["PT"],inputevents[i]["photon"][0]["eta"],inputevents[i]["photon"][0]["phi"], num_photons))

                                    # jet info:
                                    with open(datFolder + 'jet_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["jet"][0]["PT"],inputevents[i]["jet"][0]["eta"],inputevents[i]["jet"][0]["phi"]))

                                    # lepton info
                                    num_leptons = num_electrons + num_muons
                                    if flag == "electron":
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["electron"][0]["PT"],inputevents[i]["electron"][0]["eta"],inputevents[i]["electron"][0]["phi"], num_leptons, num_taus))
                                    else:
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["muon"][0]["PT"],inputevents[i]["muon"][0]["eta"],inputevents[i]["muon"][0]["phi"], num_leptons, num_taus))

                                    # MET (eta=0 by def, but I save phi because I have many jets)
                                    with open(datFolder + 'MET_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))

                                    # hadronic info
                                    with open(datFolder + 'hadinfo_bu.dat', 'a') as f:
                                        f.write('%f %f %d\n' % (HT, HT_had, num_jets))

                                    # total events that passed the cuts
                                    ZH_cut_ev += 1

print('Num initial events: ', ZH_init_ev)
print('Total pasaron los cortes: ', ZH_cut_ev)
print('Hay: ', contador_tau, ' taus')
print('Hay: ', counter_bad_jets, ' eventos con al menos un jet malo')
print('Hay: ', contador_btag, ' con btag en leading jet')

1
Esta run tuvo 134329 eventos en el lhco
('Num initial events: ', 134329)
('Total pasaron los cortes: ', 13)
('Hay: ', 0, ' taus')
('Hay: ', 0, ' eventos con al menos un jet malo')
('Hay: ', 8, ' con btag en leading jet')


In [14]:
ZH_cross = 0.124 #[pb] integrated cross-section without matching
ZH_NLO   = 1. # k-factor [14 TeV] CHECK THIS
ZH_init_sim_eve = 200000 * ZH_runs

cross       = ZH_cross * 1000 * ZH_NLO
aceptancia  = ZH_cut_ev / (1. * ZH_init_sim_eve)
luminosidad = 100

ZH_fidcross = cross * aceptancia
ZH_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', ZH_fidcross)
print('Events expected: ', ZH_expected)

('fiducial cross section: ', 0.00806)
('Events expected: ', 0.8059999999999999)


### [x] WH
- [x] Done. Se esperan 2.7 eventos con MET > 100

In [10]:
Folder    = '../data/wagner-bkgs-varios/'
datFolder = '../data/wagner-bkgs-varios/'

WH_runs  = 1

WH_init_ev = 0
WH_cut_ev  = 0

WH_photon  = []
WH_jet     = []
WH_lepton  = []
WH_MET     = []
WH_hadinfo = []


counter_bad_jets = 0
contador_tau = 0
contador_btag = 0
for ii in range(1, 1 + WH_runs):
    print(ii)
    if ii not in [0]:
        inputevents = LHCO_reader.Events(f_name = Folder + 'WH-tests.lhco')

        WH_init_ev += len(inputevents)
        print('Esta run tuvo ' + str(len(inputevents)) + ' eventos en el lhco')
        
        for i in range(len(inputevents)):

            num_jets = 0
    #        num_btag = 0
            HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets


            # at least one: photon, jet and lepton (electron or muon)
            if inputevents[i].number()["photon"] > 0 and inputevents[i].number()["jet"] > 0 and (inputevents[i].number()["electron"] > 0 or inputevents[i].number()["muon"] > 0):
                if inputevents[i]["jet"][0]["PT"] > 100 and -4.5 < inputevents[i]["jet"][0]["eta"] < 4.5:
                    if inputevents[i]["photon"][0]["PT"] > 10 and -2.37 < inputevents[i]["photon"][0]["eta"] < 2.37:
                        if inputevents[i]["MET"][0]["PT"] > 100:
                            # lepton info
                            if (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] > 0):
                                if inputevents[i]["electron"][0]["PT"] > inputevents[i]["muon"][0]["PT"]: 
                                    flag = 'electron'
                                    eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                                else: 
                                    flag = 'muon'
                                    eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron
                            elif (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] == 0): 
                                flag = 'electron'
                                eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                            elif (inputevents[i].number()["muon"] > 0) and (inputevents[i].number()["electron"] == 0): 
                                flag = 'muon'
                                eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron

                            if (-2.47 < eta_leading_lepton < -1.52) or (-1.37 < eta_leading_lepton < 1.37) or (1.52 < eta_leading_lepton < 2.47): 

                                # how many jets (total) and total hadronic pT
                                flag_bad_jets = 0                            
                                num_jets = 0
                                HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
                                HT = 0 # total HT, defined as the scalar sum of the pT of all reconstructed particles

                                for j in range(inputevents[i].number()["jet"]):
                                    if (inputevents[i]["jet"][j]["PT"] > 20) and (-4.5 < inputevents[i]["jet"][j]["eta"] < 4.5):
                                        num_jets += 1
                                        HT_had += inputevents[i]["jet"][j]["PT"]
                                        HT += inputevents[i]["jet"][j]["PT"]
                                    if (np.abs(inputevents[i]["jet"][j]["eta"]) > 4.5) or (inputevents[i]["jet"][j]["PT"] < 20):
                                        flag_bad_jets = 1

                                # how many photons                         
                                num_photons = 0
                                for j in range(inputevents[i].number()["photon"]):
                                    if inputevents[i]["photon"][j]["PT"] > 10 and -2.37 < inputevents[i]["photon"][j]["eta"] < 2.37:
                                        num_photons = num_photons + 1
                                        HT += inputevents[i]["photon"][j]["PT"]

                                # how many muons                        
                                num_muons = 0
                                for j in range(inputevents[i].number()["muon"]):
                                    if inputevents[i]["muon"][j]["PT"] > 10 and -2.7 < inputevents[i]["muon"][j]["eta"] < 2.7:
                                        num_muons = num_muons + 1
                                        HT += inputevents[i]["muon"][j]["PT"]

                                # how many electrons                        
                                num_electrons = 0
                                for j in range(inputevents[i].number()["electron"]):
                                    if inputevents[i]["electron"][j]["PT"] > 10:
                                        eta = inputevents[i]["electron"][j]["eta"]
                                        if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                            num_electrons = num_electrons + 1
                                            HT += inputevents[i]["electron"][j]["PT"]

                                if flag_bad_jets == 1: counter_bad_jets += 1 
                                if flag_bad_jets == 0:
                                    num_taus = 0
                                    if inputevents[i].number()["tau"] > 0: 
                                        contador_tau = contador_tau + 1
                                        # how many taus                    
                                        for j in range(inputevents[i].number()["tau"]):
                                            if inputevents[i]["tau"][j]["PT"] > 20:
                                                eta = inputevents[i]["tau"][j]["eta"]
                                                if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                                    num_taus = num_taus + 1
                                                    HT += inputevents[i]["tau"][j]["PT"]
                                                    
                                    if inputevents[i]["jet"][0]['btag'] == 1: contador_btag = contador_btag + 1

                                    # photon info:
                                    with open(datFolder + 'photon_bu.dat', 'a') as f:
                                        f.write('%f %f %f %d\n' % (inputevents[i]["photon"][0]["PT"],inputevents[i]["photon"][0]["eta"],inputevents[i]["photon"][0]["phi"], num_photons))

                                    # jet info:
                                    with open(datFolder + 'jet_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["jet"][0]["PT"],inputevents[i]["jet"][0]["eta"],inputevents[i]["jet"][0]["phi"]))

                                    # lepton info
                                    num_leptons = num_electrons + num_muons
                                    if flag == "electron":
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["electron"][0]["PT"],inputevents[i]["electron"][0]["eta"],inputevents[i]["electron"][0]["phi"], num_leptons, num_taus))
                                    else:
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["muon"][0]["PT"],inputevents[i]["muon"][0]["eta"],inputevents[i]["muon"][0]["phi"], num_leptons, num_taus))

                                    # MET (eta=0 by def, but I save phi because I have many jets)
                                    with open(datFolder + 'MET_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))

                                    # hadronic info
                                    with open(datFolder + 'hadinfo_bu.dat', 'a') as f:
                                        f.write('%f %f %d\n' % (HT, HT_had, num_jets))

                                    # total events that passed the cuts
                                    WH_cut_ev += 1

print('Num initial events: ', WH_init_ev)
print('Total pasaron los cortes: ', WH_cut_ev)
print('Hay: ', contador_tau, ' taus')
print('Hay: ', counter_bad_jets, ' eventos con al menos un jet malo')
print('Hay: ', contador_btag, ' con btag en leading jet')

1
Esta run tuvo 136824 eventos en el lhco
('Num initial events: ', 136824)
('Total pasaron los cortes: ', 44)
('Hay: ', 0, ' taus')
('Hay: ', 0, ' eventos con al menos un jet malo')
('Hay: ', 9, ' con btag en leading jet')


In [12]:
WH_cross = 0.1247 #[pb] integrated cross-section without matching
WH_NLO   = 1. # k-factor [14 TeV] CHECK THIS
WH_init_sim_eve = 200000 * WH_runs

cross       = WH_cross * 1000 * WH_NLO
aceptancia  = WH_cut_ev / (1. * WH_init_sim_eve)
luminosidad = 100

WH_fidcross = cross * aceptancia
WH_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', WH_fidcross)
print('Events expected: ', WH_expected)

('fiducial cross section: ', 0.027434)
('Events expected: ', 2.7434)


### [x] tH
- [x] Done. Se esperan 0.23 eventpos con MET > 100

In [8]:
Folder    = '../data/wagner-bkgs-varios/'
datFolder = '../data/wagner-bkgs-varios/'

tH_runs  = 1

tH_init_ev = 0
tH_cut_ev  = 0

tH_photon  = []
tH_jet     = []
tH_lepton  = []
tH_MET     = []
tH_hadinfo = []


counter_bad_jets = 0
contador_tau = 0
contador_btag = 0
for ii in range(1, 1 + tH_runs):
    print(ii)
    if ii not in [0]:
        inputevents = LHCO_reader.Events(f_name = Folder + 'tH-tests.lhco')

        tH_init_ev += len(inputevents)
        print('Esta run tuvo ' + str(len(inputevents)) + ' eventos en el lhco')
        
        for i in range(len(inputevents)):

            num_jets = 0
    #        num_btag = 0
            HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets


            # at least one: photon, jet and lepton (electron or muon)
            if inputevents[i].number()["photon"] > 0 and inputevents[i].number()["jet"] > 0 and (inputevents[i].number()["electron"] > 0 or inputevents[i].number()["muon"] > 0):
                if inputevents[i]["jet"][0]["PT"] > 100 and -4.5 < inputevents[i]["jet"][0]["eta"] < 4.5:
                    if inputevents[i]["photon"][0]["PT"] > 10 and -2.37 < inputevents[i]["photon"][0]["eta"] < 2.37:
                        if inputevents[i]["MET"][0]["PT"] > 100:
                            # lepton info
                            if (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] > 0):
                                if inputevents[i]["electron"][0]["PT"] > inputevents[i]["muon"][0]["PT"]: 
                                    flag = 'electron'
                                    eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                                else: 
                                    flag = 'muon'
                                    eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron
                            elif (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] == 0): 
                                flag = 'electron'
                                eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                            elif (inputevents[i].number()["muon"] > 0) and (inputevents[i].number()["electron"] == 0): 
                                flag = 'muon'
                                eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron

                            if (-2.47 < eta_leading_lepton < -1.52) or (-1.37 < eta_leading_lepton < 1.37) or (1.52 < eta_leading_lepton < 2.47): 

                                # how many jets (total) and total hadronic pT
                                flag_bad_jets = 0                            
                                num_jets = 0
                                HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
                                HT = 0 # total HT, defined as the scalar sum of the pT of all reconstructed particles

                                for j in range(inputevents[i].number()["jet"]):
                                    if (inputevents[i]["jet"][j]["PT"] > 20) and (-4.5 < inputevents[i]["jet"][j]["eta"] < 4.5):
                                        num_jets += 1
                                        HT_had += inputevents[i]["jet"][j]["PT"]
                                        HT += inputevents[i]["jet"][j]["PT"]
                                    if (np.abs(inputevents[i]["jet"][j]["eta"]) > 4.5) or (inputevents[i]["jet"][j]["PT"] < 20):
                                        flag_bad_jets = 1

                                # how many photons                         
                                num_photons = 0
                                for j in range(inputevents[i].number()["photon"]):
                                    if inputevents[i]["photon"][j]["PT"] > 10 and -2.37 < inputevents[i]["photon"][j]["eta"] < 2.37:
                                        num_photons = num_photons + 1
                                        HT += inputevents[i]["photon"][j]["PT"]

                                # how many muons                        
                                num_muons = 0
                                for j in range(inputevents[i].number()["muon"]):
                                    if inputevents[i]["muon"][j]["PT"] > 10 and -2.7 < inputevents[i]["muon"][j]["eta"] < 2.7:
                                        num_muons = num_muons + 1
                                        HT += inputevents[i]["muon"][j]["PT"]

                                # how many electrons                        
                                num_electrons = 0
                                for j in range(inputevents[i].number()["electron"]):
                                    if inputevents[i]["electron"][j]["PT"] > 10:
                                        eta = inputevents[i]["electron"][j]["eta"]
                                        if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                            num_electrons = num_electrons + 1
                                            HT += inputevents[i]["electron"][j]["PT"]

                                if flag_bad_jets == 1: counter_bad_jets += 1 
                                if flag_bad_jets == 0:
                                    num_taus = 0
                                    if inputevents[i].number()["tau"] > 0: 
                                        contador_tau = contador_tau + 1
                                        # how many taus                    
                                        for j in range(inputevents[i].number()["tau"]):
                                            if inputevents[i]["tau"][j]["PT"] > 20:
                                                eta = inputevents[i]["tau"][j]["eta"]
                                                if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                                    num_taus = num_taus + 1
                                                    HT += inputevents[i]["tau"][j]["PT"]
                                                    
                                    if inputevents[i]["jet"][0]['btag'] == 1: contador_btag = contador_btag + 1

                                    # photon info:
                                    with open(datFolder + 'photon_bu.dat', 'a') as f:
                                        f.write('%f %f %f %d\n' % (inputevents[i]["photon"][0]["PT"],inputevents[i]["photon"][0]["eta"],inputevents[i]["photon"][0]["phi"], num_photons))

                                    # jet info:
                                    with open(datFolder + 'jet_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["jet"][0]["PT"],inputevents[i]["jet"][0]["eta"],inputevents[i]["jet"][0]["phi"]))

                                    # lepton info
                                    num_leptons = num_electrons + num_muons
                                    if flag == "electron":
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["electron"][0]["PT"],inputevents[i]["electron"][0]["eta"],inputevents[i]["electron"][0]["phi"], num_leptons, num_taus))
                                    else:
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["muon"][0]["PT"],inputevents[i]["muon"][0]["eta"],inputevents[i]["muon"][0]["phi"], num_leptons, num_taus))

                                    # MET (eta=0 by def, but I save phi because I have many jets)
                                    with open(datFolder + 'MET_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))

                                    # hadronic info
                                    with open(datFolder + 'hadinfo_bu.dat', 'a') as f:
                                        f.write('%f %f %d\n' % (HT, HT_had, num_jets))

                                    # total events that passed the cuts
                                    tH_cut_ev += 1

print('Num initial events: ', tH_init_ev)
print('Total pasaron los cortes: ', tH_cut_ev)
print('Hay: ', contador_tau, ' taus')
print('Hay: ', counter_bad_jets, ' eventos con al menos un jet malo')
print('Hay: ', contador_btag, ' con btag en leading jet')

1
Esta run tuvo 200000 eventos en el lhco
('Num initial events: ', 200000)
('Total pasaron los cortes: ', 53)
('Hay: ', 2, ' taus')
('Hay: ', 0, ' eventos con al menos un jet malo')
('Hay: ', 14, ' con btag en leading jet')


In [9]:
tH_cross = 0.008851 #[pb] integrated cross-section without matching
tH_NLO   = 1. # k-factor [14 TeV] CHECK THIS
tH_init_sim_eve = 200000 * tH_runs

cross       = tH_cross * 1000 * tH_NLO
aceptancia  = tH_cut_ev / (1. * tH_init_sim_eve)
luminosidad = 100

tH_fidcross = cross * aceptancia
tH_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', tH_fidcross)
print('Events expected: ', tH_expected)

('fiducial cross section: ', 0.002345515)
('Events expected: ', 0.2345515)


### ttj
- [x] Done. Da distinto a lo q daba antes

#### ttj full
- [x] Done
    - Se esperan 10037.15 con MET > 100
    - Se esperan 1069.95 con MET > 200

In [16]:
Folder    = '../data/wagner-bkgs-varios/'
datFolder = '../data/wagner-bkgs-varios/'

ttj_runs  = 1

ttj_init_ev = 0
ttj_cut_ev  = 0

ttj_photon  = []
ttj_jet     = []
ttj_lepton  = []
ttj_MET     = []
ttj_hadinfo = []


counter_bad_jets = 0
contador_tau = 0
contador_btag = 0
for ii in range(1, 1 + ttj_runs):
    print(ii)
    if ii not in [0]:
        inputevents = LHCO_reader.Events(f_name = Folder + 'ttj-full-tests.lhco')

        ttj_init_ev += len(inputevents)
        print('Esta run tuvo ' + str(len(inputevents)) + ' eventos en el lhco')
        
        for i in range(len(inputevents)):

            num_jets = 0
    #        num_btag = 0
            HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets


            # at least one: photon, jet and lepton (electron or muon)
            if inputevents[i].number()["photon"] > 0 and inputevents[i].number()["jet"] > 0 and (inputevents[i].number()["electron"] > 0 or inputevents[i].number()["muon"] > 0):
                if inputevents[i]["jet"][0]["PT"] > 100 and -4.5 < inputevents[i]["jet"][0]["eta"] < 4.5:
                    if inputevents[i]["photon"][0]["PT"] > 10 and -2.37 < inputevents[i]["photon"][0]["eta"] < 2.37:
                        if inputevents[i]["MET"][0]["PT"] > 200:
                            # lepton info
                            if (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] > 0):
                                if inputevents[i]["electron"][0]["PT"] > inputevents[i]["muon"][0]["PT"]: 
                                    flag = 'electron'
                                    eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                                else: 
                                    flag = 'muon'
                                    eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron
                            elif (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] == 0): 
                                flag = 'electron'
                                eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                            elif (inputevents[i].number()["muon"] > 0) and (inputevents[i].number()["electron"] == 0): 
                                flag = 'muon'
                                eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron

                            if (-2.47 < eta_leading_lepton < -1.52) or (-1.37 < eta_leading_lepton < 1.37) or (1.52 < eta_leading_lepton < 2.47): 

                                # how many jets (total) and total hadronic pT
                                flag_bad_jets = 0                            
                                num_jets = 0
                                HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
                                HT = 0 # total HT, defined as the scalar sum of the pT of all reconstructed particles

                                for j in range(inputevents[i].number()["jet"]):
                                    if (inputevents[i]["jet"][j]["PT"] > 20) and (-4.5 < inputevents[i]["jet"][j]["eta"] < 4.5):
                                        num_jets += 1
                                        HT_had += inputevents[i]["jet"][j]["PT"]
                                        HT += inputevents[i]["jet"][j]["PT"]
                                    if (np.abs(inputevents[i]["jet"][j]["eta"]) > 4.5) or (inputevents[i]["jet"][j]["PT"] < 20):
                                        flag_bad_jets = 1

                                # how many photons                         
                                num_photons = 0
                                for j in range(inputevents[i].number()["photon"]):
                                    if inputevents[i]["photon"][j]["PT"] > 10 and -2.37 < inputevents[i]["photon"][j]["eta"] < 2.37:
                                        num_photons = num_photons + 1
                                        HT += inputevents[i]["photon"][j]["PT"]

                                # how many muons                        
                                num_muons = 0
                                for j in range(inputevents[i].number()["muon"]):
                                    if inputevents[i]["muon"][j]["PT"] > 10 and -2.7 < inputevents[i]["muon"][j]["eta"] < 2.7:
                                        num_muons = num_muons + 1
                                        HT += inputevents[i]["muon"][j]["PT"]

                                # how many electrons                        
                                num_electrons = 0
                                for j in range(inputevents[i].number()["electron"]):
                                    if inputevents[i]["electron"][j]["PT"] > 10:
                                        eta = inputevents[i]["electron"][j]["eta"]
                                        if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                            num_electrons = num_electrons + 1
                                            HT += inputevents[i]["electron"][j]["PT"]

                                if flag_bad_jets == 1: counter_bad_jets += 1 
                                if flag_bad_jets == 0:
                                    num_taus = 0
                                    if inputevents[i].number()["tau"] > 0: 
                                        contador_tau = contador_tau + 1
                                        # how many taus                    
                                        for j in range(inputevents[i].number()["tau"]):
                                            if inputevents[i]["tau"][j]["PT"] > 20:
                                                eta = inputevents[i]["tau"][j]["eta"]
                                                if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                                    num_taus = num_taus + 1
                                                    HT += inputevents[i]["tau"][j]["PT"]
                                                    
                                    if inputevents[i]["jet"][0]['btag'] == 1: contador_btag = contador_btag + 1

                                    # photon info:
                                    with open(datFolder + 'photon_bu.dat', 'a') as f:
                                        f.write('%f %f %f %d\n' % (inputevents[i]["photon"][0]["PT"],inputevents[i]["photon"][0]["eta"],inputevents[i]["photon"][0]["phi"], num_photons))

                                    # jet info:
                                    with open(datFolder + 'jet_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["jet"][0]["PT"],inputevents[i]["jet"][0]["eta"],inputevents[i]["jet"][0]["phi"]))

                                    # lepton info
                                    num_leptons = num_electrons + num_muons
                                    if flag == "electron":
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["electron"][0]["PT"],inputevents[i]["electron"][0]["eta"],inputevents[i]["electron"][0]["phi"], num_leptons, num_taus))
                                    else:
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["muon"][0]["PT"],inputevents[i]["muon"][0]["eta"],inputevents[i]["muon"][0]["phi"], num_leptons, num_taus))

                                    # MET (eta=0 by def, but I save phi because I have many jets)
                                    with open(datFolder + 'MET_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))

                                    # hadronic info
                                    with open(datFolder + 'hadinfo_bu.dat', 'a') as f:
                                        f.write('%f %f %d\n' % (HT, HT_had, num_jets))

                                    # total events that passed the cuts
                                    ttj_cut_ev += 1

print('Num initial events: ', ttj_init_ev)
print('Total pasaron los cortes: ', ttj_cut_ev)
print('Hay: ', contador_tau, ' taus')
print('Hay: ', counter_bad_jets, ' eventos con al menos un jet malo')
print('Hay: ', contador_btag, ' con btag en leading jet')

1
Esta run tuvo 61713 eventos en el lhco
('Num initial events: ', 61713)
('Total pasaron los cortes: ', 21)
('Hay: ', 1, ' taus')
('Hay: ', 1, ' eventos con al menos un jet malo')
('Hay: ', 8, ' con btag en leading jet')


In [17]:
ttj_cross = 101.9 #[pb] integrated cross-section without matching
ttj_NLO   = 1.47 # k-factor [14 TeV] CHECK THIS
ttj_init_sim_eve = 200000 * ttj_runs

cross       = ttj_cross * 1000 * ttj_NLO
aceptancia  = ttj_cut_ev / (1. * ttj_init_sim_eve)
luminosidad = 100

ttj_fidcross = cross * aceptancia
ttj_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', ttj_fidcross)
print('Events expected: ', ttj_expected)

('fiducial cross section: ', 10.6995)
('Events expected: ', 1069.95)


#### ttj extra jet
- [x] Done
    - se esperan 11390.32 con MET > 100
    - se esperan 2020.6399999999999 con MET > 200

In [12]:
Folder    = '../data/wagner-bkgs-varios/'
datFolder = '../data/wagner-bkgs-varios/'

ttj_runs  = 1

ttj_init_ev = 0
ttj_cut_ev  = 0

ttj_photon  = []
ttj_jet     = []
ttj_lepton  = []
ttj_MET     = []
ttj_hadinfo = []


counter_bad_jets = 0
contador_tau = 0
contador_btag = 0
for ii in range(1, 1 + ttj_runs):
    print(ii)
    if ii not in [0]:
        inputevents = LHCO_reader.Events(f_name = Folder + 'ttj-NOb-NOjj-extraj-tests.lhco')

        ttj_init_ev += len(inputevents)
        print('Esta run tuvo ' + str(len(inputevents)) + ' eventos en el lhco')
        
        for i in range(len(inputevents)):

            num_jets = 0
    #        num_btag = 0
            HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets


            # at least one: photon, jet and lepton (electron or muon)
            if inputevents[i].number()["photon"] > 0 and inputevents[i].number()["jet"] > 0 and (inputevents[i].number()["electron"] > 0 or inputevents[i].number()["muon"] > 0):
                if inputevents[i]["jet"][0]["PT"] > 100 and -4.5 < inputevents[i]["jet"][0]["eta"] < 4.5:
                    if inputevents[i]["photon"][0]["PT"] > 10 and -2.37 < inputevents[i]["photon"][0]["eta"] < 2.37:
                        if inputevents[i]["MET"][0]["PT"] > 100:
                            # lepton info
                            if (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] > 0):
                                if inputevents[i]["electron"][0]["PT"] > inputevents[i]["muon"][0]["PT"]: 
                                    flag = 'electron'
                                    eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                                else: 
                                    flag = 'muon'
                                    eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron
                            elif (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] == 0): 
                                flag = 'electron'
                                eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                            elif (inputevents[i].number()["muon"] > 0) and (inputevents[i].number()["electron"] == 0): 
                                flag = 'muon'
                                eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron

                            if (-2.47 < eta_leading_lepton < -1.52) or (-1.37 < eta_leading_lepton < 1.37) or (1.52 < eta_leading_lepton < 2.47): 

                                # how many jets (total) and total hadronic pT
                                flag_bad_jets = 0                            
                                num_jets = 0
                                HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
                                HT = 0 # total HT, defined as the scalar sum of the pT of all reconstructed particles

                                for j in range(inputevents[i].number()["jet"]):
                                    if (inputevents[i]["jet"][j]["PT"] > 20) and (-4.5 < inputevents[i]["jet"][j]["eta"] < 4.5):
                                        num_jets += 1
                                        HT_had += inputevents[i]["jet"][j]["PT"]
                                        HT += inputevents[i]["jet"][j]["PT"]
                                    if (np.abs(inputevents[i]["jet"][j]["eta"]) > 4.5) or (inputevents[i]["jet"][j]["PT"] < 20):
                                        flag_bad_jets = 1

                                # how many photons                         
                                num_photons = 0
                                for j in range(inputevents[i].number()["photon"]):
                                    if inputevents[i]["photon"][j]["PT"] > 10 and -2.37 < inputevents[i]["photon"][j]["eta"] < 2.37:
                                        num_photons = num_photons + 1
                                        HT += inputevents[i]["photon"][j]["PT"]

                                # how many muons                        
                                num_muons = 0
                                for j in range(inputevents[i].number()["muon"]):
                                    if inputevents[i]["muon"][j]["PT"] > 10 and -2.7 < inputevents[i]["muon"][j]["eta"] < 2.7:
                                        num_muons = num_muons + 1
                                        HT += inputevents[i]["muon"][j]["PT"]

                                # how many electrons                        
                                num_electrons = 0
                                for j in range(inputevents[i].number()["electron"]):
                                    if inputevents[i]["electron"][j]["PT"] > 10:
                                        eta = inputevents[i]["electron"][j]["eta"]
                                        if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                            num_electrons = num_electrons + 1
                                            HT += inputevents[i]["electron"][j]["PT"]

                                if flag_bad_jets == 1: counter_bad_jets += 1 
                                if flag_bad_jets == 0:
                                    num_taus = 0
                                    if inputevents[i].number()["tau"] > 0: 
                                        contador_tau = contador_tau + 1
                                        # how many taus                    
                                        for j in range(inputevents[i].number()["tau"]):
                                            if inputevents[i]["tau"][j]["PT"] > 20:
                                                eta = inputevents[i]["tau"][j]["eta"]
                                                if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                                    num_taus = num_taus + 1
                                                    HT += inputevents[i]["tau"][j]["PT"]
                                                    
                                    if inputevents[i]["jet"][0]['btag'] == 1: contador_btag = contador_btag + 1

                                    # photon info:
                                    with open(datFolder + 'photon_bu.dat', 'a') as f:
                                        f.write('%f %f %f %d\n' % (inputevents[i]["photon"][0]["PT"],inputevents[i]["photon"][0]["eta"],inputevents[i]["photon"][0]["phi"], num_photons))

                                    # jet info:
                                    with open(datFolder + 'jet_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["jet"][0]["PT"],inputevents[i]["jet"][0]["eta"],inputevents[i]["jet"][0]["phi"]))

                                    # lepton info
                                    num_leptons = num_electrons + num_muons
                                    if flag == "electron":
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["electron"][0]["PT"],inputevents[i]["electron"][0]["eta"],inputevents[i]["electron"][0]["phi"], num_leptons, num_taus))
                                    else:
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["muon"][0]["PT"],inputevents[i]["muon"][0]["eta"],inputevents[i]["muon"][0]["phi"], num_leptons, num_taus))

                                    # MET (eta=0 by def, but I save phi because I have many jets)
                                    with open(datFolder + 'MET_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))

                                    # hadronic info
                                    with open(datFolder + 'hadinfo_bu.dat', 'a') as f:
                                        f.write('%f %f %d\n' % (HT, HT_had, num_jets))

                                    # total events that passed the cuts
                                    ttj_cut_ev += 1

print('Num initial events: ', ttj_init_ev)
print('Total pasaron los cortes: ', ttj_cut_ev)
print('Hay: ', contador_tau, ' taus')
print('Hay: ', counter_bad_jets, ' eventos con al menos un jet malo')
print('Hay: ', contador_btag, ' con btag en leading jet')

1
Esta run tuvo 55331 eventos en el lhco
('Num initial events: ', 55331)
('Total pasaron los cortes: ', 823)
('Hay: ', 39, ' taus')
('Hay: ', 15, ' eventos con al menos un jet malo')
('Hay: ', 121, ' con btag en leading jet')


In [13]:
ttj_cross = 27.68 #[pb] integrated cross-section without matching
ttj_NLO   = 1.47 # k-factor [14 TeV] CHECK THIS
ttj_init_sim_eve = 200000 * ttj_runs

cross       = ttj_cross * 1000 * ttj_NLO
aceptancia  = ttj_cut_ev / (1. * ttj_init_sim_eve)
luminosidad = 100

ttj_fidcross = cross * aceptancia
ttj_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', ttj_fidcross)
print('Events expected: ', ttj_expected)

('fiducial cross section: ', 113.9032)
('Events expected: ', 11390.32)


#### ttj sin b bbar sin jj
- [x] Done
    - se esperan 13036.64 con MET > 100
    - se esperan 2208.96 con MET > 200

In [14]:
Folder    = '../data/wagner-bkgs-varios/'
datFolder = '../data/wagner-bkgs-varios/'

ttj_runs  = 1

ttj_init_ev = 0
ttj_cut_ev  = 0

ttj_photon  = []
ttj_jet     = []
ttj_lepton  = []
ttj_MET     = []
ttj_hadinfo = []


counter_bad_jets = 0
contador_tau = 0
contador_btag = 0
for ii in range(1, 1 + ttj_runs):
    print(ii)
    if ii not in [0]:
        inputevents = LHCO_reader.Events(f_name = Folder + 'ttj-NOb-NOjj-tests.lhco')

        ttj_init_ev += len(inputevents)
        print('Esta run tuvo ' + str(len(inputevents)) + ' eventos en el lhco')
        
        for i in range(len(inputevents)):

            num_jets = 0
    #        num_btag = 0
            HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets


            # at least one: photon, jet and lepton (electron or muon)
            if inputevents[i].number()["photon"] > 0 and inputevents[i].number()["jet"] > 0 and (inputevents[i].number()["electron"] > 0 or inputevents[i].number()["muon"] > 0):
                if inputevents[i]["jet"][0]["PT"] > 100 and -4.5 < inputevents[i]["jet"][0]["eta"] < 4.5:
                    if inputevents[i]["photon"][0]["PT"] > 10 and -2.37 < inputevents[i]["photon"][0]["eta"] < 2.37:
                        if inputevents[i]["MET"][0]["PT"] > 200:
                            # lepton info
                            if (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] > 0):
                                if inputevents[i]["electron"][0]["PT"] > inputevents[i]["muon"][0]["PT"]: 
                                    flag = 'electron'
                                    eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                                else: 
                                    flag = 'muon'
                                    eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron
                            elif (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] == 0): 
                                flag = 'electron'
                                eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                            elif (inputevents[i].number()["muon"] > 0) and (inputevents[i].number()["electron"] == 0): 
                                flag = 'muon'
                                eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron

                            if (-2.47 < eta_leading_lepton < -1.52) or (-1.37 < eta_leading_lepton < 1.37) or (1.52 < eta_leading_lepton < 2.47): 

                                # how many jets (total) and total hadronic pT
                                flag_bad_jets = 0                            
                                num_jets = 0
                                HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
                                HT = 0 # total HT, defined as the scalar sum of the pT of all reconstructed particles

                                for j in range(inputevents[i].number()["jet"]):
                                    if (inputevents[i]["jet"][j]["PT"] > 20) and (-4.5 < inputevents[i]["jet"][j]["eta"] < 4.5):
                                        num_jets += 1
                                        HT_had += inputevents[i]["jet"][j]["PT"]
                                        HT += inputevents[i]["jet"][j]["PT"]
                                    if (np.abs(inputevents[i]["jet"][j]["eta"]) > 4.5) or (inputevents[i]["jet"][j]["PT"] < 20):
                                        flag_bad_jets = 1

                                # how many photons                         
                                num_photons = 0
                                for j in range(inputevents[i].number()["photon"]):
                                    if inputevents[i]["photon"][j]["PT"] > 10 and -2.37 < inputevents[i]["photon"][j]["eta"] < 2.37:
                                        num_photons = num_photons + 1
                                        HT += inputevents[i]["photon"][j]["PT"]

                                # how many muons                        
                                num_muons = 0
                                for j in range(inputevents[i].number()["muon"]):
                                    if inputevents[i]["muon"][j]["PT"] > 10 and -2.7 < inputevents[i]["muon"][j]["eta"] < 2.7:
                                        num_muons = num_muons + 1
                                        HT += inputevents[i]["muon"][j]["PT"]

                                # how many electrons                        
                                num_electrons = 0
                                for j in range(inputevents[i].number()["electron"]):
                                    if inputevents[i]["electron"][j]["PT"] > 10:
                                        eta = inputevents[i]["electron"][j]["eta"]
                                        if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                            num_electrons = num_electrons + 1
                                            HT += inputevents[i]["electron"][j]["PT"]

                                if flag_bad_jets == 1: counter_bad_jets += 1 
                                if flag_bad_jets == 0:
                                    num_taus = 0
                                    if inputevents[i].number()["tau"] > 0: 
                                        contador_tau = contador_tau + 1
                                        # how many taus                    
                                        for j in range(inputevents[i].number()["tau"]):
                                            if inputevents[i]["tau"][j]["PT"] > 20:
                                                eta = inputevents[i]["tau"][j]["eta"]
                                                if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                                    num_taus = num_taus + 1
                                                    HT += inputevents[i]["tau"][j]["PT"]
                                                    
                                    if inputevents[i]["jet"][0]['btag'] == 1: contador_btag = contador_btag + 1

                                    # photon info:
                                    with open(datFolder + 'photon_bu.dat', 'a') as f:
                                        f.write('%f %f %f %d\n' % (inputevents[i]["photon"][0]["PT"],inputevents[i]["photon"][0]["eta"],inputevents[i]["photon"][0]["phi"], num_photons))

                                    # jet info:
                                    with open(datFolder + 'jet_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["jet"][0]["PT"],inputevents[i]["jet"][0]["eta"],inputevents[i]["jet"][0]["phi"]))

                                    # lepton info
                                    num_leptons = num_electrons + num_muons
                                    if flag == "electron":
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["electron"][0]["PT"],inputevents[i]["electron"][0]["eta"],inputevents[i]["electron"][0]["phi"], num_leptons, num_taus))
                                    else:
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["muon"][0]["PT"],inputevents[i]["muon"][0]["eta"],inputevents[i]["muon"][0]["phi"], num_leptons, num_taus))

                                    # MET (eta=0 by def, but I save phi because I have many jets)
                                    with open(datFolder + 'MET_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))

                                    # hadronic info
                                    with open(datFolder + 'hadinfo_bu.dat', 'a') as f:
                                        f.write('%f %f %d\n' % (HT, HT_had, num_jets))

                                    # total events that passed the cuts
                                    ttj_cut_ev += 1

print('Num initial events: ', ttj_init_ev)
print('Total pasaron los cortes: ', ttj_cut_ev)
print('Hay: ', contador_tau, ' taus')
print('Hay: ', counter_bad_jets, ' eventos con al menos un jet malo')
print('Hay: ', contador_btag, ' con btag en leading jet')

1
Esta run tuvo 110677 eventos en el lhco
('Num initial events: ', 110677)
('Total pasaron los cortes: ', 234)
('Hay: ', 16, ' taus')
('Hay: ', 8, ' eventos con al menos un jet malo')
('Hay: ', 36, ' con btag en leading jet')


In [15]:
ttj_cross = 18.88 #[pb] integrated cross-section without matching
ttj_NLO   = 1.47 # k-factor [14 TeV] CHECK THIS
ttj_init_sim_eve = 200000 * ttj_runs

cross       = ttj_cross * 1000 * ttj_NLO
aceptancia  = ttj_cut_ev / (1. * ttj_init_sim_eve)
luminosidad = 100

ttj_fidcross = cross * aceptancia
ttj_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', ttj_fidcross)
print('Events expected: ', ttj_expected)

('fiducial cross section: ', 22.0896)
('Events expected: ', 2208.96)


#### ttj sin b bbar
- [x] Done
    - se esperan 14411.279999999999 con MET > 100
    - se esperan 1966.8000000000002 con MET > 200

In [18]:
Folder    = '../data/wagner-bkgs-varios/'
datFolder = '../data/wagner-bkgs-varios/'

ttj_runs  = 1

ttj_init_ev = 0
ttj_cut_ev  = 0

ttj_photon  = []
ttj_jet     = []
ttj_lepton  = []
ttj_MET     = []
ttj_hadinfo = []


counter_bad_jets = 0
contador_tau = 0
contador_btag = 0
for ii in range(1, 1 + ttj_runs):
    print(ii)
    if ii not in [0]:
        inputevents = LHCO_reader.Events(f_name = Folder + 'ttj-tests.lhco')

        ttj_init_ev += len(inputevents)
        print('Esta run tuvo ' + str(len(inputevents)) + ' eventos en el lhco')
        
        for i in range(len(inputevents)):

            num_jets = 0
    #        num_btag = 0
            HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets


            # at least one: photon, jet and lepton (electron or muon)
            if inputevents[i].number()["photon"] > 0 and inputevents[i].number()["jet"] > 0 and (inputevents[i].number()["electron"] > 0 or inputevents[i].number()["muon"] > 0):
                if inputevents[i]["jet"][0]["PT"] > 100 and -4.5 < inputevents[i]["jet"][0]["eta"] < 4.5:
                    if inputevents[i]["photon"][0]["PT"] > 10 and -2.37 < inputevents[i]["photon"][0]["eta"] < 2.37:
                        if inputevents[i]["MET"][0]["PT"] > 200:
                            # lepton info
                            if (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] > 0):
                                if inputevents[i]["electron"][0]["PT"] > inputevents[i]["muon"][0]["PT"]: 
                                    flag = 'electron'
                                    eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                                else: 
                                    flag = 'muon'
                                    eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron
                            elif (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] == 0): 
                                flag = 'electron'
                                eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                            elif (inputevents[i].number()["muon"] > 0) and (inputevents[i].number()["electron"] == 0): 
                                flag = 'muon'
                                eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron

                            if (-2.47 < eta_leading_lepton < -1.52) or (-1.37 < eta_leading_lepton < 1.37) or (1.52 < eta_leading_lepton < 2.47): 

                                # how many jets (total) and total hadronic pT
                                flag_bad_jets = 0                            
                                num_jets = 0
                                HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
                                HT = 0 # total HT, defined as the scalar sum of the pT of all reconstructed particles

                                for j in range(inputevents[i].number()["jet"]):
                                    if (inputevents[i]["jet"][j]["PT"] > 20) and (-4.5 < inputevents[i]["jet"][j]["eta"] < 4.5):
                                        num_jets += 1
                                        HT_had += inputevents[i]["jet"][j]["PT"]
                                        HT += inputevents[i]["jet"][j]["PT"]
                                    if (np.abs(inputevents[i]["jet"][j]["eta"]) > 4.5) or (inputevents[i]["jet"][j]["PT"] < 20):
                                        flag_bad_jets = 1

                                # how many photons                         
                                num_photons = 0
                                for j in range(inputevents[i].number()["photon"]):
                                    if inputevents[i]["photon"][j]["PT"] > 10 and -2.37 < inputevents[i]["photon"][j]["eta"] < 2.37:
                                        num_photons = num_photons + 1
                                        HT += inputevents[i]["photon"][j]["PT"]

                                # how many muons                        
                                num_muons = 0
                                for j in range(inputevents[i].number()["muon"]):
                                    if inputevents[i]["muon"][j]["PT"] > 10 and -2.7 < inputevents[i]["muon"][j]["eta"] < 2.7:
                                        num_muons = num_muons + 1
                                        HT += inputevents[i]["muon"][j]["PT"]

                                # how many electrons                        
                                num_electrons = 0
                                for j in range(inputevents[i].number()["electron"]):
                                    if inputevents[i]["electron"][j]["PT"] > 10:
                                        eta = inputevents[i]["electron"][j]["eta"]
                                        if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                            num_electrons = num_electrons + 1
                                            HT += inputevents[i]["electron"][j]["PT"]

                                if flag_bad_jets == 1: counter_bad_jets += 1 
                                if flag_bad_jets == 0:
                                    num_taus = 0
                                    if inputevents[i].number()["tau"] > 0: 
                                        contador_tau = contador_tau + 1
                                        # how many taus                    
                                        for j in range(inputevents[i].number()["tau"]):
                                            if inputevents[i]["tau"][j]["PT"] > 20:
                                                eta = inputevents[i]["tau"][j]["eta"]
                                                if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                                    num_taus = num_taus + 1
                                                    HT += inputevents[i]["tau"][j]["PT"]
                                                    
                                    if inputevents[i]["jet"][0]['btag'] == 1: contador_btag = contador_btag + 1

                                    # photon info:
                                    with open(datFolder + 'photon_bu.dat', 'a') as f:
                                        f.write('%f %f %f %d\n' % (inputevents[i]["photon"][0]["PT"],inputevents[i]["photon"][0]["eta"],inputevents[i]["photon"][0]["phi"], num_photons))

                                    # jet info:
                                    with open(datFolder + 'jet_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["jet"][0]["PT"],inputevents[i]["jet"][0]["eta"],inputevents[i]["jet"][0]["phi"]))

                                    # lepton info
                                    num_leptons = num_electrons + num_muons
                                    if flag == "electron":
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["electron"][0]["PT"],inputevents[i]["electron"][0]["eta"],inputevents[i]["electron"][0]["phi"], num_leptons, num_taus))
                                    else:
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["muon"][0]["PT"],inputevents[i]["muon"][0]["eta"],inputevents[i]["muon"][0]["phi"], num_leptons, num_taus))

                                    # MET (eta=0 by def, but I save phi because I have many jets)
                                    with open(datFolder + 'MET_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))

                                    # hadronic info
                                    with open(datFolder + 'hadinfo_bu.dat', 'a') as f:
                                        f.write('%f %f %d\n' % (HT, HT_had, num_jets))

                                    # total events that passed the cuts
                                    ttj_cut_ev += 1

print('Num initial events: ', ttj_init_ev)
print('Total pasaron los cortes: ', ttj_cut_ev)
print('Hay: ', contador_tau, ' taus')
print('Hay: ', counter_bad_jets, ' eventos con al menos un jet malo')
print('Hay: ', contador_btag, ' con btag en leading jet')

1
Esta run tuvo 100426 eventos en el lhco
('Num initial events: ', 100426)
('Total pasaron los cortes: ', 55)
('Hay: ', 1, ' taus')
('Hay: ', 2, ' eventos con al menos un jet malo')
('Hay: ', 10, ' con btag en leading jet')


In [19]:
ttj_cross = 71.52 #[pb] integrated cross-section without matching
ttj_NLO   = 1.47 # k-factor [14 TeV] CHECK THIS
ttj_init_sim_eve = 200000 * ttj_runs

cross       = ttj_cross * 1000 * ttj_NLO
aceptancia  = ttj_cut_ev / (1. * ttj_init_sim_eve)
luminosidad = 100

ttj_fidcross = cross * aceptancia
ttj_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', ttj_fidcross)
print('Events expected: ', ttj_expected)

('fiducial cross section: ', 19.668000000000003)
('Events expected: ', 1966.8000000000002)


#### ttj normal
- [x] Done
    - se esperan 13524.679999999998 con MET > 100
    - se esperan 1845.8000000000002 con MET > 200

In [22]:
Folder    = '../data/wagner-bkgs-varios/'
datFolder = '../data/wagner-bkgs-varios/'

ttj_runs  = 1

ttj_init_ev = 0
ttj_cut_ev  = 0

ttj_photon  = []
ttj_jet     = []
ttj_lepton  = []
ttj_MET     = []
ttj_hadinfo = []


counter_bad_jets = 0
contador_tau = 0
contador_btag = 0
for ii in range(1, 1 + ttj_runs):
    print(ii)
    if ii not in [0]:
        inputevents = LHCO_reader.Events(f_name = Folder + 'ttj-tests.lhco')

        ttj_init_ev += len(inputevents)
        print('Esta run tuvo ' + str(len(inputevents)) + ' eventos en el lhco')
        
        for i in range(len(inputevents)):

            num_jets = 0
    #        num_btag = 0
            HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets


            # at least one: photon, jet and lepton (electron or muon)
            if inputevents[i].number()["photon"] > 0 and inputevents[i].number()["jet"] > 0 and (inputevents[i].number()["electron"] > 0 or inputevents[i].number()["muon"] > 0):
                if inputevents[i]["jet"][0]["PT"] > 100 and -4.5 < inputevents[i]["jet"][0]["eta"] < 4.5:
                    if inputevents[i]["photon"][0]["PT"] > 10 and -2.37 < inputevents[i]["photon"][0]["eta"] < 2.37:
                        if inputevents[i]["MET"][0]["PT"] > 200:
                            # lepton info
                            if (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] > 0):
                                if inputevents[i]["electron"][0]["PT"] > inputevents[i]["muon"][0]["PT"]: 
                                    flag = 'electron'
                                    eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                                else: 
                                    flag = 'muon'
                                    eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron
                            elif (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] == 0): 
                                flag = 'electron'
                                eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                            elif (inputevents[i].number()["muon"] > 0) and (inputevents[i].number()["electron"] == 0): 
                                flag = 'muon'
                                eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron

                            if (-2.47 < eta_leading_lepton < -1.52) or (-1.37 < eta_leading_lepton < 1.37) or (1.52 < eta_leading_lepton < 2.47): 

                                # how many jets (total) and total hadronic pT
                                flag_bad_jets = 0                            
                                num_jets = 0
                                HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
                                HT = 0 # total HT, defined as the scalar sum of the pT of all reconstructed particles

                                for j in range(inputevents[i].number()["jet"]):
                                    if (inputevents[i]["jet"][j]["PT"] > 20) and (-4.5 < inputevents[i]["jet"][j]["eta"] < 4.5):
                                        num_jets += 1
                                        HT_had += inputevents[i]["jet"][j]["PT"]
                                        HT += inputevents[i]["jet"][j]["PT"]
                                    if (np.abs(inputevents[i]["jet"][j]["eta"]) > 4.5) or (inputevents[i]["jet"][j]["PT"] < 20):
                                        flag_bad_jets = 1

                                # how many photons                         
                                num_photons = 0
                                for j in range(inputevents[i].number()["photon"]):
                                    if inputevents[i]["photon"][j]["PT"] > 10 and -2.37 < inputevents[i]["photon"][j]["eta"] < 2.37:
                                        num_photons = num_photons + 1
                                        HT += inputevents[i]["photon"][j]["PT"]

                                # how many muons                        
                                num_muons = 0
                                for j in range(inputevents[i].number()["muon"]):
                                    if inputevents[i]["muon"][j]["PT"] > 10 and -2.7 < inputevents[i]["muon"][j]["eta"] < 2.7:
                                        num_muons = num_muons + 1
                                        HT += inputevents[i]["muon"][j]["PT"]

                                # how many electrons                        
                                num_electrons = 0
                                for j in range(inputevents[i].number()["electron"]):
                                    if inputevents[i]["electron"][j]["PT"] > 10:
                                        eta = inputevents[i]["electron"][j]["eta"]
                                        if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                            num_electrons = num_electrons + 1
                                            HT += inputevents[i]["electron"][j]["PT"]

                                if flag_bad_jets == 1: counter_bad_jets += 1 
                                if flag_bad_jets == 0:
                                    num_taus = 0
                                    if inputevents[i].number()["tau"] > 0: 
                                        contador_tau = contador_tau + 1
                                        # how many taus                    
                                        for j in range(inputevents[i].number()["tau"]):
                                            if inputevents[i]["tau"][j]["PT"] > 20:
                                                eta = inputevents[i]["tau"][j]["eta"]
                                                if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                                    num_taus = num_taus + 1
                                                    HT += inputevents[i]["tau"][j]["PT"]
                                                    
                                    if inputevents[i]["jet"][0]['btag'] == 1: contador_btag = contador_btag + 1

                                    # photon info:
                                    with open(datFolder + 'photon_bu.dat', 'a') as f:
                                        f.write('%f %f %f %d\n' % (inputevents[i]["photon"][0]["PT"],inputevents[i]["photon"][0]["eta"],inputevents[i]["photon"][0]["phi"], num_photons))

                                    # jet info:
                                    with open(datFolder + 'jet_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["jet"][0]["PT"],inputevents[i]["jet"][0]["eta"],inputevents[i]["jet"][0]["phi"]))

                                    # lepton info
                                    num_leptons = num_electrons + num_muons
                                    if flag == "electron":
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["electron"][0]["PT"],inputevents[i]["electron"][0]["eta"],inputevents[i]["electron"][0]["phi"], num_leptons, num_taus))
                                    else:
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["muon"][0]["PT"],inputevents[i]["muon"][0]["eta"],inputevents[i]["muon"][0]["phi"], num_leptons, num_taus))

                                    # MET (eta=0 by def, but I save phi because I have many jets)
                                    with open(datFolder + 'MET_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))

                                    # hadronic info
                                    with open(datFolder + 'hadinfo_bu.dat', 'a') as f:
                                        f.write('%f %f %d\n' % (HT, HT_had, num_jets))

                                    # total events that passed the cuts
                                    ttj_cut_ev += 1

print('Num initial events: ', ttj_init_ev)
print('Total pasaron los cortes: ', ttj_cut_ev)
print('Hay: ', contador_tau, ' taus')
print('Hay: ', counter_bad_jets, ' eventos con al menos un jet malo')
print('Hay: ', contador_btag, ' con btag en leading jet')

1
Esta run tuvo 100426 eventos en el lhco
('Num initial events: ', 100426)
('Total pasaron los cortes: ', 55)
('Hay: ', 1, ' taus')
('Hay: ', 2, ' eventos con al menos un jet malo')
('Hay: ', 10, ' con btag en leading jet')


In [23]:
ttj_cross = 67.12 #[pb] integrated cross-section without matching
ttj_NLO   = 1.47 # k-factor [14 TeV] CHECK THIS
ttj_init_sim_eve = 200000 * ttj_runs

cross       = ttj_cross * 1000 * ttj_NLO
aceptancia  = ttj_cut_ev / (1. * ttj_init_sim_eve)
luminosidad = 100

ttj_fidcross = cross * aceptancia
ttj_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', ttj_fidcross)
print('Events expected: ', ttj_expected)

('fiducial cross section: ', 18.458000000000002)
('Events expected: ', 1845.8000000000002)


### tt gamma
- [ ] Done. Da distinto a lo q daba antes. Se esperan 3.3k con MET>100

#### tt gamma igual a rosa
- [x] Done
    - Se esperan 2.9 k con MET > 100
    - Se esperan 730 con MET > 200

In [4]:
Folder    = '../data/wagner-bkgs-varios/'
datFolder = '../data/wagner-bkgs-varios/'

ttgamma_runs  = 1

ttgamma_init_ev = 0
ttgamma_cut_ev  = 0

ttgamma_photon  = []
ttgamma_jet     = []
ttgamma_lepton  = []
ttgamma_MET     = []
ttgamma_hadinfo = []


counter_bad_jets = 0
contador_tau = 0
contador_btag = 0
for ii in range(1, 1 + ttgamma_runs):
    print(ii)
    if ii not in [0]:
        inputevents = LHCO_reader.Events(f_name = Folder + 'ttgamma-NOb-NOjj-extraj-tests.lhco')

        ttgamma_init_ev += len(inputevents)
        print('Esta run tuvo ' + str(len(inputevents)) + ' eventos en el lhco')
        
        for i in range(len(inputevents)):

            num_jets = 0
    #        num_btag = 0
            HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets


            # at least one: photon, jet and lepton (electron or muon)
            if inputevents[i].number()["photon"] > 0 and inputevents[i].number()["jet"] > 0 and (inputevents[i].number()["electron"] > 0 or inputevents[i].number()["muon"] > 0):
                if inputevents[i]["jet"][0]["PT"] > 100 and -4.5 < inputevents[i]["jet"][0]["eta"] < 4.5:
                    if inputevents[i]["photon"][0]["PT"] > 10 and -2.37 < inputevents[i]["photon"][0]["eta"] < 2.37:
                        if inputevents[i]["MET"][0]["PT"] > 200:
                            # lepton info
                            if (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] > 0):
                                if inputevents[i]["electron"][0]["PT"] > inputevents[i]["muon"][0]["PT"]: 
                                    flag = 'electron'
                                    eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                                else: 
                                    flag = 'muon'
                                    eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron
                            elif (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] == 0): 
                                flag = 'electron'
                                eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                            elif (inputevents[i].number()["muon"] > 0) and (inputevents[i].number()["electron"] == 0): 
                                flag = 'muon'
                                eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron

                            if (-2.47 < eta_leading_lepton < -1.52) or (-1.37 < eta_leading_lepton < 1.37) or (1.52 < eta_leading_lepton < 2.47): 

                                # how many jets (total) and total hadronic pT
                                flag_bad_jets = 0                            
                                num_jets = 0
                                HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
                                HT = 0 # total HT, defined as the scalar sum of the pT of all reconstructed particles

                                for j in range(inputevents[i].number()["jet"]):
                                    if (inputevents[i]["jet"][j]["PT"] > 20) and (-4.5 < inputevents[i]["jet"][j]["eta"] < 4.5):
                                        num_jets += 1
                                        HT_had += inputevents[i]["jet"][j]["PT"]
                                        HT += inputevents[i]["jet"][j]["PT"]
                                    if (np.abs(inputevents[i]["jet"][j]["eta"]) > 4.5) or (inputevents[i]["jet"][j]["PT"] < 20):
                                        flag_bad_jets = 1

                                # how many photons                         
                                num_photons = 0
                                for j in range(inputevents[i].number()["photon"]):
                                    if inputevents[i]["photon"][j]["PT"] > 10 and -2.37 < inputevents[i]["photon"][j]["eta"] < 2.37:
                                        num_photons = num_photons + 1
                                        HT += inputevents[i]["photon"][j]["PT"]

                                # how many muons                        
                                num_muons = 0
                                for j in range(inputevents[i].number()["muon"]):
                                    if inputevents[i]["muon"][j]["PT"] > 10 and -2.7 < inputevents[i]["muon"][j]["eta"] < 2.7:
                                        num_muons = num_muons + 1
                                        HT += inputevents[i]["muon"][j]["PT"]

                                # how many electrons                        
                                num_electrons = 0
                                for j in range(inputevents[i].number()["electron"]):
                                    if inputevents[i]["electron"][j]["PT"] > 10:
                                        eta = inputevents[i]["electron"][j]["eta"]
                                        if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                            num_electrons = num_electrons + 1
                                            HT += inputevents[i]["electron"][j]["PT"]

                                if flag_bad_jets == 1: counter_bad_jets += 1 
                                if flag_bad_jets == 0:
                                    num_taus = 0
                                    if inputevents[i].number()["tau"] > 0: 
                                        contador_tau = contador_tau + 1
                                        # how many taus                    
                                        for j in range(inputevents[i].number()["tau"]):
                                            if inputevents[i]["tau"][j]["PT"] > 20:
                                                eta = inputevents[i]["tau"][j]["eta"]
                                                if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                                    num_taus = num_taus + 1
                                                    HT += inputevents[i]["tau"][j]["PT"]
                                                    
                                    if inputevents[i]["jet"][0]['btag'] == 1: contador_btag = contador_btag + 1

                                    # photon info:
                                    with open(datFolder + 'photon_bu.dat', 'a') as f:
                                        f.write('%f %f %f %d\n' % (inputevents[i]["photon"][0]["PT"],inputevents[i]["photon"][0]["eta"],inputevents[i]["photon"][0]["phi"], num_photons))

                                    # jet info:
                                    with open(datFolder + 'jet_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["jet"][0]["PT"],inputevents[i]["jet"][0]["eta"],inputevents[i]["jet"][0]["phi"]))

                                    # lepton info
                                    num_leptons = num_electrons + num_muons
                                    if flag == "electron":
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["electron"][0]["PT"],inputevents[i]["electron"][0]["eta"],inputevents[i]["electron"][0]["phi"], num_leptons, num_taus))
                                    else:
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["muon"][0]["PT"],inputevents[i]["muon"][0]["eta"],inputevents[i]["muon"][0]["phi"], num_leptons, num_taus))

                                    # MET (eta=0 by def, but I save phi because I have many jets)
                                    with open(datFolder + 'MET_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))

                                    # hadronic info
                                    with open(datFolder + 'hadinfo_bu.dat', 'a') as f:
                                        f.write('%f %f %d\n' % (HT, HT_had, num_jets))

                                    # total events that passed the cuts
                                    ttgamma_cut_ev += 1

print('Num initial events: ', ttgamma_init_ev)
print('Total pasaron los cortes: ', ttgamma_cut_ev)
print('Hay: ', contador_tau, ' taus')
print('Hay: ', counter_bad_jets, ' eventos con al menos un jet malo')
print('Hay: ', contador_btag, ' con btag en leading jet')

1
Esta run tuvo 110333 eventos en el lhco
('Num initial events: ', 110333)
('Total pasaron los cortes: ', 6910)
('Hay: ', 395, ' taus')
('Hay: ', 117, ' eventos con al menos un jet malo')
('Hay: ', 1377, ' con btag en leading jet')


In [5]:
ttgamma_cross = 0.1449 #[pb] integrated cross-section without matching
ttgamma_NLO   = 1.46 # k-factor [14 TeV] CHECK THIS
ttgamma_init_sim_eve = 200000 * ttgamma_runs

cross       = ttgamma_cross * 1000 * ttgamma_NLO
aceptancia  = ttgamma_cut_ev / (1. * ttgamma_init_sim_eve)
luminosidad = 100

ttgamma_fidcross = cross * aceptancia
ttgamma_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', ttgamma_fidcross)
print('Events expected: ', ttgamma_expected)

('fiducial cross section: ', 7.309190699999999)
('Events expected: ', 730.9190699999999)


#### tt gamma sin b bbar sin jj
- [x] Done
    - se esperan 1716.256718 con MET > 100
    - se esperan 375.2706620000001 con MET > 200

In [8]:
Folder    = '../data/wagner-bkgs-varios/'
datFolder = '../data/wagner-bkgs-varios/'

ttgamma_runs  = 1

ttgamma_init_ev = 0
ttgamma_cut_ev  = 0

ttgamma_photon  = []
ttgamma_jet     = []
ttgamma_lepton  = []
ttgamma_MET     = []
ttgamma_hadinfo = []


counter_bad_jets = 0
contador_tau = 0
contador_btag = 0
for ii in range(1, 1 + ttgamma_runs):
    print(ii)
    if ii not in [0]:
        inputevents = LHCO_reader.Events(f_name = Folder + 'ttgamma-NOb-NOjj-tests.lhco')

        ttgamma_init_ev += len(inputevents)
        print('Esta run tuvo ' + str(len(inputevents)) + ' eventos en el lhco')
        
        for i in range(len(inputevents)):

            num_jets = 0
    #        num_btag = 0
            HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets


            # at least one: photon, jet and lepton (electron or muon)
            if inputevents[i].number()["photon"] > 0 and inputevents[i].number()["jet"] > 0 and (inputevents[i].number()["electron"] > 0 or inputevents[i].number()["muon"] > 0):
                if inputevents[i]["jet"][0]["PT"] > 100 and -4.5 < inputevents[i]["jet"][0]["eta"] < 4.5:
                    if inputevents[i]["photon"][0]["PT"] > 10 and -2.37 < inputevents[i]["photon"][0]["eta"] < 2.37:
                        if inputevents[i]["MET"][0]["PT"] > 200:
                            # lepton info
                            if (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] > 0):
                                if inputevents[i]["electron"][0]["PT"] > inputevents[i]["muon"][0]["PT"]: 
                                    flag = 'electron'
                                    eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                                else: 
                                    flag = 'muon'
                                    eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron
                            elif (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] == 0): 
                                flag = 'electron'
                                eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                            elif (inputevents[i].number()["muon"] > 0) and (inputevents[i].number()["electron"] == 0): 
                                flag = 'muon'
                                eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron

                            if (-2.47 < eta_leading_lepton < -1.52) or (-1.37 < eta_leading_lepton < 1.37) or (1.52 < eta_leading_lepton < 2.47): 

                                # how many jets (total) and total hadronic pT
                                flag_bad_jets = 0                            
                                num_jets = 0
                                HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
                                HT = 0 # total HT, defined as the scalar sum of the pT of all reconstructed particles

                                for j in range(inputevents[i].number()["jet"]):
                                    if (inputevents[i]["jet"][j]["PT"] > 20) and (-4.5 < inputevents[i]["jet"][j]["eta"] < 4.5):
                                        num_jets += 1
                                        HT_had += inputevents[i]["jet"][j]["PT"]
                                        HT += inputevents[i]["jet"][j]["PT"]
                                    if (np.abs(inputevents[i]["jet"][j]["eta"]) > 4.5) or (inputevents[i]["jet"][j]["PT"] < 20):
                                        flag_bad_jets = 1

                                # how many photons                         
                                num_photons = 0
                                for j in range(inputevents[i].number()["photon"]):
                                    if inputevents[i]["photon"][j]["PT"] > 10 and -2.37 < inputevents[i]["photon"][j]["eta"] < 2.37:
                                        num_photons = num_photons + 1
                                        HT += inputevents[i]["photon"][j]["PT"]

                                # how many muons                        
                                num_muons = 0
                                for j in range(inputevents[i].number()["muon"]):
                                    if inputevents[i]["muon"][j]["PT"] > 10 and -2.7 < inputevents[i]["muon"][j]["eta"] < 2.7:
                                        num_muons = num_muons + 1
                                        HT += inputevents[i]["muon"][j]["PT"]

                                # how many electrons                        
                                num_electrons = 0
                                for j in range(inputevents[i].number()["electron"]):
                                    if inputevents[i]["electron"][j]["PT"] > 10:
                                        eta = inputevents[i]["electron"][j]["eta"]
                                        if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                            num_electrons = num_electrons + 1
                                            HT += inputevents[i]["electron"][j]["PT"]

                                if flag_bad_jets == 1: counter_bad_jets += 1 
                                if flag_bad_jets == 0:
                                    num_taus = 0
                                    if inputevents[i].number()["tau"] > 0: 
                                        contador_tau = contador_tau + 1
                                        # how many taus                    
                                        for j in range(inputevents[i].number()["tau"]):
                                            if inputevents[i]["tau"][j]["PT"] > 20:
                                                eta = inputevents[i]["tau"][j]["eta"]
                                                if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                                    num_taus = num_taus + 1
                                                    HT += inputevents[i]["tau"][j]["PT"]
                                                    
                                    if inputevents[i]["jet"][0]['btag'] == 1: contador_btag = contador_btag + 1

                                    # photon info:
                                    with open(datFolder + 'photon_bu.dat', 'a') as f:
                                        f.write('%f %f %f %d\n' % (inputevents[i]["photon"][0]["PT"],inputevents[i]["photon"][0]["eta"],inputevents[i]["photon"][0]["phi"], num_photons))

                                    # jet info:
                                    with open(datFolder + 'jet_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["jet"][0]["PT"],inputevents[i]["jet"][0]["eta"],inputevents[i]["jet"][0]["phi"]))

                                    # lepton info
                                    num_leptons = num_electrons + num_muons
                                    if flag == "electron":
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["electron"][0]["PT"],inputevents[i]["electron"][0]["eta"],inputevents[i]["electron"][0]["phi"], num_leptons, num_taus))
                                    else:
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["muon"][0]["PT"],inputevents[i]["muon"][0]["eta"],inputevents[i]["muon"][0]["phi"], num_leptons, num_taus))

                                    # MET (eta=0 by def, but I save phi because I have many jets)
                                    with open(datFolder + 'MET_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))

                                    # hadronic info
                                    with open(datFolder + 'hadinfo_bu.dat', 'a') as f:
                                        f.write('%f %f %d\n' % (HT, HT_had, num_jets))

                                    # total events that passed the cuts
                                    ttgamma_cut_ev += 1

print('Num initial events: ', ttgamma_init_ev)
print('Total pasaron los cortes: ', ttgamma_cut_ev)
print('Hay: ', contador_tau, ' taus')
print('Hay: ', counter_bad_jets, ' eventos con al menos un jet malo')
print('Hay: ', contador_btag, ' con btag en leading jet')

1
Esta run tuvo 200000 eventos en el lhco
('Num initial events: ', 200000)
('Total pasaron los cortes: ', 10595)
('Hay: ', 612, ' taus')
('Hay: ', 172, ' eventos con al menos un jet malo')
('Hay: ', 2708, ' con btag en leading jet')


In [9]:
ttgamma_cross = 0.04852 #[pb] integrated cross-section without matching
ttgamma_NLO   = 1.46 # k-factor [14 TeV] CHECK THIS
ttgamma_init_sim_eve = 200000 * ttgamma_runs

cross       = ttgamma_cross * 1000 * ttgamma_NLO
aceptancia  = ttgamma_cut_ev / (1. * ttgamma_init_sim_eve)
luminosidad = 100

ttgamma_fidcross = cross * aceptancia
ttgamma_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', ttgamma_fidcross)
print('Events expected: ', ttgamma_expected)

('fiducial cross section: ', 3.7527066200000005)
('Events expected: ', 375.2706620000001)


#### tt gamma sin b bbar
- [x] Done
    - se esperan 1332.2005789999996 con MET > 100
    - se esperan 273.128989 con MET > 200

In [26]:
Folder    = '../data/wagner-bkgs-varios/'
datFolder = '../data/wagner-bkgs-varios/'

ttgamma_runs  = 1

ttgamma_init_ev = 0
ttgamma_cut_ev  = 0

ttgamma_photon  = []
ttgamma_jet     = []
ttgamma_lepton  = []
ttgamma_MET     = []
ttgamma_hadinfo = []


counter_bad_jets = 0
contador_tau = 0
contador_btag = 0
for ii in range(1, 1 + ttgamma_runs):
    print(ii)
    if ii not in [0]:
        inputevents = LHCO_reader.Events(f_name = Folder + 'ttgamma-NOb-tests.lhco')

        ttgamma_init_ev += len(inputevents)
        print('Esta run tuvo ' + str(len(inputevents)) + ' eventos en el lhco')
        
        for i in range(len(inputevents)):

            num_jets = 0
    #        num_btag = 0
            HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets


            # at least one: photon, jet and lepton (electron or muon)
            if inputevents[i].number()["photon"] > 0 and inputevents[i].number()["jet"] > 0 and (inputevents[i].number()["electron"] > 0 or inputevents[i].number()["muon"] > 0):
                if inputevents[i]["jet"][0]["PT"] > 100 and -4.5 < inputevents[i]["jet"][0]["eta"] < 4.5:
                    if inputevents[i]["photon"][0]["PT"] > 10 and -2.37 < inputevents[i]["photon"][0]["eta"] < 2.37:
                        if inputevents[i]["MET"][0]["PT"] > 200:
                            # lepton info
                            if (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] > 0):
                                if inputevents[i]["electron"][0]["PT"] > inputevents[i]["muon"][0]["PT"]: 
                                    flag = 'electron'
                                    eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                                else: 
                                    flag = 'muon'
                                    eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron
                            elif (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] == 0): 
                                flag = 'electron'
                                eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                            elif (inputevents[i].number()["muon"] > 0) and (inputevents[i].number()["electron"] == 0): 
                                flag = 'muon'
                                eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron

                            if (-2.47 < eta_leading_lepton < -1.52) or (-1.37 < eta_leading_lepton < 1.37) or (1.52 < eta_leading_lepton < 2.47): 

                                # how many jets (total) and total hadronic pT
                                flag_bad_jets = 0                            
                                num_jets = 0
                                HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
                                HT = 0 # total HT, defined as the scalar sum of the pT of all reconstructed particles

                                for j in range(inputevents[i].number()["jet"]):
                                    if (inputevents[i]["jet"][j]["PT"] > 20) and (-4.5 < inputevents[i]["jet"][j]["eta"] < 4.5):
                                        num_jets += 1
                                        HT_had += inputevents[i]["jet"][j]["PT"]
                                        HT += inputevents[i]["jet"][j]["PT"]
                                    if (np.abs(inputevents[i]["jet"][j]["eta"]) > 4.5) or (inputevents[i]["jet"][j]["PT"] < 20):
                                        flag_bad_jets = 1

                                # how many photons                         
                                num_photons = 0
                                for j in range(inputevents[i].number()["photon"]):
                                    if inputevents[i]["photon"][j]["PT"] > 10 and -2.37 < inputevents[i]["photon"][j]["eta"] < 2.37:
                                        num_photons = num_photons + 1
                                        HT += inputevents[i]["photon"][j]["PT"]

                                # how many muons                        
                                num_muons = 0
                                for j in range(inputevents[i].number()["muon"]):
                                    if inputevents[i]["muon"][j]["PT"] > 10 and -2.7 < inputevents[i]["muon"][j]["eta"] < 2.7:
                                        num_muons = num_muons + 1
                                        HT += inputevents[i]["muon"][j]["PT"]

                                # how many electrons                        
                                num_electrons = 0
                                for j in range(inputevents[i].number()["electron"]):
                                    if inputevents[i]["electron"][j]["PT"] > 10:
                                        eta = inputevents[i]["electron"][j]["eta"]
                                        if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                            num_electrons = num_electrons + 1
                                            HT += inputevents[i]["electron"][j]["PT"]

                                if flag_bad_jets == 1: counter_bad_jets += 1 
                                if flag_bad_jets == 0:
                                    num_taus = 0
                                    if inputevents[i].number()["tau"] > 0: 
                                        contador_tau = contador_tau + 1
                                        # how many taus                    
                                        for j in range(inputevents[i].number()["tau"]):
                                            if inputevents[i]["tau"][j]["PT"] > 20:
                                                eta = inputevents[i]["tau"][j]["eta"]
                                                if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                                    num_taus = num_taus + 1
                                                    HT += inputevents[i]["tau"][j]["PT"]
                                                    
                                    if inputevents[i]["jet"][0]['btag'] == 1: contador_btag = contador_btag + 1

                                    # photon info:
                                    with open(datFolder + 'photon_bu.dat', 'a') as f:
                                        f.write('%f %f %f %d\n' % (inputevents[i]["photon"][0]["PT"],inputevents[i]["photon"][0]["eta"],inputevents[i]["photon"][0]["phi"], num_photons))

                                    # jet info:
                                    with open(datFolder + 'jet_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["jet"][0]["PT"],inputevents[i]["jet"][0]["eta"],inputevents[i]["jet"][0]["phi"]))

                                    # lepton info
                                    num_leptons = num_electrons + num_muons
                                    if flag == "electron":
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["electron"][0]["PT"],inputevents[i]["electron"][0]["eta"],inputevents[i]["electron"][0]["phi"], num_leptons, num_taus))
                                    else:
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["muon"][0]["PT"],inputevents[i]["muon"][0]["eta"],inputevents[i]["muon"][0]["phi"], num_leptons, num_taus))

                                    # MET (eta=0 by def, but I save phi because I have many jets)
                                    with open(datFolder + 'MET_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))

                                    # hadronic info
                                    with open(datFolder + 'hadinfo_bu.dat', 'a') as f:
                                        f.write('%f %f %d\n' % (HT, HT_had, num_jets))

                                    # total events that passed the cuts
                                    ttgamma_cut_ev += 1

print('Num initial events: ', ttgamma_init_ev)
print('Total pasaron los cortes: ', ttgamma_cut_ev)
print('Hay: ', contador_tau, ' taus')
print('Hay: ', counter_bad_jets, ' eventos con al menos un jet malo')
print('Hay: ', contador_btag, ' con btag en leading jet')

1
Esta run tuvo 200000 eventos en el lhco
('Num initial events: ', 200000)
('Total pasaron los cortes: ', 1421)
('Hay: ', 89, ' taus')
('Hay: ', 28, ' eventos con al menos un jet malo')
('Hay: ', 371, ' con btag en leading jet')


In [27]:
ttgamma_cross = 0.2633 #[pb] integrated cross-section without matching
ttgamma_NLO   = 1.46 # k-factor [14 TeV] CHECK THIS
ttgamma_init_sim_eve = 200000 * ttgamma_runs

cross       = ttgamma_cross * 1000 * ttgamma_NLO
aceptancia  = ttgamma_cut_ev / (1. * ttgamma_init_sim_eve)
luminosidad = 100

ttgamma_fidcross = cross * aceptancia
ttgamma_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', ttgamma_fidcross)
print('Events expected: ', ttgamma_expected)

('fiducial cross section: ', 2.7312898899999998)
('Events expected: ', 273.128989)


#### tt gamma normal
- [x] Done
    - se esperan 3382.693817066667 con MET > 100
    - se esperan 374.62957600000004 con MET > 200

In [35]:
Folder    = '../data/wagner-bkgs-varios/'
datFolder = '../data/wagner-bkgs-varios/'

ttgamma_runs  = 1

ttgamma_init_ev = 0
ttgamma_cut_ev  = 0

ttgamma_photon  = []
ttgamma_jet     = []
ttgamma_lepton  = []
ttgamma_MET     = []
ttgamma_hadinfo = []


counter_bad_jets = 0
contador_tau = 0
contador_btag = 0
for ii in range(1, 1 + ttgamma_runs):
    print(ii)
    if ii not in [0]:
        inputevents = LHCO_reader.Events(f_name = Folder + 'ttgamma-tests.lhco')

        ttgamma_init_ev += len(inputevents)
        print('Esta run tuvo ' + str(len(inputevents)) + ' eventos en el lhco')
        
        for i in range(len(inputevents)):

            num_jets = 0
    #        num_btag = 0
            HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets


            # at least one: photon, jet and lepton (electron or muon)
            if inputevents[i].number()["photon"] > 0 and inputevents[i].number()["jet"] > 0 and (inputevents[i].number()["electron"] > 0 or inputevents[i].number()["muon"] > 0):
                if inputevents[i]["jet"][0]["PT"] > 100 and -4.5 < inputevents[i]["jet"][0]["eta"] < 4.5:
                    if inputevents[i]["photon"][0]["PT"] > 10 and -2.37 < inputevents[i]["photon"][0]["eta"] < 2.37:
                        if inputevents[i]["MET"][0]["PT"] > 100:
                            # lepton info
                            if (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] > 0):
                                if inputevents[i]["electron"][0]["PT"] > inputevents[i]["muon"][0]["PT"]: 
                                    flag = 'electron'
                                    eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                                else: 
                                    flag = 'muon'
                                    eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron
                            elif (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] == 0): 
                                flag = 'electron'
                                eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                            elif (inputevents[i].number()["muon"] > 0) and (inputevents[i].number()["electron"] == 0): 
                                flag = 'muon'
                                eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron

                            if (-2.47 < eta_leading_lepton < -1.52) or (-1.37 < eta_leading_lepton < 1.37) or (1.52 < eta_leading_lepton < 2.47): 

                                # how many jets (total) and total hadronic pT
                                flag_bad_jets = 0                            
                                num_jets = 0
                                HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
                                HT = 0 # total HT, defined as the scalar sum of the pT of all reconstructed particles

                                for j in range(inputevents[i].number()["jet"]):
                                    if (inputevents[i]["jet"][j]["PT"] > 20) and (-4.5 < inputevents[i]["jet"][j]["eta"] < 4.5):
                                        num_jets += 1
                                        HT_had += inputevents[i]["jet"][j]["PT"]
                                        HT += inputevents[i]["jet"][j]["PT"]
                                    if (np.abs(inputevents[i]["jet"][j]["eta"]) > 4.5) or (inputevents[i]["jet"][j]["PT"] < 20):
                                        flag_bad_jets = 1

                                # how many photons                         
                                num_photons = 0
                                for j in range(inputevents[i].number()["photon"]):
                                    if inputevents[i]["photon"][j]["PT"] > 10 and -2.37 < inputevents[i]["photon"][j]["eta"] < 2.37:
                                        num_photons = num_photons + 1
                                        HT += inputevents[i]["photon"][j]["PT"]

                                # how many muons                        
                                num_muons = 0
                                for j in range(inputevents[i].number()["muon"]):
                                    if inputevents[i]["muon"][j]["PT"] > 10 and -2.7 < inputevents[i]["muon"][j]["eta"] < 2.7:
                                        num_muons = num_muons + 1
                                        HT += inputevents[i]["muon"][j]["PT"]

                                # how many electrons                        
                                num_electrons = 0
                                for j in range(inputevents[i].number()["electron"]):
                                    if inputevents[i]["electron"][j]["PT"] > 10:
                                        eta = inputevents[i]["electron"][j]["eta"]
                                        if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                            num_electrons = num_electrons + 1
                                            HT += inputevents[i]["electron"][j]["PT"]

                                if flag_bad_jets == 1: counter_bad_jets += 1 
                                if flag_bad_jets == 0:
                                    num_taus = 0
                                    if inputevents[i].number()["tau"] > 0: 
                                        contador_tau = contador_tau + 1
                                        # how many taus                    
                                        for j in range(inputevents[i].number()["tau"]):
                                            if inputevents[i]["tau"][j]["PT"] > 20:
                                                eta = inputevents[i]["tau"][j]["eta"]
                                                if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                                    num_taus = num_taus + 1
                                                    HT += inputevents[i]["tau"][j]["PT"]
                                                    
                                    if inputevents[i]["jet"][0]['btag'] == 1: contador_btag = contador_btag + 1

                                    # photon info:
                                    with open(datFolder + 'photon_bu.dat', 'a') as f:
                                        f.write('%f %f %f %d\n' % (inputevents[i]["photon"][0]["PT"],inputevents[i]["photon"][0]["eta"],inputevents[i]["photon"][0]["phi"], num_photons))

                                    # jet info:
                                    with open(datFolder + 'jet_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["jet"][0]["PT"],inputevents[i]["jet"][0]["eta"],inputevents[i]["jet"][0]["phi"]))

                                    # lepton info
                                    num_leptons = num_electrons + num_muons
                                    if flag == "electron":
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["electron"][0]["PT"],inputevents[i]["electron"][0]["eta"],inputevents[i]["electron"][0]["phi"], num_leptons, num_taus))
                                    else:
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["muon"][0]["PT"],inputevents[i]["muon"][0]["eta"],inputevents[i]["muon"][0]["phi"], num_leptons, num_taus))

                                    # MET (eta=0 by def, but I save phi because I have many jets)
                                    with open(datFolder + 'MET_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))

                                    # hadronic info
                                    with open(datFolder + 'hadinfo_bu.dat', 'a') as f:
                                        f.write('%f %f %d\n' % (HT, HT_had, num_jets))

                                    # total events that passed the cuts
                                    ttgamma_cut_ev += 1

print('Num initial events: ', ttgamma_init_ev)
print('Total pasaron los cortes: ', ttgamma_cut_ev)
print('Hay: ', contador_tau, ' taus')
print('Hay: ', counter_bad_jets, ' eventos con al menos un jet malo')
print('Hay: ', contador_btag, ' con btag en leading jet')

1
Esta run tuvo 200000 eventos en el lhco
('Num initial events: ', 200000)
('Total pasaron los cortes: ', 9686)
('Hay: ', 505, ' taus')
('Hay: ', 210, ' eventos con al menos un jet malo')
('Hay: ', 3793, ' con btag en leading jet')


In [29]:
ttgamma_cross = 0.3352 #[pb] integrated cross-section without matching
ttgamma_NLO   = 1.46 # k-factor [14 TeV] CHECK THIS
ttgamma_init_sim_eve = 200000 * ttgamma_runs

cross       = ttgamma_cross * 1000 * ttgamma_NLO
aceptancia  = ttgamma_cut_ev / (1. * ttgamma_init_sim_eve)
luminosidad = 100

ttgamma_fidcross = cross * aceptancia
ttgamma_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', ttgamma_fidcross)
print('Events expected: ', ttgamma_expected)

('fiducial cross section: ', 3.74629576)
('Events expected: ', 374.62957600000004)


### [x] ttH 
- [x] Done. 12 eventos esperados con MET > 100

In [12]:
Folder    = '../data/wagner-bkgs-varios/'
datFolder = '../data/wagner-bkgs-varios/'

ttH_runs  = 1

ttH_init_ev = 0
ttH_cut_ev  = 0

ttH_photon  = []
ttH_jet     = []
ttH_lepton  = []
ttH_MET     = []
ttH_hadinfo = []


counter_bad_jets = 0
contador_tau = 0
contador_btag = 0
for ii in range(1, 1 + ttH_runs):
    print(ii)
    if ii not in [0]:
        inputevents = LHCO_reader.Events(f_name = Folder + 'ttH-tests.lhco')

        ttH_init_ev += len(inputevents)
        print('Esta run tuvo ' + str(len(inputevents)) + ' eventos en el lhco')
        
        for i in range(len(inputevents)):

            num_jets = 0
    #        num_btag = 0
            HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets


            # at least one: photon, jet and lepton (electron or muon)
            if inputevents[i].number()["photon"] > 0 and inputevents[i].number()["jet"] > 0 and (inputevents[i].number()["electron"] > 0 or inputevents[i].number()["muon"] > 0):
                if inputevents[i]["jet"][0]["PT"] > 100 and -4.5 < inputevents[i]["jet"][0]["eta"] < 4.5:
                    if inputevents[i]["photon"][0]["PT"] > 10 and -2.37 < inputevents[i]["photon"][0]["eta"] < 2.37:
                        if inputevents[i]["MET"][0]["PT"] > 100:
                            # lepton info
                            if (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] > 0):
                                if inputevents[i]["electron"][0]["PT"] > inputevents[i]["muon"][0]["PT"]: 
                                    flag = 'electron'
                                    eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                                else: 
                                    flag = 'muon'
                                    eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron
                            elif (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] == 0): 
                                flag = 'electron'
                                eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                            elif (inputevents[i].number()["muon"] > 0) and (inputevents[i].number()["electron"] == 0): 
                                flag = 'muon'
                                eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron

                            if (-2.47 < eta_leading_lepton < -1.52) or (-1.37 < eta_leading_lepton < 1.37) or (1.52 < eta_leading_lepton < 2.47): 

                                # how many jets (total) and total hadronic pT
                                flag_bad_jets = 0                            
                                num_jets = 0
                                HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
                                HT = 0 # total HT, defined as the scalar sum of the pT of all reconstructed particles

                                for j in range(inputevents[i].number()["jet"]):
                                    if (inputevents[i]["jet"][j]["PT"] > 20) and (-4.5 < inputevents[i]["jet"][j]["eta"] < 4.5):
                                        num_jets += 1
                                        HT_had += inputevents[i]["jet"][j]["PT"]
                                        HT += inputevents[i]["jet"][j]["PT"]
                                    if (np.abs(inputevents[i]["jet"][j]["eta"]) > 4.5) or (inputevents[i]["jet"][j]["PT"] < 20):
                                        flag_bad_jets = 1

                                # how many photons                         
                                num_photons = 0
                                for j in range(inputevents[i].number()["photon"]):
                                    if inputevents[i]["photon"][j]["PT"] > 10 and -2.37 < inputevents[i]["photon"][j]["eta"] < 2.37:
                                        num_photons = num_photons + 1
                                        HT += inputevents[i]["photon"][j]["PT"]

                                # how many muons                        
                                num_muons = 0
                                for j in range(inputevents[i].number()["muon"]):
                                    if inputevents[i]["muon"][j]["PT"] > 10 and -2.7 < inputevents[i]["muon"][j]["eta"] < 2.7:
                                        num_muons = num_muons + 1
                                        HT += inputevents[i]["muon"][j]["PT"]

                                # how many electrons                        
                                num_electrons = 0
                                for j in range(inputevents[i].number()["electron"]):
                                    if inputevents[i]["electron"][j]["PT"] > 10:
                                        eta = inputevents[i]["electron"][j]["eta"]
                                        if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                            num_electrons = num_electrons + 1
                                            HT += inputevents[i]["electron"][j]["PT"]

                                if flag_bad_jets == 1: counter_bad_jets += 1 
                                if flag_bad_jets == 0:
                                    num_taus = 0
                                    if inputevents[i].number()["tau"] > 0: 
                                        contador_tau = contador_tau + 1
                                        # how many taus                    
                                        for j in range(inputevents[i].number()["tau"]):
                                            if inputevents[i]["tau"][j]["PT"] > 20:
                                                eta = inputevents[i]["tau"][j]["eta"]
                                                if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                                    num_taus = num_taus + 1
                                                    HT += inputevents[i]["tau"][j]["PT"]
                                                    
                                    if inputevents[i]["jet"][0]['btag'] == 1: contador_btag = contador_btag + 1

                                    # photon info:
                                    with open(datFolder + 'photon_bu.dat', 'a') as f:
                                        f.write('%f %f %f %d\n' % (inputevents[i]["photon"][0]["PT"],inputevents[i]["photon"][0]["eta"],inputevents[i]["photon"][0]["phi"], num_photons))

                                    # jet info:
                                    with open(datFolder + 'jet_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["jet"][0]["PT"],inputevents[i]["jet"][0]["eta"],inputevents[i]["jet"][0]["phi"]))

                                    # lepton info
                                    num_leptons = num_electrons + num_muons
                                    if flag == "electron":
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["electron"][0]["PT"],inputevents[i]["electron"][0]["eta"],inputevents[i]["electron"][0]["phi"], num_leptons, num_taus))
                                    else:
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["muon"][0]["PT"],inputevents[i]["muon"][0]["eta"],inputevents[i]["muon"][0]["phi"], num_leptons, num_taus))

                                    # MET (eta=0 by def, but I save phi because I have many jets)
                                    with open(datFolder + 'MET_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))

                                    # hadronic info
                                    with open(datFolder + 'hadinfo_bu.dat', 'a') as f:
                                        f.write('%f %f %d\n' % (HT, HT_had, num_jets))

                                    # total events that passed the cuts
                                    ttH_cut_ev += 1

print('Num initial events: ', ttH_init_ev)
print('Total pasaron los cortes: ', ttH_cut_ev)
print('Hay: ', contador_tau, ' taus')
print('Hay: ', counter_bad_jets, ' eventos con al menos un jet malo')
print('Hay: ', contador_btag, ' con btag en leading jet')

1
Esta run tuvo 200000 eventos en el lhco
('Num initial events: ', 200000)
('Total pasaron los cortes: ', 2601)
('Hay: ', 208, ' taus')
('Hay: ', 49, ' eventos con al menos un jet malo')
('Hay: ', 1478, ' con btag en leading jet')


In [14]:
ttH_cross = 0.009616 #[pb] integrated cross-section without matching
ttH_NLO   = 1. # k-factor [14 TeV] CHECK THIS
ttH_init_sim_eve = 200000 * ttH_runs

cross       = ttH_cross * 1000 * ttH_NLO
aceptancia  = ttH_cut_ev / (1. * ttH_init_sim_eve)
luminosidad = 100

ttH_fidcross = cross * aceptancia
ttH_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', ttH_fidcross)
print('Events expected: ', ttH_expected)

('fiducial cross section: ', 0.12505608)
('Events expected: ', 12.505607999999999)


### [x] ttZ 
- [x] Done
39 eventos esperados con MET > 100

In [8]:
Folder    = '../data/wagner-bkgs-varios/'
datFolder = '../data/wagner-bkgs-varios/'

ttZ_runs  = 1

ttZ_init_ev = 0
ttZ_cut_ev  = 0

ttZ_photon  = []
ttZ_jet     = []
ttZ_lepton  = []
ttZ_MET     = []
ttZ_hadinfo = []


counter_bad_jets = 0
contador_tau = 0
contador_btag = 0
for ii in range(1, 1 + ttZ_runs):
    print(ii)
    if ii not in [0]:
        inputevents = LHCO_reader.Events(f_name = Folder + 'ttZ-tests.lhco')

        ttZ_init_ev += len(inputevents)
        print('Esta run tuvo ' + str(len(inputevents)) + ' eventos en el lhco')
        
        for i in range(len(inputevents)):

            num_jets = 0
    #        num_btag = 0
            HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets


            # at least one: photon, jet and lepton (electron or muon)
            if inputevents[i].number()["photon"] > 0 and inputevents[i].number()["jet"] > 0 and (inputevents[i].number()["electron"] > 0 or inputevents[i].number()["muon"] > 0):
                if inputevents[i]["jet"][0]["PT"] > 100 and -4.5 < inputevents[i]["jet"][0]["eta"] < 4.5:
                    if inputevents[i]["photon"][0]["PT"] > 10 and -2.37 < inputevents[i]["photon"][0]["eta"] < 2.37:
                        if inputevents[i]["MET"][0]["PT"] > 100:
                            # lepton info
                            if (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] > 0):
                                if inputevents[i]["electron"][0]["PT"] > inputevents[i]["muon"][0]["PT"]: 
                                    flag = 'electron'
                                    eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                                else: 
                                    flag = 'muon'
                                    eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron
                            elif (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] == 0): 
                                flag = 'electron'
                                eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                            elif (inputevents[i].number()["muon"] > 0) and (inputevents[i].number()["electron"] == 0): 
                                flag = 'muon'
                                eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron

                            if (-2.47 < eta_leading_lepton < -1.52) or (-1.37 < eta_leading_lepton < 1.37) or (1.52 < eta_leading_lepton < 2.47): 

                                # how many jets (total) and total hadronic pT
                                flag_bad_jets = 0                            
                                num_jets = 0
                                HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
                                HT = 0 # total HT, defined as the scalar sum of the pT of all reconstructed particles

                                for j in range(inputevents[i].number()["jet"]):
                                    if (inputevents[i]["jet"][j]["PT"] > 20) and (-4.5 < inputevents[i]["jet"][j]["eta"] < 4.5):
                                        num_jets += 1
                                        HT_had += inputevents[i]["jet"][j]["PT"]
                                        HT += inputevents[i]["jet"][j]["PT"]
                                    if (np.abs(inputevents[i]["jet"][j]["eta"]) > 4.5) or (inputevents[i]["jet"][j]["PT"] < 20):
                                        flag_bad_jets = 1

                                # how many photons                         
                                num_photons = 0
                                for j in range(inputevents[i].number()["photon"]):
                                    if inputevents[i]["photon"][j]["PT"] > 10 and -2.37 < inputevents[i]["photon"][j]["eta"] < 2.37:
                                        num_photons = num_photons + 1
                                        HT += inputevents[i]["photon"][j]["PT"]

                                # how many muons                        
                                num_muons = 0
                                for j in range(inputevents[i].number()["muon"]):
                                    if inputevents[i]["muon"][j]["PT"] > 10 and -2.7 < inputevents[i]["muon"][j]["eta"] < 2.7:
                                        num_muons = num_muons + 1
                                        HT += inputevents[i]["muon"][j]["PT"]

                                # how many electrons                        
                                num_electrons = 0
                                for j in range(inputevents[i].number()["electron"]):
                                    if inputevents[i]["electron"][j]["PT"] > 10:
                                        eta = inputevents[i]["electron"][j]["eta"]
                                        if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                            num_electrons = num_electrons + 1
                                            HT += inputevents[i]["electron"][j]["PT"]

                                if flag_bad_jets == 1: counter_bad_jets += 1 
                                if flag_bad_jets == 0:
                                    num_taus = 0
                                    if inputevents[i].number()["tau"] > 0: 
                                        contador_tau = contador_tau + 1
                                        # how many taus                    
                                        for j in range(inputevents[i].number()["tau"]):
                                            if inputevents[i]["tau"][j]["PT"] > 20:
                                                eta = inputevents[i]["tau"][j]["eta"]
                                                if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                                    num_taus = num_taus + 1
                                                    HT += inputevents[i]["tau"][j]["PT"]
                                                    
                                    if inputevents[i]["jet"][0]['btag'] == 1: contador_btag = contador_btag + 1

                                    # photon info:
                                    with open(datFolder + 'photon_bu.dat', 'a') as f:
                                        f.write('%f %f %f %d\n' % (inputevents[i]["photon"][0]["PT"],inputevents[i]["photon"][0]["eta"],inputevents[i]["photon"][0]["phi"], num_photons))

                                    # jet info:
                                    with open(datFolder + 'jet_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["jet"][0]["PT"],inputevents[i]["jet"][0]["eta"],inputevents[i]["jet"][0]["phi"]))

                                    # lepton info
                                    num_leptons = num_electrons + num_muons
                                    if flag == "electron":
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["electron"][0]["PT"],inputevents[i]["electron"][0]["eta"],inputevents[i]["electron"][0]["phi"], num_leptons, num_taus))
                                    else:
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["muon"][0]["PT"],inputevents[i]["muon"][0]["eta"],inputevents[i]["muon"][0]["phi"], num_leptons, num_taus))

                                    # MET (eta=0 by def, but I save phi because I have many jets)
                                    with open(datFolder + 'MET_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))

                                    # hadronic info
                                    with open(datFolder + 'hadinfo_bu.dat', 'a') as f:
                                        f.write('%f %f %d\n' % (HT, HT_had, num_jets))

                                    # total events that passed the cuts
                                    ttZ_cut_ev += 1

print('Num initial events: ', ttZ_init_ev)
print('Total pasaron los cortes: ', ttZ_cut_ev)
print('Hay: ', contador_tau, ' taus')
print('Hay: ', counter_bad_jets, ' eventos con al menos un jet malo')
print('Hay: ', contador_btag, ' con btag en leading jet')

1
Esta run tuvo 200000 eventos en el lhco
('Num initial events: ', 200000)
('Total pasaron los cortes: ', 2075)
('Hay: ', 129, ' taus')
('Hay: ', 33, ' eventos con al menos un jet malo')
('Hay: ', 822, ' con btag en leading jet')


In [11]:
ttZ_cross = 0.03795 #[pb] integrated cross-section without matching
ttZ_NLO   = 1. # k-factor [14 TeV] CHECK THIS
ttZ_init_sim_eve = 200000 * ttZ_runs

cross       = ttZ_cross * 1000 * ttZ_NLO
aceptancia  = ttZ_cut_ev / (1. * ttZ_init_sim_eve)
luminosidad = 100

ttZ_fidcross = cross * aceptancia
ttZ_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', ttZ_fidcross)
print('Events expected: ', ttZ_expected)

('fiducial cross section: ', 0.39373125)
('Events expected: ', 39.373124999999995)


### [x] ttW 
- [x] Done
    - (28 esperados con MET > 100)

In [4]:
!ls ../data/wagner-bkgs-varios

cross-wagner-muchosprocesos  ttH-tests.lhco  ttZ-tests.lhco
tH-tests.lhco		     ttj-tests.lhco  WH-tests.lhco
ttgamma-tests.lhco	     ttW-tests.lhco  ZH-tests.lhco


In [5]:
Folder    = '../data/wagner-bkgs-varios/'
datFolder = '../data/wagner-bkgs-varios/'

ttW_runs  = 1

ttW_init_ev = 0
ttW_cut_ev  = 0

ttW_photon  = []
ttW_jet     = []
ttW_lepton  = []
ttW_MET     = []
ttW_hadinfo = []


counter_bad_jets = 0
contador_tau = 0
contador_btag = 0
for ii in range(1, 1 + ttW_runs):
    print(ii)
    if ii not in [0]:
        inputevents = LHCO_reader.Events(f_name = Folder + 'ttW-tests.lhco')

        ttW_init_ev += len(inputevents)
        print('Esta run tuvo ' + str(len(inputevents)) + ' eventos en el lhco')
        
        for i in range(len(inputevents)):

            num_jets = 0
    #        num_btag = 0
            HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets


            # at least one: photon, jet and lepton (electron or muon)
            if inputevents[i].number()["photon"] > 0 and inputevents[i].number()["jet"] > 0 and (inputevents[i].number()["electron"] > 0 or inputevents[i].number()["muon"] > 0):
                if inputevents[i]["jet"][0]["PT"] > 100 and -4.5 < inputevents[i]["jet"][0]["eta"] < 4.5:
                    if inputevents[i]["photon"][0]["PT"] > 10 and -2.37 < inputevents[i]["photon"][0]["eta"] < 2.37:
                        if inputevents[i]["MET"][0]["PT"] > 100:
                            # lepton info
                            if (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] > 0):
                                if inputevents[i]["electron"][0]["PT"] > inputevents[i]["muon"][0]["PT"]: 
                                    flag = 'electron'
                                    eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                                else: 
                                    flag = 'muon'
                                    eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron
                            elif (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] == 0): 
                                flag = 'electron'
                                eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                            elif (inputevents[i].number()["muon"] > 0) and (inputevents[i].number()["electron"] == 0): 
                                flag = 'muon'
                                eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron

                            if (-2.47 < eta_leading_lepton < -1.52) or (-1.37 < eta_leading_lepton < 1.37) or (1.52 < eta_leading_lepton < 2.47): 

                                # how many jets (total) and total hadronic pT
                                flag_bad_jets = 0                            
                                num_jets = 0
                                HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
                                HT = 0 # total HT, defined as the scalar sum of the pT of all reconstructed particles

                                for j in range(inputevents[i].number()["jet"]):
                                    if (inputevents[i]["jet"][j]["PT"] > 20) and (-4.5 < inputevents[i]["jet"][j]["eta"] < 4.5):
                                        num_jets += 1
                                        HT_had += inputevents[i]["jet"][j]["PT"]
                                        HT += inputevents[i]["jet"][j]["PT"]
                                    if (np.abs(inputevents[i]["jet"][j]["eta"]) > 4.5) or (inputevents[i]["jet"][j]["PT"] < 20):
                                        flag_bad_jets = 1

                                # how many photons                         
                                num_photons = 0
                                for j in range(inputevents[i].number()["photon"]):
                                    if inputevents[i]["photon"][j]["PT"] > 10 and -2.37 < inputevents[i]["photon"][j]["eta"] < 2.37:
                                        num_photons = num_photons + 1
                                        HT += inputevents[i]["photon"][j]["PT"]

                                # how many muons                        
                                num_muons = 0
                                for j in range(inputevents[i].number()["muon"]):
                                    if inputevents[i]["muon"][j]["PT"] > 10 and -2.7 < inputevents[i]["muon"][j]["eta"] < 2.7:
                                        num_muons = num_muons + 1
                                        HT += inputevents[i]["muon"][j]["PT"]

                                # how many electrons                        
                                num_electrons = 0
                                for j in range(inputevents[i].number()["electron"]):
                                    if inputevents[i]["electron"][j]["PT"] > 10:
                                        eta = inputevents[i]["electron"][j]["eta"]
                                        if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                            num_electrons = num_electrons + 1
                                            HT += inputevents[i]["electron"][j]["PT"]

                                if flag_bad_jets == 1: counter_bad_jets += 1 
                                if flag_bad_jets == 0:
                                    num_taus = 0
                                    if inputevents[i].number()["tau"] > 0: 
                                        contador_tau = contador_tau + 1
                                        # how many taus                    
                                        for j in range(inputevents[i].number()["tau"]):
                                            if inputevents[i]["tau"][j]["PT"] > 20:
                                                eta = inputevents[i]["tau"][j]["eta"]
                                                if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                                    num_taus = num_taus + 1
                                                    HT += inputevents[i]["tau"][j]["PT"]
                                                    
                                    if inputevents[i]["jet"][0]['btag'] == 1: contador_btag = contador_btag + 1

                                    # photon info:
                                    with open(datFolder + 'photon_bu.dat', 'a') as f:
                                        f.write('%f %f %f %d\n' % (inputevents[i]["photon"][0]["PT"],inputevents[i]["photon"][0]["eta"],inputevents[i]["photon"][0]["phi"], num_photons))

                                    # jet info:
                                    with open(datFolder + 'jet_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["jet"][0]["PT"],inputevents[i]["jet"][0]["eta"],inputevents[i]["jet"][0]["phi"]))

                                    # lepton info
                                    num_leptons = num_electrons + num_muons
                                    if flag == "electron":
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["electron"][0]["PT"],inputevents[i]["electron"][0]["eta"],inputevents[i]["electron"][0]["phi"], num_leptons, num_taus))
                                    else:
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["muon"][0]["PT"],inputevents[i]["muon"][0]["eta"],inputevents[i]["muon"][0]["phi"], num_leptons, num_taus))

                                    # MET (eta=0 by def, but I save phi because I have many jets)
                                    with open(datFolder + 'MET_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))

                                    # hadronic info
                                    with open(datFolder + 'hadinfo_bu.dat', 'a') as f:
                                        f.write('%f %f %d\n' % (HT, HT_had, num_jets))

                                    # total events that passed the cuts
                                    ttW_cut_ev += 1

print('Num initial events: ', ttW_init_ev)
print('Total pasaron los cortes: ', ttW_cut_ev)
print('Hay: ', contador_tau, ' taus')
print('Hay: ', counter_bad_jets, ' eventos con al menos un jet malo')
print('Hay: ', contador_btag, ' con btag en leading jet')

1


/home/martinrios/.virtualenvs/pheno2.7/lib/python2.7/site-packages/LHCO_reader/LHCO_reader.py:359: UserWarning: Couldn't read total number of events from LHCO
  warnings.warn("Couldn't read total number of events from LHCO")


Esta run tuvo 500000 eventos en el lhco
('Num initial events: ', 500000)
('Total pasaron los cortes: ', 15413)
('Hay: ', 829, ' taus')
('Hay: ', 92, ' eventos con al menos un jet malo')
('Hay: ', 6131, ' con btag en leading jet')


In [7]:
ttW_cross = 0.009259 #[pb] integrated cross-section without matching
ttW_NLO   = 1. # k-factor [14 TeV] CHECK THIS
ttW_init_sim_eve = 500000 * ttW_runs

cross       = ttW_cross * 1000 * ttW_NLO
aceptancia  = ttW_cut_ev / (1. * ttW_init_sim_eve)
luminosidad = 100

ttW_fidcross = cross * aceptancia
ttW_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', ttW_fidcross)
print('Events expected: ', ttW_expected)

('fiducial cross section: ', 0.285417934)
('Events expected: ', 28.5417934)


### single tops 
- [x] Done
    - se esperan (790.70 + 1152.578 + 1272.042 = 3215.32) con MET > 100
    - se esperan (93.48 + 129.37 + 219.75 = 442.6) con MET > 200

#### single top + j
- [x] Done
    - se esperran 790.70 con MET > 100
    - se esperran 93.48 con MET > 200

In [25]:
Folder    = '../data/singleTop+j/lhco/'
datFolder = '../data/singleTop+j/'

singletj_runs  = 1

singletj_init_ev = 0
singletj_cut_ev  = 0

singletj_photon  = []
singletj_jet     = []
singletj_lepton  = []
singletj_MET     = []
singletj_hadinfo = []


counter_bad_jets = 0
contador_tau = 0
contador_btag = 0
for ii in range(1, 1 + singletj_runs):
    print(ii)
    if ii not in [0]:
        inputevents = LHCO_reader.Events(f_name = Folder + 'single-top+j_1M.lhco')

        singletj_init_ev += len(inputevents)
        print('Esta run tuvo ' + str(len(inputevents)) + ' eventos en el lhco')
        
        for i in range(len(inputevents)):

            num_jets = 0
    #        num_btag = 0
            HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets


            # at least one: photon, jet and lepton (electron or muon)
            if inputevents[i].number()["photon"] > 0 and inputevents[i].number()["jet"] > 0 and (inputevents[i].number()["electron"] > 0 or inputevents[i].number()["muon"] > 0):
                if inputevents[i]["jet"][0]["PT"] > 100 and -4.5 < inputevents[i]["jet"][0]["eta"] < 4.5:
                    if inputevents[i]["photon"][0]["PT"] > 10 and -2.37 < inputevents[i]["photon"][0]["eta"] < 2.37:
                        if inputevents[i]["MET"][0]["PT"] > 100:
                            # lepton info
                            if (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] > 0):
                                if inputevents[i]["electron"][0]["PT"] > inputevents[i]["muon"][0]["PT"]: 
                                    flag = 'electron'
                                    eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                                else: 
                                    flag = 'muon'
                                    eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron
                            elif (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] == 0): 
                                flag = 'electron'
                                eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                            elif (inputevents[i].number()["muon"] > 0) and (inputevents[i].number()["electron"] == 0): 
                                flag = 'muon'
                                eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron

                            if (-2.47 < eta_leading_lepton < -1.52) or (-1.37 < eta_leading_lepton < 1.37) or (1.52 < eta_leading_lepton < 2.47): 

                                # how many jets (total) and total hadronic pT
                                flag_bad_jets = 0                            
                                num_jets = 0
                                HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
                                HT = 0 # total HT, defined as the scalar sum of the pT of all reconstructed particles

                                for j in range(inputevents[i].number()["jet"]):
                                    if (inputevents[i]["jet"][j]["PT"] > 20) and (-4.5 < inputevents[i]["jet"][j]["eta"] < 4.5):
                                        num_jets += 1
                                        HT_had += inputevents[i]["jet"][j]["PT"]
                                        HT += inputevents[i]["jet"][j]["PT"]
                                    if (np.abs(inputevents[i]["jet"][j]["eta"]) > 4.5) or (inputevents[i]["jet"][j]["PT"] < 20):
                                        flag_bad_jets = 1

                                # how many photons                         
                                num_photons = 0
                                for j in range(inputevents[i].number()["photon"]):
                                    if inputevents[i]["photon"][j]["PT"] > 10 and -2.37 < inputevents[i]["photon"][j]["eta"] < 2.37:
                                        num_photons = num_photons + 1
                                        HT += inputevents[i]["photon"][j]["PT"]

                                # how many muons                        
                                num_muons = 0
                                for j in range(inputevents[i].number()["muon"]):
                                    if inputevents[i]["muon"][j]["PT"] > 10 and -2.7 < inputevents[i]["muon"][j]["eta"] < 2.7:
                                        num_muons = num_muons + 1
                                        HT += inputevents[i]["muon"][j]["PT"]

                                # how many electrons                        
                                num_electrons = 0
                                for j in range(inputevents[i].number()["electron"]):
                                    if inputevents[i]["electron"][j]["PT"] > 10:
                                        eta = inputevents[i]["electron"][j]["eta"]
                                        if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                            num_electrons = num_electrons + 1
                                            HT += inputevents[i]["electron"][j]["PT"]

                                if flag_bad_jets == 1: counter_bad_jets += 1 
                                if flag_bad_jets == 0:
                                    num_taus = 0
                                    if inputevents[i].number()["tau"] > 0: 
                                        contador_tau = contador_tau + 1
                                        # how many taus                    
                                        for j in range(inputevents[i].number()["tau"]):
                                            if inputevents[i]["tau"][j]["PT"] > 20:
                                                eta = inputevents[i]["tau"][j]["eta"]
                                                if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                                    num_taus = num_taus + 1
                                                    HT += inputevents[i]["tau"][j]["PT"]
                                                    
                                    if inputevents[i]["jet"][0]['btag'] == 1: contador_btag = contador_btag + 1

                                    # photon info:
                                    with open(datFolder + 'photon_bu.dat', 'a') as f:
                                        f.write('%f %f %f %d\n' % (inputevents[i]["photon"][0]["PT"],inputevents[i]["photon"][0]["eta"],inputevents[i]["photon"][0]["phi"], num_photons))

                                    # jet info:
                                    with open(datFolder + 'jet_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["jet"][0]["PT"],inputevents[i]["jet"][0]["eta"],inputevents[i]["jet"][0]["phi"]))

                                    # lepton info
                                    num_leptons = num_electrons + num_muons
                                    if flag == "electron":
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["electron"][0]["PT"],inputevents[i]["electron"][0]["eta"],inputevents[i]["electron"][0]["phi"], num_leptons, num_taus))
                                    else:
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["muon"][0]["PT"],inputevents[i]["muon"][0]["eta"],inputevents[i]["muon"][0]["phi"], num_leptons, num_taus))

                                    # MET (eta=0 by def, but I save phi because I have many jets)
                                    with open(datFolder + 'MET_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))

                                    # hadronic info
                                    with open(datFolder + 'hadinfo_bu.dat', 'a') as f:
                                        f.write('%f %f %d\n' % (HT, HT_had, num_jets))

                                    # total events that passed the cuts
                                    singletj_cut_ev += 1

print('Num initial events: ', singletj_init_ev)
print('Total pasaron los cortes: ', singletj_cut_ev)
print('Hay: ', contador_tau, ' taus')
print('Hay: ', counter_bad_jets, ' eventos con al menos un jet malo')
print('Hay: ', contador_btag, ' con btag en leading jet')

1
Esta run tuvo 1000000 eventos en el lhco
('Num initial events: ', 1000000)
('Total pasaron los cortes: ', 4804)
('Hay: ', 158, ' taus')
('Hay: ', 54, ' eventos con al menos un jet malo')
('Hay: ', 228, ' con btag en leading jet')


In [26]:
singletj_cross = 1.598 #[pb] integrated cross-section without matching
singletj_NLO   = 1.03 # k-factor [13 TeV]
singletj_init_sim_eve = 1000000 * singletj_runs

cross       = singletj_cross * 1000 * singletj_NLO
aceptancia  = singletj_cut_ev / (1. * singletj_init_sim_eve)
luminosidad = 100

singletj_fidcross = cross * aceptancia
singletj_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', singletj_fidcross)
print('Events expected: ', singletj_expected)

('fiducial cross section: ', 7.907095760000001)
('Events expected: ', 790.7095760000001)


#### single top + b
- [x] Done
    - se esperan 1152.578 con MET > 100
    - se esperan 129.37 con MET > 200

In [9]:
!ls ../data/singleTop+b/lhco

single-top+b.lhco


In [29]:
Folder    = '../data/singleTop+b/lhco/'
datFolder = '../data/singleTop+b/'

singletb_runs  = 2

singletb_init_ev = 0
singletb_cut_ev  = 0

singletb_photon  = []
singletb_jet     = []
singletb_lepton  = []
singletb_MET     = []
singletb_hadinfo = []


counter_bad_jets = 0
contador_tau = 0
contador_btag = 0
for ii in range(1, 1 + singletb_runs):
    print(ii)
    if ii not in [0]:
        inputevents = LHCO_reader.Events(f_name = Folder + 'single-top+b_1M_' + str(ii) + '.lhco')

        singletb_init_ev += len(inputevents)
        print('Esta run tuvo ' + str(len(inputevents)) + ' eventos en el lhco')
        
        for i in range(len(inputevents)):

            num_jets = 0
    #        num_btag = 0
            HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets


            # at least one: photon, jet and lepton (electron or muon)
            if inputevents[i].number()["photon"] > 0 and inputevents[i].number()["jet"] > 0 and (inputevents[i].number()["electron"] > 0 or inputevents[i].number()["muon"] > 0):
                if inputevents[i]["jet"][0]["PT"] > 100 and -4.5 < inputevents[i]["jet"][0]["eta"] < 4.5:
                    if inputevents[i]["photon"][0]["PT"] > 10 and -2.37 < inputevents[i]["photon"][0]["eta"] < 2.37:
                        if inputevents[i]["MET"][0]["PT"] > 100:
                            # lepton info
                            if (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] > 0):
                                if inputevents[i]["electron"][0]["PT"] > inputevents[i]["muon"][0]["PT"]: 
                                    flag = 'electron'
                                    eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                                else: 
                                    flag = 'muon'
                                    eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron
                            elif (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] == 0): 
                                flag = 'electron'
                                eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                            elif (inputevents[i].number()["muon"] > 0) and (inputevents[i].number()["electron"] == 0): 
                                flag = 'muon'
                                eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron

                            if (-2.47 < eta_leading_lepton < -1.52) or (-1.37 < eta_leading_lepton < 1.37) or (1.52 < eta_leading_lepton < 2.47): 

                                # how many jets (total) and total hadronic pT
                                flag_bad_jets = 0                            
                                num_jets = 0
                                HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
                                HT = 0 # total HT, defined as the scalar sum of the pT of all reconstructed particles

                                for j in range(inputevents[i].number()["jet"]):
                                    if (inputevents[i]["jet"][j]["PT"] > 20) and (-4.5 < inputevents[i]["jet"][j]["eta"] < 4.5):
                                        num_jets += 1
                                        HT_had += inputevents[i]["jet"][j]["PT"]
                                        HT += inputevents[i]["jet"][j]["PT"]
                                    if (np.abs(inputevents[i]["jet"][j]["eta"]) > 4.5) or (inputevents[i]["jet"][j]["PT"] < 20):
                                        flag_bad_jets = 1

                                # how many photons                         
                                num_photons = 0
                                for j in range(inputevents[i].number()["photon"]):
                                    if inputevents[i]["photon"][j]["PT"] > 10 and -2.37 < inputevents[i]["photon"][j]["eta"] < 2.37:
                                        num_photons = num_photons + 1
                                        HT += inputevents[i]["photon"][j]["PT"]

                                # how many muons                        
                                num_muons = 0
                                for j in range(inputevents[i].number()["muon"]):
                                    if inputevents[i]["muon"][j]["PT"] > 10 and -2.7 < inputevents[i]["muon"][j]["eta"] < 2.7:
                                        num_muons = num_muons + 1
                                        HT += inputevents[i]["muon"][j]["PT"]

                                # how many electrons                        
                                num_electrons = 0
                                for j in range(inputevents[i].number()["electron"]):
                                    if inputevents[i]["electron"][j]["PT"] > 10:
                                        eta = inputevents[i]["electron"][j]["eta"]
                                        if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                            num_electrons = num_electrons + 1
                                            HT += inputevents[i]["electron"][j]["PT"]

                                if flag_bad_jets == 1: counter_bad_jets += 1 
                                if flag_bad_jets == 0:
                                    num_taus = 0
                                    if inputevents[i].number()["tau"] > 0: 
                                        contador_tau = contador_tau + 1
                                        # how many taus                    
                                        for j in range(inputevents[i].number()["tau"]):
                                            if inputevents[i]["tau"][j]["PT"] > 20:
                                                eta = inputevents[i]["tau"][j]["eta"]
                                                if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                                    num_taus = num_taus + 1
                                                    HT += inputevents[i]["tau"][j]["PT"]
                                                    
                                    if inputevents[i]["jet"][0]['btag'] == 1: contador_btag = contador_btag + 1

                                    # photon info:
                                    with open(datFolder + 'photon_4.dat', 'a') as f:
                                        f.write('%f %f %f %d\n' % (inputevents[i]["photon"][0]["PT"],inputevents[i]["photon"][0]["eta"],inputevents[i]["photon"][0]["phi"], num_photons))

                                    # jet info:
                                    with open(datFolder + 'jet_4.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["jet"][0]["PT"],inputevents[i]["jet"][0]["eta"],inputevents[i]["jet"][0]["phi"]))

                                    # lepton info
                                    num_leptons = num_electrons + num_muons
                                    if flag == "electron":
                                        with open(datFolder + 'lepton_4.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["electron"][0]["PT"],inputevents[i]["electron"][0]["eta"],inputevents[i]["electron"][0]["phi"], num_leptons, num_taus))
                                    else:
                                        with open(datFolder + 'lepton_4.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["muon"][0]["PT"],inputevents[i]["muon"][0]["eta"],inputevents[i]["muon"][0]["phi"], num_leptons, num_taus))

                                    # MET (eta=0 by def, but I save phi because I have many jets)
                                    with open(datFolder + 'MET_4.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))

                                    # hadronic info
                                    with open(datFolder + 'hadinfo_4.dat', 'a') as f:
                                        f.write('%f %f %d\n' % (HT, HT_had, num_jets))

                                    # total events that passed the cuts
                                    singletb_cut_ev += 1

print('Num initial events: ', singletb_init_ev)
print('Total pasaron los cortes: ', singletb_cut_ev)
print('Hay: ', contador_tau, ' taus')
print('Hay: ', counter_bad_jets, ' eventos con al menos un jet malo')
print('Hay: ', contador_btag, ' con btag en leading jet')

1
Esta run tuvo 938114 eventos en el lhco
2
Esta run tuvo 937507 eventos en el lhco
('Num initial events: ', 1875621)
('Total pasaron los cortes: ', 5969)
('Hay: ', 233, ' taus')
('Hay: ', 118, ' eventos con al menos un jet malo')
('Hay: ', 1771, ' con btag en leading jet')


In [30]:
singletb_cross = 2.882 #[pb] integrated cross-section without matching
singletb_NLO   = 1.34 # k-factor [13 TeV]
singletb_init_sim_eve = 1000000 * singletb_runs

cross       = singletb_cross * 1000 * singletb_NLO
aceptancia  = singletb_cut_ev / (1. * singletb_init_sim_eve)
luminosidad = 100

singletb_fidcross = cross * aceptancia
singletb_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', singletb_fidcross)
print('Events expected: ', singletb_expected)

('fiducial cross section: ', 11.525780860000001)
('Events expected: ', 1152.5780860000002)


#### Single top + w
- [x] Done
    - se esperan 1272.042 con MET > 100
    - se esperan 219.75 con MET > 200

In [33]:
Folder    = '../data/singleTop+W/lhco/'
datFolder = '../data/singleTop+W/'

singlet_runs  = 2

singlet_init_ev = 0
singlet_cut_ev  = 0

singlet_photon  = []
singlet_jet     = []
singlet_lepton  = []
singlet_MET     = []
singlet_hadinfo = []


counter_bad_jets = 0
contador_tau = 0
contador_btag = 0
for ii in range(1, 1 + singlet_runs):
    print(ii)
    if ii not in [0]:
        inputevents = LHCO_reader.Events(f_name = Folder + 'single-top+W_1M_' + str(ii) + '.lhco')

        singlet_init_ev += len(inputevents)
        print('Esta run tuvo ' + str(len(inputevents)) + ' eventos en el lhco')
        
        for i in range(len(inputevents)):

            num_jets = 0
    #        num_btag = 0
            HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets


            # at least one: photon, jet and lepton (electron or muon)
            if inputevents[i].number()["photon"] > 0 and inputevents[i].number()["jet"] > 0 and (inputevents[i].number()["electron"] > 0 or inputevents[i].number()["muon"] > 0):
                if inputevents[i]["jet"][0]["PT"] > 100 and -4.5 < inputevents[i]["jet"][0]["eta"] < 4.5:
                    if inputevents[i]["photon"][0]["PT"] > 10 and -2.37 < inputevents[i]["photon"][0]["eta"] < 2.37:
                        if inputevents[i]["MET"][0]["PT"] > 100:
                            # lepton info
                            if (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] > 0):
                                if inputevents[i]["electron"][0]["PT"] > inputevents[i]["muon"][0]["PT"]: 
                                    flag = 'electron'
                                    eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                                else: 
                                    flag = 'muon'
                                    eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron
                            elif (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] == 0): 
                                flag = 'electron'
                                eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                            elif (inputevents[i].number()["muon"] > 0) and (inputevents[i].number()["electron"] == 0): 
                                flag = 'muon'
                                eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron

                            if (-2.47 < eta_leading_lepton < -1.52) or (-1.37 < eta_leading_lepton < 1.37) or (1.52 < eta_leading_lepton < 2.47): 

                                # how many jets (total) and total hadronic pT
                                flag_bad_jets = 0                            
                                num_jets = 0
                                HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
                                HT = 0 # total HT, defined as the scalar sum of the pT of all reconstructed particles

                                for j in range(inputevents[i].number()["jet"]):
                                    if (inputevents[i]["jet"][j]["PT"] > 20) and (-4.5 < inputevents[i]["jet"][j]["eta"] < 4.5):
                                        num_jets += 1
                                        HT_had += inputevents[i]["jet"][j]["PT"]
                                        HT += inputevents[i]["jet"][j]["PT"]
                                    if (np.abs(inputevents[i]["jet"][j]["eta"]) > 4.5) or (inputevents[i]["jet"][j]["PT"] < 20):
                                        flag_bad_jets = 1

                                # how many photons                         
                                num_photons = 0
                                for j in range(inputevents[i].number()["photon"]):
                                    if inputevents[i]["photon"][j]["PT"] > 10 and -2.37 < inputevents[i]["photon"][j]["eta"] < 2.37:
                                        num_photons = num_photons + 1
                                        HT += inputevents[i]["photon"][j]["PT"]

                                # how many muons                        
                                num_muons = 0
                                for j in range(inputevents[i].number()["muon"]):
                                    if inputevents[i]["muon"][j]["PT"] > 10 and -2.7 < inputevents[i]["muon"][j]["eta"] < 2.7:
                                        num_muons = num_muons + 1
                                        HT += inputevents[i]["muon"][j]["PT"]

                                # how many electrons                        
                                num_electrons = 0
                                for j in range(inputevents[i].number()["electron"]):
                                    if inputevents[i]["electron"][j]["PT"] > 10:
                                        eta = inputevents[i]["electron"][j]["eta"]
                                        if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                            num_electrons = num_electrons + 1
                                            HT += inputevents[i]["electron"][j]["PT"]

                                if flag_bad_jets == 1: counter_bad_jets += 1 
                                if flag_bad_jets == 0:
                                    num_taus = 0
                                    if inputevents[i].number()["tau"] > 0: 
                                        contador_tau = contador_tau + 1
                                        # how many taus                    
                                        for j in range(inputevents[i].number()["tau"]):
                                            if inputevents[i]["tau"][j]["PT"] > 20:
                                                eta = inputevents[i]["tau"][j]["eta"]
                                                if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                                    num_taus = num_taus + 1
                                                    HT += inputevents[i]["tau"][j]["PT"]
                                                    
                                    if inputevents[i]["jet"][0]['btag'] == 1: contador_btag = contador_btag + 1

                                    # photon info:
                                    with open(datFolder + 'photon_4.dat', 'a') as f:
                                        f.write('%f %f %f %d\n' % (inputevents[i]["photon"][0]["PT"],inputevents[i]["photon"][0]["eta"],inputevents[i]["photon"][0]["phi"], num_photons))

                                    # jet info:
                                    with open(datFolder + 'jet_4.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["jet"][0]["PT"],inputevents[i]["jet"][0]["eta"],inputevents[i]["jet"][0]["phi"]))

                                    # lepton info
                                    num_leptons = num_electrons + num_muons
                                    if flag == "electron":
                                        with open(datFolder + 'lepton_4.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["electron"][0]["PT"],inputevents[i]["electron"][0]["eta"],inputevents[i]["electron"][0]["phi"], num_leptons, num_taus))
                                    else:
                                        with open(datFolder + 'lepton_4.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["muon"][0]["PT"],inputevents[i]["muon"][0]["eta"],inputevents[i]["muon"][0]["phi"], num_leptons, num_taus))

                                    # MET (eta=0 by def, but I save phi because I have many jets)
                                    with open(datFolder + 'MET_4.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))

                                    # hadronic info
                                    with open(datFolder + 'hadinfo_4.dat', 'a') as f:
                                        f.write('%f %f %d\n' % (HT, HT_had, num_jets))

                                    # total events that passed the cuts
                                    singlet_cut_ev += 1

print('Num initial events: ', singlet_init_ev)
print('Total pasaron los cortes: ', singlet_cut_ev)
print('Hay: ', contador_tau, ' taus')
print('Hay: ', counter_bad_jets, ' eventos con al menos un jet malo')
print('Hay: ', contador_btag, ' con btag en leading jet')

1
Esta run tuvo 495691 eventos en el lhco
2
Esta run tuvo 496596 eventos en el lhco
('Num initial events: ', 992287)
('Total pasaron los cortes: ', 17655)
('Hay: ', 622, ' taus')
('Hay: ', 288, ' eventos con al menos un jet malo')
('Hay: ', 4957, ' con btag en leading jet')


In [34]:
singlet_cross = 1.441 #[pb] integrated cross-section without matching
singlet_NLO   = 1. # k-factor [13 TeV]
singlet_init_sim_eve = 1000000 * singlet_runs

cross       = singlet_cross * 1000 * singlet_NLO
aceptancia  = singlet_cut_ev / (1. * singlet_init_sim_eve)
luminosidad = 100

singlet_fidcross = cross * aceptancia
singlet_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', singlet_fidcross)
print('Events expected: ', singlet_expected)

('fiducial cross section: ', 12.7204275)
('Events expected: ', 1272.04275)


### dibosons
- [x] Done.
    - Se esperan (674.0334 + 84.3997 + 1582.03 = 2340.4631) con MET > 100
    - Se esperan (220.74 + 28.48 + 419.715 = 668.935) con MET > 200

#### wz
- [x] Done
    - se esperan 674.0334 con MET > 100
    - se esperan 220.74 con MET > 200

In [9]:
Folder    = '../data/wz/lhco/'
datFolder = '../data/wz/'

wz_runs  = 2

wz_init_ev = 0
wz_cut_ev  = 0

wz_photon  = []
wz_jet     = []
wz_lepton  = []
wz_MET     = []
wz_hadinfo = []


counter_bad_jets = 0
contador_tau = 0
contador_btag = 0
for ii in range(1, 1 + wz_runs):
    print(ii)
    if ii not in [0]:
        inputevents = LHCO_reader.Events(f_name = Folder + 'WZ_1.lhco')

        wz_init_ev += len(inputevents)
        print('Esta run tuvo ' + str(len(inputevents)) + ' eventos en el lhco')
        
        for i in range(len(inputevents)):

            num_jets = 0
    #        num_btag = 0
            HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets


            # at least one: photon, jet and lepton (electron or muon)
            if inputevents[i].number()["photon"] > 0 and inputevents[i].number()["jet"] > 0 and (inputevents[i].number()["electron"] > 0 or inputevents[i].number()["muon"] > 0):
                if inputevents[i]["jet"][0]["PT"] > 100 and -4.5 < inputevents[i]["jet"][0]["eta"] < 4.5:
                    if inputevents[i]["photon"][0]["PT"] > 10 and -2.37 < inputevents[i]["photon"][0]["eta"] < 2.37:
                        if inputevents[i]["MET"][0]["PT"] > 200:
                            # lepton info
                            if (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] > 0):
                                if inputevents[i]["electron"][0]["PT"] > inputevents[i]["muon"][0]["PT"]: 
                                    flag = 'electron'
                                    eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                                else: 
                                    flag = 'muon'
                                    eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron
                            elif (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] == 0): 
                                flag = 'electron'
                                eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                            elif (inputevents[i].number()["muon"] > 0) and (inputevents[i].number()["electron"] == 0): 
                                flag = 'muon'
                                eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron

                            if (-2.47 < eta_leading_lepton < -1.52) or (-1.37 < eta_leading_lepton < 1.37) or (1.52 < eta_leading_lepton < 2.47): 

                                # how many jets (total) and total hadronic pT
                                flag_bad_jets = 0                            
                                num_jets = 0
                                HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
                                HT = 0 # total HT, defined as the scalar sum of the pT of all reconstructed particles

                                for j in range(inputevents[i].number()["jet"]):
                                    if (inputevents[i]["jet"][j]["PT"] > 20) and (-4.5 < inputevents[i]["jet"][j]["eta"] < 4.5):
                                        num_jets += 1
                                        HT_had += inputevents[i]["jet"][j]["PT"]
                                        HT += inputevents[i]["jet"][j]["PT"]
                                    if (np.abs(inputevents[i]["jet"][j]["eta"]) > 4.5) or (inputevents[i]["jet"][j]["PT"] < 20):
                                        flag_bad_jets = 1

                                # how many photons                         
                                num_photons = 0
                                for j in range(inputevents[i].number()["photon"]):
                                    if inputevents[i]["photon"][j]["PT"] > 10 and -2.37 < inputevents[i]["photon"][j]["eta"] < 2.37:
                                        num_photons = num_photons + 1
                                        HT += inputevents[i]["photon"][j]["PT"]

                                # how many muons                        
                                num_muons = 0
                                for j in range(inputevents[i].number()["muon"]):
                                    if inputevents[i]["muon"][j]["PT"] > 10 and -2.7 < inputevents[i]["muon"][j]["eta"] < 2.7:
                                        num_muons = num_muons + 1
                                        HT += inputevents[i]["muon"][j]["PT"]

                                # how many electrons                        
                                num_electrons = 0
                                for j in range(inputevents[i].number()["electron"]):
                                    if inputevents[i]["electron"][j]["PT"] > 10:
                                        eta = inputevents[i]["electron"][j]["eta"]
                                        if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                            num_electrons = num_electrons + 1
                                            HT += inputevents[i]["electron"][j]["PT"]

                                if flag_bad_jets == 1: counter_bad_jets += 1 
                                if flag_bad_jets == 0:
                                    num_taus = 0
                                    if inputevents[i].number()["tau"] > 0: 
                                        contador_tau = contador_tau + 1
                                        # how many taus                    
                                        for j in range(inputevents[i].number()["tau"]):
                                            if inputevents[i]["tau"][j]["PT"] > 20:
                                                eta = inputevents[i]["tau"][j]["eta"]
                                                if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                                    num_taus = num_taus + 1
                                                    HT += inputevents[i]["tau"][j]["PT"]
                                                    
                                    if inputevents[i]["jet"][0]['btag'] == 1: contador_btag = contador_btag + 1

                                    # photon info:
                                    with open(datFolder + 'photon_6.dat', 'a') as f:
                                        f.write('%f %f %f %d\n' % (inputevents[i]["photon"][0]["PT"],inputevents[i]["photon"][0]["eta"],inputevents[i]["photon"][0]["phi"], num_photons))

                                    # jet info:
                                    with open(datFolder + 'jet_6.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["jet"][0]["PT"],inputevents[i]["jet"][0]["eta"],inputevents[i]["jet"][0]["phi"]))

                                    # lepton info
                                    num_leptons = num_electrons + num_muons
                                    if flag == "electron":
                                        with open(datFolder + 'lepton_6.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["electron"][0]["PT"],inputevents[i]["electron"][0]["eta"],inputevents[i]["electron"][0]["phi"], num_leptons, num_taus))
                                    else:
                                        with open(datFolder + 'lepton_6.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["muon"][0]["PT"],inputevents[i]["muon"][0]["eta"],inputevents[i]["muon"][0]["phi"], num_leptons, num_taus))

                                    # MET (eta=0 by def, but I save phi because I have many jets)
                                    with open(datFolder + 'MET_6.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))

                                    # hadronic info
                                    with open(datFolder + 'hadinfo_6.dat', 'a') as f:
                                        f.write('%f %f %d\n' % (HT, HT_had, num_jets))

                                    # total events that passed the cuts
                                    wz_cut_ev += 1

print('Num initial events: ', wz_init_ev)
print('Total pasaron los cortes: ', wz_cut_ev)
print('Hay: ', contador_tau, ' taus')
print('Hay: ', counter_bad_jets, ' eventos con al menos un jet malo')
print('Hay: ', contador_btag, ' con btag en leading jet')

1
Esta run tuvo 856284 eventos en el lhco
2
Esta run tuvo 856284 eventos en el lhco
('Num initial events: ', 1712568)
('Total pasaron los cortes: ', 3028)
('Hay: ', 70, ' taus')
('Hay: ', 44, ' eventos con al menos un jet malo')
('Hay: ', 66, ' con btag en leading jet')


In [10]:
wz_cross = 0.90 #[pb] integrated cross-section without matching
wz_NLO   = 1.62 # k-factor [14 TeV]
wz_init_sim_eve = 1000000 * wz_runs

cross       = wz_cross * 1000 * wz_NLO
aceptancia  = wz_cut_ev / (1. * wz_init_sim_eve)
luminosidad = 100

wz_fidcross = cross * aceptancia
wz_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', wz_fidcross)
print('Events expected: ', wz_expected)

('fiducial cross section: ', 2.2074119999999997)
('Events expected: ', 220.74119999999996)


#### zz
- [x] Done
    - se esperan 84.3997464 con MET > 100
    - se esperan 28.48 con MET > 200

In [16]:
Folder    = '../data/zz/lhco/'
datFolder = '../data/zz/'

zz_runs  = 1

zz_init_ev = 0
zz_cut_ev  = 0

zz_photon  = []
zz_jet     = []
zz_lepton  = []
zz_MET     = []
zz_hadinfo = []


counter_bad_jets = 0
contador_tau = 0
contador_btag = 0
for ii in range(1, 1 + zz_runs):
    print(ii)
    if ii not in [0]:
        inputevents = LHCO_reader.Events(f_name = Folder + 'ZZ_1.lhco')

        zz_init_ev += len(inputevents)
        print('Esta run tuvo ' + str(len(inputevents)) + ' eventos en el lhco')
        
        for i in range(len(inputevents)):

            num_jets = 0
    #        num_btag = 0
            HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets


            # at least one: photon, jet and lepton (electron or muon)
            if inputevents[i].number()["photon"] > 0 and inputevents[i].number()["jet"] > 0 and (inputevents[i].number()["electron"] > 0 or inputevents[i].number()["muon"] > 0):
                if inputevents[i]["jet"][0]["PT"] > 100 and -4.5 < inputevents[i]["jet"][0]["eta"] < 4.5:
                    if inputevents[i]["photon"][0]["PT"] > 10 and -2.37 < inputevents[i]["photon"][0]["eta"] < 2.37:
                        if inputevents[i]["MET"][0]["PT"] > 200:
                            # lepton info
                            if (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] > 0):
                                if inputevents[i]["electron"][0]["PT"] > inputevents[i]["muon"][0]["PT"]: 
                                    flag = 'electron'
                                    eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                                else: 
                                    flag = 'muon'
                                    eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron
                            elif (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] == 0): 
                                flag = 'electron'
                                eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                            elif (inputevents[i].number()["muon"] > 0) and (inputevents[i].number()["electron"] == 0): 
                                flag = 'muon'
                                eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron

                            if (-2.47 < eta_leading_lepton < -1.52) or (-1.37 < eta_leading_lepton < 1.37) or (1.52 < eta_leading_lepton < 2.47): 

                                # how many jets (total) and total hadronic pT
                                flag_bad_jets = 0                            
                                num_jets = 0
                                HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
                                HT = 0 # total HT, defined as the scalar sum of the pT of all reconstructed particles

                                for j in range(inputevents[i].number()["jet"]):
                                    if (inputevents[i]["jet"][j]["PT"] > 20) and (-4.5 < inputevents[i]["jet"][j]["eta"] < 4.5):
                                        num_jets += 1
                                        HT_had += inputevents[i]["jet"][j]["PT"]
                                        HT += inputevents[i]["jet"][j]["PT"]
                                    if (np.abs(inputevents[i]["jet"][j]["eta"]) > 4.5) or (inputevents[i]["jet"][j]["PT"] < 20):
                                        flag_bad_jets = 1

                                # how many photons                         
                                num_photons = 0
                                for j in range(inputevents[i].number()["photon"]):
                                    if inputevents[i]["photon"][j]["PT"] > 10 and -2.37 < inputevents[i]["photon"][j]["eta"] < 2.37:
                                        num_photons = num_photons + 1
                                        HT += inputevents[i]["photon"][j]["PT"]

                                # how many muons                        
                                num_muons = 0
                                for j in range(inputevents[i].number()["muon"]):
                                    if inputevents[i]["muon"][j]["PT"] > 10 and -2.7 < inputevents[i]["muon"][j]["eta"] < 2.7:
                                        num_muons = num_muons + 1
                                        HT += inputevents[i]["muon"][j]["PT"]

                                # how many electrons                        
                                num_electrons = 0
                                for j in range(inputevents[i].number()["electron"]):
                                    if inputevents[i]["electron"][j]["PT"] > 10:
                                        eta = inputevents[i]["electron"][j]["eta"]
                                        if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                            num_electrons = num_electrons + 1
                                            HT += inputevents[i]["electron"][j]["PT"]

                                if flag_bad_jets == 1: counter_bad_jets += 1 
                                if flag_bad_jets == 0:
                                    num_taus = 0
                                    if inputevents[i].number()["tau"] > 0: 
                                        contador_tau = contador_tau + 1
                                        # how many taus                    
                                        for j in range(inputevents[i].number()["tau"]):
                                            if inputevents[i]["tau"][j]["PT"] > 20:
                                                eta = inputevents[i]["tau"][j]["eta"]
                                                if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                                    num_taus = num_taus + 1
                                                    HT += inputevents[i]["tau"][j]["PT"]
                                                    
                                    if inputevents[i]["jet"][0]['btag'] == 1: contador_btag = contador_btag + 1

                                    # photon info:
                                    with open(datFolder + 'photon_bu.dat', 'a') as f:
                                        f.write('%f %f %f %d\n' % (inputevents[i]["photon"][0]["PT"],inputevents[i]["photon"][0]["eta"],inputevents[i]["photon"][0]["phi"], num_photons))

                                    # jet info:
                                    with open(datFolder + 'jet_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["jet"][0]["PT"],inputevents[i]["jet"][0]["eta"],inputevents[i]["jet"][0]["phi"]))

                                    # lepton info
                                    num_leptons = num_electrons + num_muons
                                    if flag == "electron":
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["electron"][0]["PT"],inputevents[i]["electron"][0]["eta"],inputevents[i]["electron"][0]["phi"], num_leptons, num_taus))
                                    else:
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["muon"][0]["PT"],inputevents[i]["muon"][0]["eta"],inputevents[i]["muon"][0]["phi"], num_leptons, num_taus))

                                    # MET (eta=0 by def, but I save phi because I have many jets)
                                    with open(datFolder + 'MET_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))

                                    # hadronic info
                                    with open(datFolder + 'hadinfo_bu.dat', 'a') as f:
                                        f.write('%f %f %d\n' % (HT, HT_had, num_jets))

                                    # total events that passed the cuts
                                    zz_cut_ev += 1

print('Num initial events: ', zz_init_ev)
print('Total pasaron los cortes: ', zz_cut_ev)
print('Hay: ', contador_tau, ' taus')
print('Hay: ', counter_bad_jets, ' eventos con al menos un jet malo')
print('Hay: ', contador_btag, ' con btag en leading jet')

1
Esta run tuvo 740888 eventos en el lhco
('Num initial events: ', 740888)
('Total pasaron los cortes: ', 844)
('Hay: ', 9, ' taus')
('Hay: ', 6, ' eventos con al menos un jet malo')
('Hay: ', 16, ' con btag en leading jet')


In [17]:
zz_cross = 0.2616 #[pb] metched cross-section
zz_NLO   = 1.29 # k-factor [14 TeV] CHECK THIS
zz_init_sim_eve = 1000000 * zz_runs

cross       = zz_cross * 1000 * zz_NLO
aceptancia  = zz_cut_ev / (1. * zz_init_sim_eve)
luminosidad = 100

zz_fidcross = cross * aceptancia
zz_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', zz_fidcross)
print('Events expected: ', zz_expected)

('fiducial cross section: ', 0.28481961600000005)
('Events expected: ', 28.481961600000005)


#### ww
- [x] Done
    - se esperan 1582.03 con MET > 100
    - se esperan 419.715 con MET > 200

In [21]:
Folder    = '../data/ww/lhco/'
datFolder = '../data/ww/'

ww_runs  = 1

ww_init_ev = 0
ww_cut_ev  = 0

ww_photon  = []
ww_jet     = []
ww_lepton  = []
ww_MET     = []
ww_hadinfo = []


counter_bad_jets = 0
contador_tau = 0
contador_btag = 0
for ii in range(1, 1 + ww_runs):
    print(ii)
    if ii not in [0]:
        inputevents = LHCO_reader.Events(f_name = Folder + 'WW_1.lhco')

        ww_init_ev += len(inputevents)
        print('Esta run tuvo ' + str(len(inputevents)) + ' eventos en el lhco')
        
        for i in range(len(inputevents)):

            num_jets = 0
    #        num_btag = 0
            HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets


            # at least one: photon, jet and lepton (electron or muon)
            if inputevents[i].number()["photon"] > 0 and inputevents[i].number()["jet"] > 0 and (inputevents[i].number()["electron"] > 0 or inputevents[i].number()["muon"] > 0):
                if inputevents[i]["jet"][0]["PT"] > 100 and -4.5 < inputevents[i]["jet"][0]["eta"] < 4.5:
                    if inputevents[i]["photon"][0]["PT"] > 10 and -2.37 < inputevents[i]["photon"][0]["eta"] < 2.37:
                        if inputevents[i]["MET"][0]["PT"] > 200:
                            # lepton info
                            if (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] > 0):
                                if inputevents[i]["electron"][0]["PT"] > inputevents[i]["muon"][0]["PT"]: 
                                    flag = 'electron'
                                    eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                                else: 
                                    flag = 'muon'
                                    eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron
                            elif (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] == 0): 
                                flag = 'electron'
                                eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                            elif (inputevents[i].number()["muon"] > 0) and (inputevents[i].number()["electron"] == 0): 
                                flag = 'muon'
                                eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron

                            if (-2.47 < eta_leading_lepton < -1.52) or (-1.37 < eta_leading_lepton < 1.37) or (1.52 < eta_leading_lepton < 2.47): 

                                # how many jets (total) and total hadronic pT
                                flag_bad_jets = 0                            
                                num_jets = 0
                                HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
                                HT = 0 # total HT, defined as the scalar sum of the pT of all reconstructed particles

                                for j in range(inputevents[i].number()["jet"]):
                                    if (inputevents[i]["jet"][j]["PT"] > 20) and (-4.5 < inputevents[i]["jet"][j]["eta"] < 4.5):
                                        num_jets += 1
                                        HT_had += inputevents[i]["jet"][j]["PT"]
                                        HT += inputevents[i]["jet"][j]["PT"]
                                    if (np.abs(inputevents[i]["jet"][j]["eta"]) > 4.5) or (inputevents[i]["jet"][j]["PT"] < 20):
                                        flag_bad_jets = 1

                                # how many photons                         
                                num_photons = 0
                                for j in range(inputevents[i].number()["photon"]):
                                    if inputevents[i]["photon"][j]["PT"] > 10 and -2.37 < inputevents[i]["photon"][j]["eta"] < 2.37:
                                        num_photons = num_photons + 1
                                        HT += inputevents[i]["photon"][j]["PT"]

                                # how many muons                        
                                num_muons = 0
                                for j in range(inputevents[i].number()["muon"]):
                                    if inputevents[i]["muon"][j]["PT"] > 10 and -2.7 < inputevents[i]["muon"][j]["eta"] < 2.7:
                                        num_muons = num_muons + 1
                                        HT += inputevents[i]["muon"][j]["PT"]

                                # how many electrons                        
                                num_electrons = 0
                                for j in range(inputevents[i].number()["electron"]):
                                    if inputevents[i]["electron"][j]["PT"] > 10:
                                        eta = inputevents[i]["electron"][j]["eta"]
                                        if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                            num_electrons = num_electrons + 1
                                            HT += inputevents[i]["electron"][j]["PT"]

                                if flag_bad_jets == 1: counter_bad_jets += 1 
                                if flag_bad_jets == 0:
                                    num_taus = 0
                                    if inputevents[i].number()["tau"] > 0: 
                                        contador_tau = contador_tau + 1
                                        # how many taus                    
                                        for j in range(inputevents[i].number()["tau"]):
                                            if inputevents[i]["tau"][j]["PT"] > 20:
                                                eta = inputevents[i]["tau"][j]["eta"]
                                                if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                                    num_taus = num_taus + 1
                                                    HT += inputevents[i]["tau"][j]["PT"]
                                                    
                                    if inputevents[i]["jet"][0]['btag'] == 1: contador_btag = contador_btag + 1

                                    # photon info:
                                    with open(datFolder + 'photon_bu.dat', 'a') as f:
                                        f.write('%f %f %f %d\n' % (inputevents[i]["photon"][0]["PT"],inputevents[i]["photon"][0]["eta"],inputevents[i]["photon"][0]["phi"], num_photons))

                                    # jet info:
                                    with open(datFolder + 'jet_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["jet"][0]["PT"],inputevents[i]["jet"][0]["eta"],inputevents[i]["jet"][0]["phi"]))

                                    # lepton info
                                    num_leptons = num_electrons + num_muons
                                    if flag == "electron":
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["electron"][0]["PT"],inputevents[i]["electron"][0]["eta"],inputevents[i]["electron"][0]["phi"], num_leptons, num_taus))
                                    else:
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["muon"][0]["PT"],inputevents[i]["muon"][0]["eta"],inputevents[i]["muon"][0]["phi"], num_leptons, num_taus))

                                    # MET (eta=0 by def, but I save phi because I have many jets)
                                    with open(datFolder + 'MET_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))

                                    # hadronic info
                                    with open(datFolder + 'hadinfo_bu.dat', 'a') as f:
                                        f.write('%f %f %d\n' % (HT, HT_had, num_jets))

                                    # total events that passed the cuts
                                    ww_cut_ev += 1

print('Num initial events: ', ww_init_ev)
print('Total pasaron los cortes: ', ww_cut_ev)
print('Hay: ', contador_tau, ' taus')
print('Hay: ', counter_bad_jets, ' eventos con al menos un jet malo')
print('Hay: ', contador_btag, ' con btag en leading jet')

1
Esta run tuvo 827913 eventos en el lhco
('Num initial events: ', 827913)
('Total pasaron los cortes: ', 7330)
('Hay: ', 135, ' taus')
('Hay: ', 67, ' eventos con al menos un jet malo')
('Hay: ', 172, ' con btag en leading jet')


In [22]:
ww_cross = 0.409 #[pb] integrated cross-section without matching
ww_NLO   = 1.40 # k-factor [14 TeV]
ww_init_sim_eve = 1000000 * ww_runs

cross       = ww_cross * 1000 * ww_NLO
aceptancia  = ww_cut_ev / (1. * ww_init_sim_eve)
luminosidad = 100

ww_fidcross = cross * aceptancia
ww_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', ww_fidcross)
print('Events expected: ', ww_expected)

('fiducial cross section: ', 4.197157999999999)
('Events expected: ', 419.7157999999999)


### Z+jets
- [ ] Done

- Cuando analizo ../data/ZJets/lhco/ZJets.lhco se esperan ~4.2k eventos.
- Cuando analizo ../data/ZJets/lhco/Z+jets_1M_1-10/ se esperan ~2.5k eventos
- Cuando analizo .../data/ZJets/lhco/Z+jets_2_1M.lhco (rama leptonica) se esperan ~4.9k eventos
- Cuando analizo .../data/ZJets/lhco/Z+jets_3_1M.lhco (rama neutrinica) se esperan ~0k eventos
- Cuando analizo ../data/Zlep+j/lhco/ se esperan ~2.5k eventos

In [23]:
!ls ../data/ZJets/lhco/

wetransfer_z-jets_1m_2024-05-09_0745	  Z+jets_1M_1-10.tar.gz  ZJets.lhco
wetransfer_z-jets_1m_2024-05-09_0745.zip  Z+jets_2_1M.lhco
Z+jets_1M_1-10				  Z+jets_3_1M.lhco


In [9]:
Folder    = '../data/ZJets/lhco/'
datFolder = '../data/ZJets/'

ZJets_runs  = 1

ZJets_init_ev = 0
ZJets_cut_ev  = 0

ZJets_photon  = []
ZJets_jet     = []
ZJets_lepton  = []
ZJets_MET     = []
ZJets_hadinfo = []


counter_bad_jets = 0
contador_tau = 0
contador_btag = 0
for ii in range(1, 1 + ZJets_runs):
    print(ii)
    if ii not in [0]:
        inputevents = LHCO_reader.Events(f_name = Folder + 'ZJets.lhco')

        ZJets_init_ev += len(inputevents)
        print('Esta run tuvo ' + str(len(inputevents)) + ' eventos en el lhco')
        
        for i in range(len(inputevents)):

            num_jets = 0
    #        num_btag = 0
            HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets


            # at least one: photon, jet and lepton (electron or muon)
            if inputevents[i].number()["photon"] > 0 and inputevents[i].number()["jet"] > 0 and (inputevents[i].number()["electron"] > 0 or inputevents[i].number()["muon"] > 0):
                if inputevents[i]["jet"][0]["PT"] > 100 and -4.5 < inputevents[i]["jet"][0]["eta"] < 4.5:
                    if inputevents[i]["photon"][0]["PT"] > 10 and -2.37 < inputevents[i]["photon"][0]["eta"] < 2.37:
                        if inputevents[i]["MET"][0]["PT"] > 100:
                            # lepton info
                            if (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] > 0):
                                if inputevents[i]["electron"][0]["PT"] > inputevents[i]["muon"][0]["PT"]: 
                                    flag = 'electron'
                                    eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                                else: 
                                    flag = 'muon'
                                    eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron
                            elif (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] == 0): 
                                flag = 'electron'
                                eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                            elif (inputevents[i].number()["muon"] > 0) and (inputevents[i].number()["electron"] == 0): 
                                flag = 'muon'
                                eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron

                            if (-2.47 < eta_leading_lepton < -1.52) or (-1.37 < eta_leading_lepton < 1.37) or (1.52 < eta_leading_lepton < 2.47): 

                                # how many jets (total) and total hadronic pT
                                flag_bad_jets = 0                            
                                num_jets = 0
                                HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
                                HT = 0 # total HT, defined as the scalar sum of the pT of all reconstructed particles

                                for j in range(inputevents[i].number()["jet"]):
                                    if (inputevents[i]["jet"][j]["PT"] > 20) and (-4.5 < inputevents[i]["jet"][j]["eta"] < 4.5):
                                        num_jets += 1
                                        HT_had += inputevents[i]["jet"][j]["PT"]
                                        HT += inputevents[i]["jet"][j]["PT"]
                                    if (np.abs(inputevents[i]["jet"][j]["eta"]) > 4.5) or (inputevents[i]["jet"][j]["PT"] < 20):
                                        flag_bad_jets = 1

                                # how many photons                         
                                num_photons = 0
                                for j in range(inputevents[i].number()["photon"]):
                                    if inputevents[i]["photon"][j]["PT"] > 10 and -2.37 < inputevents[i]["photon"][j]["eta"] < 2.37:
                                        num_photons = num_photons + 1
                                        HT += inputevents[i]["photon"][j]["PT"]

                                # how many muons                        
                                num_muons = 0
                                for j in range(inputevents[i].number()["muon"]):
                                    if inputevents[i]["muon"][j]["PT"] > 10 and -2.7 < inputevents[i]["muon"][j]["eta"] < 2.7:
                                        num_muons = num_muons + 1
                                        HT += inputevents[i]["muon"][j]["PT"]

                                # how many electrons                        
                                num_electrons = 0
                                for j in range(inputevents[i].number()["electron"]):
                                    if inputevents[i]["electron"][j]["PT"] > 10:
                                        eta = inputevents[i]["electron"][j]["eta"]
                                        if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                            num_electrons = num_electrons + 1
                                            HT += inputevents[i]["electron"][j]["PT"]

                                if flag_bad_jets == 1: counter_bad_jets += 1 
                                if flag_bad_jets == 0:
                                    num_taus = 0
                                    if inputevents[i].number()["tau"] > 0: 
                                        contador_tau = contador_tau + 1
                                        # how many taus                    
                                        for j in range(inputevents[i].number()["tau"]):
                                            if inputevents[i]["tau"][j]["PT"] > 20:
                                                eta = inputevents[i]["tau"][j]["eta"]
                                                if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                                    num_taus = num_taus + 1
                                                    HT += inputevents[i]["tau"][j]["PT"]
                                                    
                                    if inputevents[i]["jet"][0]['btag'] == 1: contador_btag = contador_btag + 1

                                    # photon info:
                                    with open(datFolder + 'photon_bu.dat', 'a') as f:
                                        f.write('%f %f %f %d\n' % (inputevents[i]["photon"][0]["PT"],inputevents[i]["photon"][0]["eta"],inputevents[i]["photon"][0]["phi"], num_photons))

                                    # jet info:
                                    with open(datFolder + 'jet_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["jet"][0]["PT"],inputevents[i]["jet"][0]["eta"],inputevents[i]["jet"][0]["phi"]))

                                    # lepton info
                                    num_leptons = num_electrons + num_muons
                                    if flag == "electron":
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["electron"][0]["PT"],inputevents[i]["electron"][0]["eta"],inputevents[i]["electron"][0]["phi"], num_leptons, num_taus))
                                    else:
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["muon"][0]["PT"],inputevents[i]["muon"][0]["eta"],inputevents[i]["muon"][0]["phi"], num_leptons, num_taus))

                                    # MET (eta=0 by def, but I save phi because I have many jets)
                                    with open(datFolder + 'MET_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))

                                    # hadronic info
                                    with open(datFolder + 'hadinfo_bu.dat', 'a') as f:
                                        f.write('%f %f %d\n' % (HT, HT_had, num_jets))

                                    # total events that passed the cuts
                                    ZJets_cut_ev += 1

print('Num initial events: ', ZJets_init_ev)
print('Total pasaron los cortes: ', ZJets_cut_ev)
print('Hay: ', contador_tau, ' taus')
print('Hay: ', counter_bad_jets, ' eventos con al menos un jet malo')
print('Hay: ', contador_btag, ' con btag en leading jet')

1
Esta run tuvo 56701 eventos en el lhco
('Num initial events: ', 56701)
('Total pasaron los cortes: ', 15)
('Hay: ', 0, ' taus')
('Hay: ', 0, ' eventos con al menos un jet malo')
('Hay: ', 1, ' con btag en leading jet')


In [11]:
ZJets_cross = 280.7 #65.85 # 2 leptones ; # 214.6 # 3 neutrinos ; # 280.7 #[pb] Total cross-section without matching
ZJets_NLO   = 1.35 # k-factor [14 TeV] CHECK THIS
ZJets_init_sim_eve = 100000 * ZJets_runs

cross       = ZJets_cross * 1000 * ZJets_NLO
aceptancia  = ZJets_cut_ev / (1. * ZJets_init_sim_eve)
luminosidad = 100

ZJets_fidcross = cross * aceptancia
ZJets_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', ZJets_fidcross)
print('Events expected: ', ZJets_expected)

('fiducial cross section: ', 42.105)
('Events expected: ', 4210.5)


### Z+jets (solamente simulamos la rama leptonica!!!)
- [x] Done
    - se esperan 3360.9922312500003 eventos con MET>100
    - se esperan 350.25615 eventos con MET>200

In [13]:
!ls ../data/Zlep+j/lhco/

Zlep+jets_1M_10.lhco  Zlep+jets_1M_23.lhco  Zlep+jets_1M_36.lhco
Zlep+jets_1M_11.lhco  Zlep+jets_1M_24.lhco  Zlep+jets_1M_37.lhco
Zlep+jets_1M_12.lhco  Zlep+jets_1M_25.lhco  Zlep+jets_1M_38.lhco
Zlep+jets_1M_13.lhco  Zlep+jets_1M_26.lhco  Zlep+jets_1M_39.lhco
Zlep+jets_1M_14.lhco  Zlep+jets_1M_27.lhco  Zlep+jets_1M_3.lhco
Zlep+jets_1M_15.lhco  Zlep+jets_1M_28.lhco  Zlep+jets_1M_40.lhco
Zlep+jets_1M_16.lhco  Zlep+jets_1M_29.lhco  Zlep+jets_1M_4.lhco
Zlep+jets_1M_17.lhco  Zlep+jets_1M_2.lhco   Zlep+jets_1M_5.lhco
Zlep+jets_1M_18.lhco  Zlep+jets_1M_30.lhco  Zlep+jets_1M_6.lhco
Zlep+jets_1M_19.lhco  Zlep+jets_1M_31.lhco  Zlep+jets_1M_7.lhco
Zlep+jets_1M_1.lhco   Zlep+jets_1M_32.lhco  Zlep+jets_1M_8.lhco
Zlep+jets_1M_20.lhco  Zlep+jets_1M_33.lhco  Zlep+jets_1M_9.lhco
Zlep+jets_1M_21.lhco  Zlep+jets_1M_34.lhco
Zlep+jets_1M_22.lhco  Zlep+jets_1M_35.lhco


In [5]:
Folder    = '../data/Zlep+j/lhco/'
datFolder = '../data/Zlep+j/'

ZJets_runs  = 40

ZJets_init_ev = 0
ZJets_cut_ev  = 0

ZJets_photon  = []
ZJets_jet     = []
ZJets_lepton  = []
ZJets_MET     = []
ZJets_hadinfo = []


counter_bad_jets = 0
contador_tau = 0
contador_btag = 0
for ii in range(1, 1 + ZJets_runs):
    print(ii)
    if ii not in [0]:
        inputevents = LHCO_reader.Events(f_name = Folder + 'Zlep+jets_1M_' + str(ii) + '.lhco')

        ZJets_init_ev += len(inputevents)
        print('Esta run tuvo ' + str(len(inputevents)) + ' eventos en el lhco')
        
        for i in range(len(inputevents)):

            num_jets = 0
    #        num_btag = 0
            HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets


            # at least one: photon, jet and lepton (electron or muon)
            if inputevents[i].number()["photon"] > 0 and inputevents[i].number()["jet"] > 0 and (inputevents[i].number()["electron"] > 0 or inputevents[i].number()["muon"] > 0):
                if inputevents[i]["jet"][0]["PT"] > 100 and -4.5 < inputevents[i]["jet"][0]["eta"] < 4.5:
                    if inputevents[i]["photon"][0]["PT"] > 10 and -2.37 < inputevents[i]["photon"][0]["eta"] < 2.37:
                        if inputevents[i]["MET"][0]["PT"] > 100:
                            # lepton info
                            if (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] > 0):
                                if inputevents[i]["electron"][0]["PT"] > inputevents[i]["muon"][0]["PT"]: 
                                    flag = 'electron'
                                    eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                                else: 
                                    flag = 'muon'
                                    eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron
                            elif (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] == 0): 
                                flag = 'electron'
                                eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                            elif (inputevents[i].number()["muon"] > 0) and (inputevents[i].number()["electron"] == 0): 
                                flag = 'muon'
                                eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron

                            if (-2.47 < eta_leading_lepton < -1.52) or (-1.37 < eta_leading_lepton < 1.37) or (1.52 < eta_leading_lepton < 2.47): 

                                # how many jets (total) and total hadronic pT
                                flag_bad_jets = 0                            
                                num_jets = 0
                                HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
                                HT = 0 # total HT, defined as the scalar sum of the pT of all reconstructed particles

                                for j in range(inputevents[i].number()["jet"]):
                                    if (inputevents[i]["jet"][j]["PT"] > 20) and (-4.5 < inputevents[i]["jet"][j]["eta"] < 4.5):
                                        num_jets += 1
                                        HT_had += inputevents[i]["jet"][j]["PT"]
                                        HT += inputevents[i]["jet"][j]["PT"]
                                    if (np.abs(inputevents[i]["jet"][j]["eta"]) > 4.5) or (inputevents[i]["jet"][j]["PT"] < 20):
                                        flag_bad_jets = 1

                                # how many photons                         
                                num_photons = 0
                                for j in range(inputevents[i].number()["photon"]):
                                    if inputevents[i]["photon"][j]["PT"] > 10 and -2.37 < inputevents[i]["photon"][j]["eta"] < 2.37:
                                        num_photons = num_photons + 1
                                        HT += inputevents[i]["photon"][j]["PT"]

                                # how many muons                        
                                num_muons = 0
                                for j in range(inputevents[i].number()["muon"]):
                                    if inputevents[i]["muon"][j]["PT"] > 10 and -2.7 < inputevents[i]["muon"][j]["eta"] < 2.7:
                                        num_muons = num_muons + 1
                                        HT += inputevents[i]["muon"][j]["PT"]

                                # how many electrons                        
                                num_electrons = 0
                                for j in range(inputevents[i].number()["electron"]):
                                    if inputevents[i]["electron"][j]["PT"] > 10:
                                        eta = inputevents[i]["electron"][j]["eta"]
                                        if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                            num_electrons = num_electrons + 1
                                            HT += inputevents[i]["electron"][j]["PT"]

                                if flag_bad_jets == 1: counter_bad_jets += 1 
                                if flag_bad_jets == 0:
                                    num_taus = 0
                                    if inputevents[i].number()["tau"] > 0: 
                                        contador_tau = contador_tau + 1
                                        # how many taus                    
                                        for j in range(inputevents[i].number()["tau"]):
                                            if inputevents[i]["tau"][j]["PT"] > 20:
                                                eta = inputevents[i]["tau"][j]["eta"]
                                                if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                                    num_taus = num_taus + 1
                                                    HT += inputevents[i]["tau"][j]["PT"]
                                                    
                                    if inputevents[i]["jet"][0]['btag'] == 1: contador_btag = contador_btag + 1

                                    # photon info:
                                    with open(datFolder + 'photon_bu.dat', 'a') as f:
                                        f.write('%f %f %f %d\n' % (inputevents[i]["photon"][0]["PT"],inputevents[i]["photon"][0]["eta"],inputevents[i]["photon"][0]["phi"], num_photons))

                                    # jet info:
                                    with open(datFolder + 'jet_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["jet"][0]["PT"],inputevents[i]["jet"][0]["eta"],inputevents[i]["jet"][0]["phi"]))

                                    # lepton info
                                    num_leptons = num_electrons + num_muons
                                    if flag == "electron":
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["electron"][0]["PT"],inputevents[i]["electron"][0]["eta"],inputevents[i]["electron"][0]["phi"], num_leptons, num_taus))
                                    else:
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["muon"][0]["PT"],inputevents[i]["muon"][0]["eta"],inputevents[i]["muon"][0]["phi"], num_leptons, num_taus))

                                    # MET (eta=0 by def, but I save phi because I have many jets)
                                    with open(datFolder + 'MET_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))

                                    # hadronic info
                                    with open(datFolder + 'hadinfo_bu.dat', 'a') as f:
                                        f.write('%f %f %d\n' % (HT, HT_had, num_jets))

                                    # total events that passed the cuts
                                    ZJets_cut_ev += 1

print('Num initial events: ', ZJets_init_ev)
print('Total pasaron los cortes: ', ZJets_cut_ev)
print('Hay: ', contador_tau, ' taus')
print('Hay: ', counter_bad_jets, ' eventos con al menos un jet malo')
print('Hay: ', contador_btag, ' con btag en leading jet')

1
Esta run tuvo 602387 eventos en el lhco
2
Esta run tuvo 604095 eventos en el lhco
3
Esta run tuvo 602646 eventos en el lhco
4
Esta run tuvo 603020 eventos en el lhco
5
Esta run tuvo 603879 eventos en el lhco
6
Esta run tuvo 602505 eventos en el lhco
7
Esta run tuvo 603028 eventos en el lhco
8
Esta run tuvo 602702 eventos en el lhco
9
Esta run tuvo 603199 eventos en el lhco
10
Esta run tuvo 603612 eventos en el lhco
11
Esta run tuvo 602695 eventos en el lhco
12
Esta run tuvo 604220 eventos en el lhco
13
Esta run tuvo 602305 eventos en el lhco
14
Esta run tuvo 602965 eventos en el lhco
15
Esta run tuvo 602935 eventos en el lhco
16
Esta run tuvo 602731 eventos en el lhco
17
Esta run tuvo 602079 eventos en el lhco
18
Esta run tuvo 602165 eventos en el lhco
19
Esta run tuvo 603135 eventos en el lhco
20
Esta run tuvo 601943 eventos en el lhco
21
Esta run tuvo 604222 eventos en el lhco
22
Esta run tuvo 602823 eventos en el lhco
23
Esta run tuvo 602713 eventos en el lhco
24
Esta run tuvo 602

In [6]:
ZJets_cross = 65.85 #65.85 # 2 leptones ; # 214.6 # 3 neutrinos ; # 280.7 #[pb] Total matched cross-section
ZJets_NLO   = 1.35 # k-factor [14 TeV] 
ZJets_init_sim_eve = 1000000 * ZJets_runs

cross       = ZJets_cross * 1000 * ZJets_NLO
aceptancia  = ZJets_cut_ev / (1. * ZJets_init_sim_eve)
luminosidad = 100

ZJets_fidcross = cross * aceptancia
ZJets_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', ZJets_fidcross)
print('Events expected: ', ZJets_expected)

('fiducial cross section: ', 33.609922312500004)
('Events expected: ', 3360.9922312500003)


### [x] Gamma+jets
- [x] Done
    - se esperan 700 eventos con MET>100

In [29]:
!ls ../data/gammaJets/lhco

Gamma+jets_1M.lhco  gammaJets.lhco


In [5]:
Folder    = '../data/gammaJets/lhco/'
datFolder = '../data/gammaJets/'

gammaJets_runs  = 1
gammaJets_init_sim_eve = 1000000 * gammaJets_runs

gammaJets_init_ev = 0
gammaJets_cut_ev  = 0

gammaJets_photon  = []
gammaJets_jet     = []
gammaJets_lepton  = []
gammaJets_MET     = []
gammaJets_hadinfo = []


counter_bad_jets = 0
contador_tau = 0
contador_btag = 0
for ii in range(1, 1 + gammaJets_runs):
    print(ii)
    if ii not in [0]:
        inputevents = LHCO_reader.Events(f_name = Folder + 'Gamma+jets_1M.lhco')

        gammaJets_init_ev += len(inputevents)
        print('Esta run tuvo ' + str(len(inputevents)) + ' eventos en el lhco')
        
        for i in range(len(inputevents)):

            num_jets = 0
    #        num_btag = 0
            HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets


            # at least one: photon, jet and lepton (electron or muon)
            if inputevents[i].number()["photon"] > 0 and inputevents[i].number()["jet"] > 0 and (inputevents[i].number()["electron"] > 0 or inputevents[i].number()["muon"] > 0):
                if inputevents[i]["jet"][0]["PT"] > 100 and -4.5 < inputevents[i]["jet"][0]["eta"] < 4.5:
                    if inputevents[i]["photon"][0]["PT"] > 10 and -2.37 < inputevents[i]["photon"][0]["eta"] < 2.37:
                        if inputevents[i]["MET"][0]["PT"] > 100:
                            # lepton info
                            if (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] > 0):
                                if inputevents[i]["electron"][0]["PT"] > inputevents[i]["muon"][0]["PT"]: 
                                    flag = 'electron'
                                    eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                                else: 
                                    flag = 'muon'
                                    eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron
                            elif (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] == 0): 
                                flag = 'electron'
                                eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                            elif (inputevents[i].number()["muon"] > 0) and (inputevents[i].number()["electron"] == 0): 
                                flag = 'muon'
                                eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron

                            if (-2.47 < eta_leading_lepton < -1.52) or (-1.37 < eta_leading_lepton < 1.37) or (1.52 < eta_leading_lepton < 2.47): 

                                # how many jets (total) and total hadronic pT
                                flag_bad_jets = 0                            
                                num_jets = 0
                                HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
                                HT = 0 # total HT, defined as the scalar sum of the pT of all reconstructed particles

                                for j in range(inputevents[i].number()["jet"]):
                                    if (inputevents[i]["jet"][j]["PT"] > 20) and (-4.5 < inputevents[i]["jet"][j]["eta"] < 4.5):
                                        num_jets += 1
                                        HT_had += inputevents[i]["jet"][j]["PT"]
                                        HT += inputevents[i]["jet"][j]["PT"]
                                    if (np.abs(inputevents[i]["jet"][j]["eta"]) > 4.5) or (inputevents[i]["jet"][j]["PT"] < 20):
                                        flag_bad_jets = 1

                                # how many photons                         
                                num_photons = 0
                                for j in range(inputevents[i].number()["photon"]):
                                    if inputevents[i]["photon"][j]["PT"] > 10 and -2.37 < inputevents[i]["photon"][j]["eta"] < 2.37:
                                        num_photons = num_photons + 1
                                        HT += inputevents[i]["photon"][j]["PT"]

                                # how many muons                        
                                num_muons = 0
                                for j in range(inputevents[i].number()["muon"]):
                                    if inputevents[i]["muon"][j]["PT"] > 10 and -2.7 < inputevents[i]["muon"][j]["eta"] < 2.7:
                                        num_muons = num_muons + 1
                                        HT += inputevents[i]["muon"][j]["PT"]

                                # how many electrons                        
                                num_electrons = 0
                                for j in range(inputevents[i].number()["electron"]):
                                    if inputevents[i]["electron"][j]["PT"] > 10:
                                        eta = inputevents[i]["electron"][j]["eta"]
                                        if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                            num_electrons = num_electrons + 1
                                            HT += inputevents[i]["electron"][j]["PT"]

                                if flag_bad_jets == 1: counter_bad_jets += 1 
                                if flag_bad_jets == 0:
                                    num_taus = 0
                                    if inputevents[i].number()["tau"] > 0: 
                                        contador_tau = contador_tau + 1
                                        # how many taus                    
                                        for j in range(inputevents[i].number()["tau"]):
                                            if inputevents[i]["tau"][j]["PT"] > 20:
                                                eta = inputevents[i]["tau"][j]["eta"]
                                                if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                                    num_taus = num_taus + 1
                                                    HT += inputevents[i]["tau"][j]["PT"]
                                                    
                                    if inputevents[i]["jet"][0]['btag'] == 1: contador_btag = contador_btag + 1

                                    # photon info:
                                    with open(datFolder + 'photon_bu.dat', 'a') as f:
                                        f.write('%f %f %f %d\n' % (inputevents[i]["photon"][0]["PT"],inputevents[i]["photon"][0]["eta"],inputevents[i]["photon"][0]["phi"], num_photons))

                                    # jet info:
                                    with open(datFolder + 'jet_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["jet"][0]["PT"],inputevents[i]["jet"][0]["eta"],inputevents[i]["jet"][0]["phi"]))

                                    # lepton info
                                    num_leptons = num_electrons + num_muons
                                    if flag == "electron":
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["electron"][0]["PT"],inputevents[i]["electron"][0]["eta"],inputevents[i]["electron"][0]["phi"], num_leptons, num_taus))
                                    else:
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["muon"][0]["PT"],inputevents[i]["muon"][0]["eta"],inputevents[i]["muon"][0]["phi"], num_leptons, num_taus))

                                    # MET (eta=0 by def, but I save phi because I have many jets)
                                    with open(datFolder + 'MET_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))

                                    # hadronic info
                                    with open(datFolder + 'hadinfo_bu.dat', 'a') as f:
                                        f.write('%f %f %d\n' % (HT, HT_had, num_jets))

                                    # total events that passed the cuts
                                    gammaJets_cut_ev += 1

print('Num initial events: ', gammaJets_init_ev)
print('Total pasaron los cortes: ', gammaJets_cut_ev)
print('Hay: ', contador_tau, ' taus')
print('Hay: ', counter_bad_jets, ' eventos con al menos un jet malo')
print('Hay: ', contador_btag, ' con btag en leading jet')

1


/home/martinrios/.virtualenvs/pheno2.7/lib/python2.7/site-packages/LHCO_reader/LHCO_reader.py:359: UserWarning: Couldn't read total number of events from LHCO
  warnings.warn("Couldn't read total number of events from LHCO")


Esta run tuvo 476917 eventos en el lhco
('Num initial events: ', 476917)
('Total pasaron los cortes: ', 1)
('Hay: ', 0, ' taus')
('Hay: ', 0, ' eventos con al menos un jet malo')
('Hay: ', 0, ' con btag en leading jet')


In [6]:
gammaJets_cross = 7140 #[pb] Integrated cross-section witouh matching
gammaJets_NLO   = 1. # k-factor [14 TeV] 
gammaJets_init_sim_eve = 1000000 * gammaJets_runs

cross       = gammaJets_cross * 1000 * gammaJets_NLO
aceptancia  = gammaJets_cut_ev / (1. * gammaJets_init_sim_eve)
luminosidad = 100

gammaJets_fidcross = cross * aceptancia
gammaJets_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', gammaJets_fidcross)
print('Events expected: ', gammaJets_expected)

('fiducial cross section: ', 7.14)
('Events expected: ', 714.0)


### [x] Z gamma
- [x] Done.
    - se esperan 85 eventos con MET>100

In [16]:
Folder    = '../data/Zgamma/lhco/'
datFolder = '../data/Zgamma/'

Zgamma_runs  = 1
Zgamma_init_sim_eve = 100000 * Zgamma_runs

Zgamma_init_ev = 0
Zgamma_cut_ev  = 0

Zgamma_photon  = []
Zgamma_jet     = []
Zgamma_lepton  = []
Zgamma_MET     = []
Zgamma_hadinfo = []


counter_bad_jets = 0
contador_tau = 0
contador_btag = 0
for ii in range(1, 1 + Zgamma_runs):
    print(ii)
    if ii not in [0]:
        inputevents = LHCO_reader.Events(f_name = Folder + 'Zgamma.lhco')

        Zgamma_init_ev += len(inputevents)
        print('Esta run tuvo ' + str(len(inputevents)) + ' eventos en el lhco')
        
        for i in range(len(inputevents)):

            num_jets = 0
    #        num_btag = 0
            HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets


            # at least one: photon, jet and lepton (electron or muon)
            if inputevents[i].number()["photon"] > 0 and inputevents[i].number()["jet"] > 0 and (inputevents[i].number()["electron"] > 0 or inputevents[i].number()["muon"] > 0):
                if inputevents[i]["jet"][0]["PT"] > 100 and -4.5 < inputevents[i]["jet"][0]["eta"] < 4.5:
                    if inputevents[i]["photon"][0]["PT"] > 10 and -2.37 < inputevents[i]["photon"][0]["eta"] < 2.37:
                        if inputevents[i]["MET"][0]["PT"] > 100:
                            # lepton info
                            if (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] > 0):
                                if inputevents[i]["electron"][0]["PT"] > inputevents[i]["muon"][0]["PT"]: 
                                    flag = 'electron'
                                    eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                                else: 
                                    flag = 'muon'
                                    eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron
                            elif (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] == 0): 
                                flag = 'electron'
                                eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                            elif (inputevents[i].number()["muon"] > 0) and (inputevents[i].number()["electron"] == 0): 
                                flag = 'muon'
                                eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron

                            if (-2.47 < eta_leading_lepton < -1.52) or (-1.37 < eta_leading_lepton < 1.37) or (1.52 < eta_leading_lepton < 2.47): 

                                # how many jets (total) and total hadronic pT
                                flag_bad_jets = 0                            
                                num_jets = 0
                                HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
                                HT = 0 # total HT, defined as the scalar sum of the pT of all reconstructed particles

                                for j in range(inputevents[i].number()["jet"]):
                                    if (inputevents[i]["jet"][j]["PT"] > 20) and (-4.5 < inputevents[i]["jet"][j]["eta"] < 4.5):
                                        num_jets += 1
                                        HT_had += inputevents[i]["jet"][j]["PT"]
                                        HT += inputevents[i]["jet"][j]["PT"]
                                    if (np.abs(inputevents[i]["jet"][j]["eta"]) > 4.5) or (inputevents[i]["jet"][j]["PT"] < 20):
                                        flag_bad_jets = 1

                                # how many photons                         
                                num_photons = 0
                                for j in range(inputevents[i].number()["photon"]):
                                    if inputevents[i]["photon"][j]["PT"] > 10 and -2.37 < inputevents[i]["photon"][j]["eta"] < 2.37:
                                        num_photons = num_photons + 1
                                        HT += inputevents[i]["photon"][j]["PT"]

                                # how many muons                        
                                num_muons = 0
                                for j in range(inputevents[i].number()["muon"]):
                                    if inputevents[i]["muon"][j]["PT"] > 10 and -2.7 < inputevents[i]["muon"][j]["eta"] < 2.7:
                                        num_muons = num_muons + 1
                                        HT += inputevents[i]["muon"][j]["PT"]

                                # how many electrons                        
                                num_electrons = 0
                                for j in range(inputevents[i].number()["electron"]):
                                    if inputevents[i]["electron"][j]["PT"] > 10:
                                        eta = inputevents[i]["electron"][j]["eta"]
                                        if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                            num_electrons = num_electrons + 1
                                            HT += inputevents[i]["electron"][j]["PT"]

                                if flag_bad_jets == 1: counter_bad_jets += 1 
                                if flag_bad_jets == 0:
                                    num_taus = 0
                                    if inputevents[i].number()["tau"] > 0: 
                                        contador_tau = contador_tau + 1
                                        # how many taus                    
                                        for j in range(inputevents[i].number()["tau"]):
                                            if inputevents[i]["tau"][j]["PT"] > 20:
                                                eta = inputevents[i]["tau"][j]["eta"]
                                                if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                                    num_taus = num_taus + 1
                                                    HT += inputevents[i]["tau"][j]["PT"]
                                                    
                                    if inputevents[i]["jet"][0]['btag'] == 1: contador_btag = contador_btag + 1

                                    # photon info:
                                    with open(datFolder + 'photon_bu.dat', 'a') as f:
                                        f.write('%f %f %f %d\n' % (inputevents[i]["photon"][0]["PT"],inputevents[i]["photon"][0]["eta"],inputevents[i]["photon"][0]["phi"], num_photons))

                                    # jet info:
                                    with open(datFolder + 'jet_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["jet"][0]["PT"],inputevents[i]["jet"][0]["eta"],inputevents[i]["jet"][0]["phi"]))

                                    # lepton info
                                    num_leptons = num_electrons + num_muons
                                    if flag == "electron":
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["electron"][0]["PT"],inputevents[i]["electron"][0]["eta"],inputevents[i]["electron"][0]["phi"], num_leptons, num_taus))
                                    else:
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["muon"][0]["PT"],inputevents[i]["muon"][0]["eta"],inputevents[i]["muon"][0]["phi"], num_leptons, num_taus))

                                    # MET (eta=0 by def, but I save phi because I have many jets)
                                    with open(datFolder + 'MET_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))

                                    # hadronic info
                                    with open(datFolder + 'hadinfo_bu.dat', 'a') as f:
                                        f.write('%f %f %d\n' % (HT, HT_had, num_jets))

                                    # total events that passed the cuts
                                    Zgamma_cut_ev += 1

print('Num initial events: ', Zgamma_init_ev)
print('Total pasaron los cortes: ', Zgamma_cut_ev)
print('Hay: ', contador_tau, ' taus')
print('Hay: ', counter_bad_jets, ' eventos con al menos un jet malo')
print('Hay: ', contador_btag, ' con btag en leading jet')

1
Esta run tuvo 85412 eventos en el lhco
('Num initial events: ', 85412)
('Total pasaron los cortes: ', 20)
('Hay: ', 0, ' taus')
('Hay: ', 0, ' eventos con al menos un jet malo')
('Hay: ', 1, ' con btag en leading jet')


In [17]:
Zgamma_cross = 4.26 #[pb] Integrated cross-section without matching
Zgamma_NLO   = 1. # k-factor [14 TeV] CHECK THIS
Zgamma_init_sim_eve = 100000 * Zgamma_runs

cross       = Zgamma_cross * 1000 * Zgamma_NLO
aceptancia  = Zgamma_cut_ev / (1. * Zgamma_init_sim_eve)
luminosidad = 100

Zgamma_fidcross = cross * aceptancia
Zgamma_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', Zgamma_fidcross)
print('Events expected: ', Zgamma_expected)

('fiducial cross section: ', 0.8520000000000001)
('Events expected: ', 85.2)


### tt > jets
- [x] Done. 
  - Se esperan 11132,94 con MET > 100
  - Se esperan 1985.591 con MET > 200
  
New runs
- [x] Done. 
  - Se esperan 18051.07217301 con MET > 100
  - Se esperan 2238.59925988 con MET > 200

In [2]:
!ls ../data/ttjets/lhco

run_100.lhco  run_140.lhco  run_180.lhco  run_21.lhco	run_60.lhco
run_101.lhco  run_141.lhco  run_181.lhco  run_220.lhco	run_61.lhco
run_102.lhco  run_142.lhco  run_182.lhco  run_22.lhco	run_62.lhco
run_103.lhco  run_143.lhco  run_183.lhco  run_23.lhco	run_63.lhco
run_104.lhco  run_144.lhco  run_184.lhco  run_24.lhco	run_64.lhco
run_105.lhco  run_145.lhco  run_185.lhco  run_25.lhco	run_65.lhco
run_106.lhco  run_146.lhco  run_186.lhco  run_26.lhco	run_66.lhco
run_107.lhco  run_147.lhco  run_187.lhco  run_27.lhco	run_67.lhco
run_108.lhco  run_148.lhco  run_188.lhco  run_28.lhco	run_68.lhco
run_109.lhco  run_149.lhco  run_189.lhco  run_29.lhco	run_69.lhco
run_10.lhco   run_14.lhco   run_18.lhco   run_2.lhco	run_6.lhco
run_110.lhco  run_150.lhco  run_190.lhco  run_30.lhco	run_70.lhco
run_111.lhco  run_151.lhco  run_191.lhco  run_31.lhco	run_71.lhco
run_112.lhco  run_152.lhco  run_192.lhco  run_32.lhco	run_72.lhco
run_113.lhco  run_153.lhco  run_193.lhco  run_33.lhco	run_73.lhco
run_114.lhc

In [5]:
Folder    = '../data/ttjets/lhco/'
datFolder = '../data/ttjets/'

ttjets_runs  = 220

ttjets_init_ev = 0
ttjets_cut_ev  = 0

ttjets_photon  = []
ttjets_jet     = []
ttjets_lepton  = []
ttjets_MET     = []
ttjets_hadinfo = []


counter_bad_jets = 0
contador_tau = 0
contador_btag = 0
for ii in range(1, 1 + ttjets_runs):
    print(ii)
    if ii not in [0]:
        inputevents = LHCO_reader.Events(f_name = Folder + 'run_' + str(ii) + '.lhco')

        ttjets_init_ev += len(inputevents)
        print('Esta run tuvo ' + str(len(inputevents)) + ' eventos en el lhco')
        
        for i in range(len(inputevents)):

            num_jets = 0
    #        num_btag = 0
            HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets


            # at least one: photon, jet and lepton (electron or muon)
            if inputevents[i].number()["photon"] > 0 and inputevents[i].number()["jet"] > 0 and (inputevents[i].number()["electron"] > 0 or inputevents[i].number()["muon"] > 0):
                if inputevents[i]["jet"][0]["PT"] > 100 and -4.5 < inputevents[i]["jet"][0]["eta"] < 4.5:
                    if inputevents[i]["photon"][0]["PT"] > 10 and -2.37 < inputevents[i]["photon"][0]["eta"] < 2.37:
                        if inputevents[i]["MET"][0]["PT"] > 100:
                            # lepton info
                            if (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] > 0):
                                if inputevents[i]["electron"][0]["PT"] > inputevents[i]["muon"][0]["PT"]: 
                                    flag = 'electron'
                                    eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                                else: 
                                    flag = 'muon'
                                    eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron
                            elif (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] == 0): 
                                flag = 'electron'
                                eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                            elif (inputevents[i].number()["muon"] > 0) and (inputevents[i].number()["electron"] == 0): 
                                flag = 'muon'
                                eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron

                            if (-2.47 < eta_leading_lepton < -1.52) or (-1.37 < eta_leading_lepton < 1.37) or (1.52 < eta_leading_lepton < 2.47): 

                                # how many jets (total) and total hadronic pT
                                flag_bad_jets = 0                            
                                num_jets = 0
                                HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
                                HT = 0 # total HT, defined as the scalar sum of the pT of all reconstructed particles

                                for j in range(inputevents[i].number()["jet"]):
                                    if (inputevents[i]["jet"][j]["PT"] > 20) and (-4.5 < inputevents[i]["jet"][j]["eta"] < 4.5):
                                        num_jets += 1
                                        HT_had += inputevents[i]["jet"][j]["PT"]
                                        HT += inputevents[i]["jet"][j]["PT"]
                                    if (np.abs(inputevents[i]["jet"][j]["eta"]) > 4.5) or (inputevents[i]["jet"][j]["PT"] < 20):
                                        flag_bad_jets = 1

                                # how many photons                         
                                num_photons = 0
                                for j in range(inputevents[i].number()["photon"]):
                                    if inputevents[i]["photon"][j]["PT"] > 10 and -2.37 < inputevents[i]["photon"][j]["eta"] < 2.37:
                                        num_photons = num_photons + 1
                                        HT += inputevents[i]["photon"][j]["PT"]

                                # how many muons                        
                                num_muons = 0
                                for j in range(inputevents[i].number()["muon"]):
                                    if inputevents[i]["muon"][j]["PT"] > 10 and -2.7 < inputevents[i]["muon"][j]["eta"] < 2.7:
                                        num_muons = num_muons + 1
                                        HT += inputevents[i]["muon"][j]["PT"]

                                # how many electrons                        
                                num_electrons = 0
                                for j in range(inputevents[i].number()["electron"]):
                                    if inputevents[i]["electron"][j]["PT"] > 10:
                                        eta = inputevents[i]["electron"][j]["eta"]
                                        if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                            num_electrons = num_electrons + 1
                                            HT += inputevents[i]["electron"][j]["PT"]

                                if flag_bad_jets == 1: counter_bad_jets += 1 
                                if flag_bad_jets == 0:
                                    num_taus = 0
                                    if inputevents[i].number()["tau"] > 0: 
                                        contador_tau = contador_tau + 1
                                        # how many taus                    
                                        for j in range(inputevents[i].number()["tau"]):
                                            if inputevents[i]["tau"][j]["PT"] > 20:
                                                eta = inputevents[i]["tau"][j]["eta"]
                                                if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                                    num_taus = num_taus + 1
                                                    HT += inputevents[i]["tau"][j]["PT"]
                                                    
                                    if inputevents[i]["jet"][0]['btag'] == 1: contador_btag = contador_btag + 1

                                    # photon info:
                                    with open(datFolder + 'photon_4.dat', 'a') as f:
                                        f.write('%f %f %f %d\n' % (inputevents[i]["photon"][0]["PT"],inputevents[i]["photon"][0]["eta"],inputevents[i]["photon"][0]["phi"], num_photons))

                                    # jet info:
                                    with open(datFolder + 'jet_4.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["jet"][0]["PT"],inputevents[i]["jet"][0]["eta"],inputevents[i]["jet"][0]["phi"]))

                                    # lepton info
                                    num_leptons = num_electrons + num_muons
                                    if flag == "electron":
                                        with open(datFolder + 'lepton_4.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["electron"][0]["PT"],inputevents[i]["electron"][0]["eta"],inputevents[i]["electron"][0]["phi"], num_leptons, num_taus))
                                    else:
                                        with open(datFolder + 'lepton_4.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["muon"][0]["PT"],inputevents[i]["muon"][0]["eta"],inputevents[i]["muon"][0]["phi"], num_leptons, num_taus))

                                    # MET (eta=0 by def, but I save phi because I have many jets)
                                    with open(datFolder + 'MET_4.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))

                                    # hadronic info
                                    with open(datFolder + 'hadinfo_4.dat', 'a') as f:
                                        f.write('%f %f %d\n' % (HT, HT_had, num_jets))

                                    # total events that passed the cuts
                                    ttjets_cut_ev += 1

print('Num initial events: ', ttjets_init_ev)
print('Total pasaron los cortes: ', ttjets_cut_ev)
print('Hay: ', contador_tau, ' taus')
print('Hay: ', counter_bad_jets, ' eventos con al menos un jet malo')
print('Hay: ', contador_btag, ' con btag en leading jet')

1
Esta run tuvo 29403 eventos en el lhco
2
Esta run tuvo 29469 eventos en el lhco
3
Esta run tuvo 29560 eventos en el lhco
4
Esta run tuvo 29421 eventos en el lhco
5
Esta run tuvo 29396 eventos en el lhco
6
Esta run tuvo 29250 eventos en el lhco
7
Esta run tuvo 29396 eventos en el lhco
8
Esta run tuvo 29439 eventos en el lhco
9
Esta run tuvo 29009 eventos en el lhco
10
Esta run tuvo 29425 eventos en el lhco
11
Esta run tuvo 29130 eventos en el lhco
12
Esta run tuvo 29505 eventos en el lhco
13
Esta run tuvo 29377 eventos en el lhco
14
Esta run tuvo 29495 eventos en el lhco
15
Esta run tuvo 29538 eventos en el lhco
16
Esta run tuvo 29682 eventos en el lhco
17
Esta run tuvo 29587 eventos en el lhco
18
Esta run tuvo 29635 eventos en el lhco
19
Esta run tuvo 29639 eventos en el lhco
20
Esta run tuvo 29500 eventos en el lhco
21
Esta run tuvo 29726 eventos en el lhco
22
Esta run tuvo 29233 eventos en el lhco
23
Esta run tuvo 29494 eventos en el lhco
24
Esta run tuvo 29437 eventos en el lhco
2

In [6]:
ttjets_cross = 29.04955004446046 #27.664 #[pb] Total integrated cross section witout matching
ttjets_NLO   = 1.47 # k-factor [14 TeV] 
ttjets_init_sim_eve = 100000 * ttjets_runs

cross       = ttjets_cross * 1000 * ttjets_NLO
aceptancia  = ttjets_cut_ev / (1. * ttjets_init_sim_eve)
luminosidad = 100

ttjets_fidcross = cross * aceptancia
ttjets_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', ttjets_fidcross)
print('Events expected: ', ttjets_expected)

('fiducial cross section: ', 180.51072173011335)
('Events expected: ', 18051.072173011336)


### pp > tt $\gamma$
- [x] Done. 
    - Se esperan ~2.9k con MET > 100
    - Se esperan 709.602048 con MET > 200
    
New runs
- [x] Done. 
    - Se esperan 2498.413858375 con MET > 100
    - Se esperan 523.82031475 con MET > 200

In [2]:
!ls ../data/ttGamma/lhco

run100k_10.lhco  run100k_33.lhco  run100k_56.lhco  run100k_79.lhco
run100k_11.lhco  run100k_34.lhco  run100k_57.lhco  run100k_7.lhco
run100k_12.lhco  run100k_35.lhco  run100k_58.lhco  run100k_80.lhco
run100k_13.lhco  run100k_36.lhco  run100k_59.lhco  run100k_81.lhco
run100k_14.lhco  run100k_37.lhco  run100k_5.lhco   run100k_82.lhco
run100k_15.lhco  run100k_38.lhco  run100k_60.lhco  run100k_83.lhco
run100k_16.lhco  run100k_39.lhco  run100k_61.lhco  run100k_84.lhco
run100k_17.lhco  run100k_3.lhco   run100k_62.lhco  run100k_8.lhco
run100k_18.lhco  run100k_40.lhco  run100k_63.lhco  run100k_9.lhco
run100k_19.lhco  run100k_41.lhco  run100k_64.lhco  tt+gamma_1M_10.lhco
run100k_1.lhco	 run100k_42.lhco  run100k_65.lhco  tt+gamma_1M_11.lhco
run100k_20.lhco  run100k_43.lhco  run100k_66.lhco  tt+gamma_1M_12.lhco
run100k_21.lhco  run100k_44.lhco  run100k_67.lhco  tt+gamma_1M_13.lhco
run100k_22.lhco  run100k_45.lhco  run100k_68.lhco  tt+gamma_1M_1.lhco
run100k_23.lhco  run100k_46.lhco  run100k_69.lh

In [17]:
Folder    = '../data/ttGamma/lhco/'
datFolder = '../data/ttGamma/'

ttgamma_100k_runs = 84
ttgamma_1M_runs   = 13

ttgamma_init_ev = 0
ttgamma_cut_ev  = 0

ttgamma_photon  = []
ttgamma_jet     = []
ttgamma_lepton  = []
ttgamma_MET     = []
ttgamma_hadinfo = []


counter_bad_jets = 0
contador_tau = 0
contador_btag = 0

contador_100k_runs = 0
contador_1M_runs = 0
for ii in range(1, 1 + (ttgamma_100k_runs + ttgamma_1M_runs)):
    print(ii)
    if ii not in [15, 39, 45, 53, 63, 66]:
        if ii <= ttgamma_100k_runs:
            contador_100k_runs = contador_100k_runs + 1
            f_name = Folder + "run100k_" + str(ii) + ".lhco"
        else:
            contador_1M_runs = contador_1M_runs + 1
            f_name = Folder + "tt+gamma_1M_" + str(ii - ttgamma_100k_runs) + ".lhco"

        inputevents = LHCO_reader.Events(f_name = f_name)
        ttgamma_init_ev += len(inputevents)
        print(f_name + ' tuvo ' + str(len(inputevents)) + ' eventos en el lhco')

        for i in range(len(inputevents)):

            num_jets = 0
    #        num_btag = 0
            HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets


            # at least one: photon, jet and lepton (electron or muon)
            if inputevents[i].number()["photon"] > 0 and inputevents[i].number()["jet"] > 0 and (inputevents[i].number()["electron"] > 0 or inputevents[i].number()["muon"] > 0):
                if inputevents[i]["jet"][0]["PT"] > 100 and -4.5 < inputevents[i]["jet"][0]["eta"] < 4.5:
                    if inputevents[i]["photon"][0]["PT"] > 10 and -2.37 < inputevents[i]["photon"][0]["eta"] < 2.37:
                        if inputevents[i]["MET"][0]["PT"] > 100:
                            # lepton info
                            if (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] > 0):
                                if inputevents[i]["electron"][0]["PT"] > inputevents[i]["muon"][0]["PT"]: 
                                    flag = 'electron'
                                    eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                                else: 
                                    flag = 'muon'
                                    eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron
                            elif (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] == 0): 
                                flag = 'electron'
                                eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                            elif (inputevents[i].number()["muon"] > 0) and (inputevents[i].number()["electron"] == 0): 
                                flag = 'muon'
                                eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron

                            if (-2.47 < eta_leading_lepton < -1.52) or (-1.37 < eta_leading_lepton < 1.37) or (1.52 < eta_leading_lepton < 2.47): 

                                # how many jets (total) and total hadronic pT
                                flag_bad_jets = 0                            
                                num_jets = 0
                                HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
                                HT = 0 # total HT, defined as the scalar sum of the pT of all reconstructed particles

                                for j in range(inputevents[i].number()["jet"]):
                                    if (inputevents[i]["jet"][j]["PT"] > 20) and (-4.5 < inputevents[i]["jet"][j]["eta"] < 4.5):
                                        num_jets += 1
                                        HT_had += inputevents[i]["jet"][j]["PT"]
                                        HT += inputevents[i]["jet"][j]["PT"]
                                    if (np.abs(inputevents[i]["jet"][j]["eta"]) > 4.5) or (inputevents[i]["jet"][j]["PT"] < 20):
                                        flag_bad_jets = 1

                                # how many photons                         
                                num_photons = 0
                                for j in range(inputevents[i].number()["photon"]):
                                    if inputevents[i]["photon"][j]["PT"] > 10 and -2.37 < inputevents[i]["photon"][j]["eta"] < 2.37:
                                        num_photons = num_photons + 1
                                        HT += inputevents[i]["photon"][j]["PT"]

                                # how many muons                        
                                num_muons = 0
                                for j in range(inputevents[i].number()["muon"]):
                                    if inputevents[i]["muon"][j]["PT"] > 10 and -2.7 < inputevents[i]["muon"][j]["eta"] < 2.7:
                                        num_muons = num_muons + 1
                                        HT += inputevents[i]["muon"][j]["PT"]

                                # how many electrons                        
                                num_electrons = 0
                                for j in range(inputevents[i].number()["electron"]):
                                    if inputevents[i]["electron"][j]["PT"] > 10:
                                        eta = inputevents[i]["electron"][j]["eta"]
                                        if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                            num_electrons = num_electrons + 1
                                            HT += inputevents[i]["electron"][j]["PT"]

                                if flag_bad_jets == 1: counter_bad_jets += 1 
                                if flag_bad_jets == 0:
                                    num_taus = 0
                                    if inputevents[i].number()["tau"] > 0: 
                                        contador_tau = contador_tau + 1
                                        # how many taus                    
                                        for j in range(inputevents[i].number()["tau"]):
                                            if inputevents[i]["tau"][j]["PT"] > 20:
                                                eta = inputevents[i]["tau"][j]["eta"]
                                                if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                                    num_taus = num_taus + 1
                                                    HT += inputevents[i]["tau"][j]["PT"]

                                    if inputevents[i]["jet"][0]['btag'] == 1: contador_btag = contador_btag + 1

                                    # photon info:
                                    with open(datFolder + 'photon_4.dat', 'a') as f:
                                        f.write('%f %f %f %d\n' % (inputevents[i]["photon"][0]["PT"],inputevents[i]["photon"][0]["eta"],inputevents[i]["photon"][0]["phi"], num_photons))

                                    # jet info:
                                    with open(datFolder + 'jet_4.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["jet"][0]["PT"],inputevents[i]["jet"][0]["eta"],inputevents[i]["jet"][0]["phi"]))

                                    # lepton info
                                    num_leptons = num_electrons + num_muons
                                    if flag == "electron":
                                        with open(datFolder + 'lepton_4.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["electron"][0]["PT"],inputevents[i]["electron"][0]["eta"],inputevents[i]["electron"][0]["phi"], num_leptons, num_taus))
                                    else:
                                        with open(datFolder + 'lepton_4.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["muon"][0]["PT"],inputevents[i]["muon"][0]["eta"],inputevents[i]["muon"][0]["phi"], num_leptons, num_taus))

                                    # MET (eta=0 by def, but I save phi because I have many jets)
                                    with open(datFolder + 'MET_4.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))

                                    # hadronic info
                                    with open(datFolder + 'hadinfo_4.dat', 'a') as f:
                                        f.write('%f %f %d\n' % (HT, HT_had, num_jets))

                                    # total events that passed the cuts
                                    ttgamma_cut_ev += 1

print('Num initial events: ', ttgamma_init_ev)
print('Total pasaron los cortes: ', ttgamma_cut_ev)
print('Hay: ', contador_tau, ' taus')
print('Hay: ', counter_bad_jets, ' eventos con al menos un jet malo')
print('Hay: ', contador_btag, ' con btag en leading jet')

1
../data/ttGamma/lhco/run100k_1.lhco tuvo 49981 eventos en el lhco
2
../data/ttGamma/lhco/run100k_2.lhco tuvo 50077 eventos en el lhco
3
../data/ttGamma/lhco/run100k_3.lhco tuvo 50239 eventos en el lhco
4
../data/ttGamma/lhco/run100k_4.lhco tuvo 49876 eventos en el lhco
5
../data/ttGamma/lhco/run100k_5.lhco tuvo 49843 eventos en el lhco
6
../data/ttGamma/lhco/run100k_6.lhco tuvo 49638 eventos en el lhco
7
../data/ttGamma/lhco/run100k_7.lhco tuvo 49887 eventos en el lhco
8
../data/ttGamma/lhco/run100k_8.lhco tuvo 49864 eventos en el lhco
9
../data/ttGamma/lhco/run100k_9.lhco tuvo 50215 eventos en el lhco
10
../data/ttGamma/lhco/run100k_10.lhco tuvo 49951 eventos en el lhco
11
../data/ttGamma/lhco/run100k_11.lhco tuvo 49843 eventos en el lhco
12
../data/ttGamma/lhco/run100k_12.lhco tuvo 50064 eventos en el lhco
13
../data/ttGamma/lhco/run100k_13.lhco tuvo 49742 eventos en el lhco
14
../data/ttGamma/lhco/run100k_14.lhco tuvo 48429 eventos en el lhco
15
16
../data/ttGamma/lhco/run100k_16.

In [18]:
ttgamma_cross = 0.13 #[pb]# Integrated cross section without matching
ttgamma_NLO   = 1.46 # k-factor [14 TeV] 
ttgamma_init_sim_eve = (100000 * contador_100k_runs) + (1000000 * contador_1M_runs)

In [19]:
cross       = ttgamma_cross * 1000 * ttgamma_NLO
aceptancia  = ttgamma_cut_ev / (1. * ttgamma_init_sim_eve)
luminosidad = 100

ttgamma_fidcross = cross * aceptancia
ttgamma_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', ttgamma_fidcross)
print('Events expected: ', ttgamma_expected)

('fiducial cross section: ', 24.98413858375)
('Events expected: ', 2498.413858375)


### p p > W $\gamma$ > $\nu$ + l + $\gamma$
- [x] Done

In [14]:
Folder    = '/home/martinrios/Downloads/MG5aMC_LTS_2.9.9/MG5_aMC_v2_9_9/pptoWgamma/'
datFolder = '../data/Wgamma/'

Wgamma_cross = 0.944 #[pb]  # Integrated cross section without matching
Wgamma_runs  = 137
Wgamma_NLO   = 2.37 # k-factor [14 TeV]
Wgamma_init_sim_eve = 100000 * Wgamma_runs

Wgamma_init_ev = 0
Wgamma_cut_ev  = 0

Wgamma_photon  = []
Wgamma_jet     = []
Wgamma_lepton  = []
Wgamma_MET     = []
Wgamma_hadinfo = []


counter_bad_jets = 0
contador_tau = 0
contador_btag = 0
for ii in range(1, 1 + Wgamma_runs):
    print(ii)
    if ii not in [0]:
        inputevents = LHCO_reader.Events(f_name = Folder + "run100k_" + str(ii) + ".lhco")

        Wgamma_init_ev += len(inputevents)
        print('Esta run tuvo ' + str(len(inputevents)) + ' eventos en el lhco')
        
        for i in range(len(inputevents)):

            num_jets = 0
    #        num_btag = 0
            HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets


            # at least one: photon, jet and lepton (electron or muon)
            if inputevents[i].number()["photon"] > 0 and inputevents[i].number()["jet"] > 0 and (inputevents[i].number()["electron"] > 0 or inputevents[i].number()["muon"] > 0):
                if inputevents[i]["jet"][0]["PT"] > 100 and -4.5 < inputevents[i]["jet"][0]["eta"] < 4.5:
                    if inputevents[i]["photon"][0]["PT"] > 10 and -2.37 < inputevents[i]["photon"][0]["eta"] < 2.37:
                        if inputevents[i]["MET"][0]["PT"] > 200:
                            # lepton info
                            if (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] > 0):
                                if inputevents[i]["electron"][0]["PT"] > inputevents[i]["muon"][0]["PT"]: 
                                    flag = 'electron'
                                    eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                                else: 
                                    flag = 'muon'
                                    eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron
                            elif (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] == 0): 
                                flag = 'electron'
                                eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                            elif (inputevents[i].number()["muon"] > 0) and (inputevents[i].number()["electron"] == 0): 
                                flag = 'muon'
                                eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron

                            if (-2.47 < eta_leading_lepton < -1.52) or (-1.37 < eta_leading_lepton < 1.37) or (1.52 < eta_leading_lepton < 2.47): 

                                # how many jets (total) and total hadronic pT
                                flag_bad_jets = 0                            
                                num_jets = 0
                                HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
                                HT = 0 # total HT, defined as the scalar sum of the pT of all reconstructed particles

                                for j in range(inputevents[i].number()["jet"]):
                                    if (inputevents[i]["jet"][j]["PT"] > 20) and (-4.5 < inputevents[i]["jet"][j]["eta"] < 4.5):
                                        num_jets += 1
                                        HT_had += inputevents[i]["jet"][j]["PT"]
                                        HT += inputevents[i]["jet"][j]["PT"]
                                    if (np.abs(inputevents[i]["jet"][j]["eta"]) > 4.5) or (inputevents[i]["jet"][j]["PT"] < 20):
                                        flag_bad_jets = 1

                                # how many photons                         
                                num_photons = 0
                                for j in range(inputevents[i].number()["photon"]):
                                    if inputevents[i]["photon"][j]["PT"] > 10 and -2.37 < inputevents[i]["photon"][j]["eta"] < 2.37:
                                        num_photons = num_photons + 1
                                        HT += inputevents[i]["photon"][j]["PT"]

                                # how many muons                        
                                num_muons = 0
                                for j in range(inputevents[i].number()["muon"]):
                                    if inputevents[i]["muon"][j]["PT"] > 10 and -2.7 < inputevents[i]["muon"][j]["eta"] < 2.7:
                                        num_muons = num_muons + 1
                                        HT += inputevents[i]["muon"][j]["PT"]

                                # how many electrons                        
                                num_electrons = 0
                                for j in range(inputevents[i].number()["electron"]):
                                    if inputevents[i]["electron"][j]["PT"] > 10:
                                        eta = inputevents[i]["electron"][j]["eta"]
                                        if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                            num_electrons = num_electrons + 1
                                            HT += inputevents[i]["electron"][j]["PT"]

                                if flag_bad_jets == 1: counter_bad_jets += 1 
                                if flag_bad_jets == 0:
                                    num_taus = 0
                                    if inputevents[i].number()["tau"] > 0: 
                                        contador_tau = contador_tau + 1
                                        # how many taus                    
                                        for j in range(inputevents[i].number()["tau"]):
                                            if inputevents[i]["tau"][j]["PT"] > 20:
                                                eta = inputevents[i]["tau"][j]["eta"]
                                                if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                                    num_taus = num_taus + 1
                                                    HT += inputevents[i]["tau"][j]["PT"]

                                                    
                                    if inputevents[i]["jet"][0]['btag'] == 1: contador_btag = contador_btag + 1
                                    
                                    # photon info:
                                    with open(datFolder + 'photon_bu.dat', 'a') as f:
                                        f.write('%f %f %f %d\n' % (inputevents[i]["photon"][0]["PT"],inputevents[i]["photon"][0]["eta"],inputevents[i]["photon"][0]["phi"], num_photons))

                                    # jet info:
                                    with open(datFolder + 'jet_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["jet"][0]["PT"],inputevents[i]["jet"][0]["eta"],inputevents[i]["jet"][0]["phi"]))

                                    # lepton info
                                    num_leptons = num_electrons + num_muons
                                    if flag == "electron":
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["electron"][0]["PT"],inputevents[i]["electron"][0]["eta"],inputevents[i]["electron"][0]["phi"], num_leptons, num_taus))
                                    else:
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["muon"][0]["PT"],inputevents[i]["muon"][0]["eta"],inputevents[i]["muon"][0]["phi"], num_leptons, num_taus))

                                    # MET (eta=0 by def, but I save phi because I have many jets)
                                    with open(datFolder + 'MET_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))

                                    # hadronic info
                                    with open(datFolder + 'hadinfo_bu.dat', 'a') as f:
                                        f.write('%f %f %d\n' % (HT, HT_had, num_jets))

                                    # total events that passed the cuts
                                    Wgamma_cut_ev += 1

print('Num initial events: ', Wgamma_init_ev)
print('Total pasaron los cortes: ', Wgamma_cut_ev)
print('Hay: ', contador_tau, ' taus')
print('Hay: ', counter_bad_jets, ' eventos con al menos un jet malo')
print('Hay: ', contador_btag, ' con btag en leading jet')

1
Esta run tuvo 96394 eventos en el lhco
2
Esta run tuvo 96333 eventos en el lhco
3
Esta run tuvo 96331 eventos en el lhco
4
Esta run tuvo 96491 eventos en el lhco
5
Esta run tuvo 96378 eventos en el lhco
6
Esta run tuvo 96527 eventos en el lhco
7
Esta run tuvo 96453 eventos en el lhco
8
Esta run tuvo 96428 eventos en el lhco
9
Esta run tuvo 96467 eventos en el lhco
10
Esta run tuvo 96433 eventos en el lhco
11
Esta run tuvo 96472 eventos en el lhco
12
Esta run tuvo 96347 eventos en el lhco
13
Esta run tuvo 96530 eventos en el lhco
14
Esta run tuvo 96510 eventos en el lhco
15
Esta run tuvo 96353 eventos en el lhco
16
Esta run tuvo 96462 eventos en el lhco
17
Esta run tuvo 96376 eventos en el lhco
18
Esta run tuvo 96341 eventos en el lhco
19
Esta run tuvo 96392 eventos en el lhco
20
Esta run tuvo 96418 eventos en el lhco
21
Esta run tuvo 96464 eventos en el lhco
22
Esta run tuvo 96371 eventos en el lhco
23
Esta run tuvo 96395 eventos en el lhco
24
Esta run tuvo 96402 eventos en el lhco
2

In [16]:
14215./673720

0.02109926972629579

In [23]:
Wgamma_cut_ev = 673720#3579946
Wgamma_init_ev = 13210085

In [24]:
Wgamma_cross = 0.944 #[pb]  # Integrated cross section without matching
Wgamma_runs  = 137
Wgamma_NLO   = 2.37 # k-factor [14 TeV]
Wgamma_init_sim_eve = 100000 * Wgamma_runs

In [25]:
#Wgamma_cut_ev = 3810394
cross       = Wgamma_cross * 1000 * Wgamma_NLO
aceptancia  = Wgamma_cut_ev / (1. * Wgamma_init_sim_eve)
luminosidad = 100

Wgamma_fidcross = cross * aceptancia
Wgamma_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', Wgamma_fidcross)
print('Events expected: ', Wgamma_expected)

('fiducial cross section: ', 110.02191836496351)
('Events expected: ', 11002.191836496351)


In [26]:
Wgamma_init_sim_eve

13700000

### p p > W jets > $\nu$ l jets 
- [x] Done

In [21]:
# PAPER MADGRAPH for the NLO corrections
# https://arxiv.org/pdf/1405.0301.pdf

In [17]:
Folder    = '/home/martinrios/Downloads/MG5aMC_LTS_2.9.9/MG5_aMC_v2_9_9/pptowjtolnuj/'
datFolder = '../data/Wjets/'

Wjets_runs  = 500
Wjets_runs_1M = 40

Wjets_init_ev = 0
Wjets_cut_ev  = 0

Wjets_photon  = []
Wjets_jet     = []
Wjets_lepton  = []
Wjets_MET     = []
Wjets_hadinfo = []


counter_bad_jets = 0
contador_tau = 0
i1M_contador = 0
contador_btag = 0

analyzed_100k_runs = 0
analyzed_1M_runs = 0
for ii in range(1, Wjets_runs + Wjets_runs_1M):
    if ii not in [19, 33, 106, 325]: # Runs que fueron cancelados
        if ii < Wjets_runs:
            f_name = Folder + "run100k_" + str(ii) + ".lhco"
            analyzed_100k_runs += 1
        else:
            i1M_contador += 1
            f_name = Folder + "run1M_" + str(i1M_contador) + ".lhco"
            analyzed_1M_runs += 1
            
        print('Reading ', f_name)
        inputevents = LHCO_reader.Events(f_name = f_name)

        Wjets_init_ev += len(inputevents)
        print('Esta run tuvo ' + str(len(inputevents)) + ' eventos en el lhco')
        
        for i in range(len(inputevents)):

            num_jets = 0
    #        num_btag = 0
            HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets


            # at least one: photon, jet and lepton (electron or muon)
            if inputevents[i].number()["photon"] > 0 and inputevents[i].number()["jet"] > 0 and (inputevents[i].number()["electron"] > 0 or inputevents[i].number()["muon"] > 0):
                if inputevents[i]["jet"][0]["PT"] > 100 and -4.5 < inputevents[i]["jet"][0]["eta"] < 4.5:
                    if inputevents[i]["photon"][0]["PT"] > 10 and -2.37 < inputevents[i]["photon"][0]["eta"] < 2.37:
                        if inputevents[i]["MET"][0]["PT"] > 200:
                            # lepton info
                            if (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] > 0):
                                if inputevents[i]["electron"][0]["PT"] > inputevents[i]["muon"][0]["PT"]: 
                                    flag = 'electron'
                                    eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                                else: 
                                    flag = 'muon'
                                    eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron
                            elif (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] == 0): 
                                flag = 'electron'
                                eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                            elif (inputevents[i].number()["muon"] > 0) and (inputevents[i].number()["electron"] == 0): 
                                flag = 'muon'
                                eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron

                            if (-2.47 < eta_leading_lepton < -1.52) or (-1.37 < eta_leading_lepton < 1.37) or (1.52 < eta_leading_lepton < 2.47): 

                                # how many jets (total) and total hadronic pT
                                flag_bad_jets = 0                            
                                num_jets = 0
                                HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
                                HT = 0 # total HT, defined as the scalar sum of the pT of all reconstructed particles

                                for j in range(inputevents[i].number()["jet"]):
                                    if (inputevents[i]["jet"][j]["PT"] > 20) and (-4.5 < inputevents[i]["jet"][j]["eta"] < 4.5):
                                        num_jets += 1
                                        HT_had += inputevents[i]["jet"][j]["PT"]
                                        HT += inputevents[i]["jet"][j]["PT"]
                                    if (np.abs(inputevents[i]["jet"][j]["eta"]) > 4.5) or (inputevents[i]["jet"][j]["PT"] < 20):
                                        flag_bad_jets = 1

                                # how many photons                         
                                num_photons = 0
                                for j in range(inputevents[i].number()["photon"]):
                                    if inputevents[i]["photon"][j]["PT"] > 10 and -2.37 < inputevents[i]["photon"][j]["eta"] < 2.37:
                                        num_photons = num_photons + 1
                                        HT += inputevents[i]["photon"][j]["PT"]

                                # how many muons                        
                                num_muons = 0
                                for j in range(inputevents[i].number()["muon"]):
                                    if inputevents[i]["muon"][j]["PT"] > 10 and -2.7 < inputevents[i]["muon"][j]["eta"] < 2.7:
                                        num_muons = num_muons + 1
                                        HT += inputevents[i]["muon"][j]["PT"]

                                # how many electrons                        
                                num_electrons = 0
                                for j in range(inputevents[i].number()["electron"]):
                                    if inputevents[i]["electron"][j]["PT"] > 10:
                                        eta = inputevents[i]["electron"][j]["eta"]
                                        if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                            num_electrons = num_electrons + 1
                                            HT += inputevents[i]["electron"][j]["PT"]

                                if flag_bad_jets == 1: counter_bad_jets += 1 
                                if flag_bad_jets == 0:
                                    num_taus = 0
                                    if inputevents[i].number()["tau"] > 0: 
                                        contador_tau = contador_tau + 1
                                        # how many taus                    
                                        for j in range(inputevents[i].number()["tau"]):
                                            if inputevents[i]["tau"][j]["PT"] > 20:
                                                eta = inputevents[i]["tau"][j]["eta"]
                                                if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                                    num_taus = num_taus + 1
                                                    HT += inputevents[i]["tau"][j]["PT"]
                                    
                                    
                                    if inputevents[i]["jet"][0]['btag'] == 1: contador_btag = contador_btag + 1
                                    
                                    # photon info:
                                    with open(datFolder + 'photon_bu.dat', 'a') as f:
                                        f.write('%f %f %f %d\n' % (inputevents[i]["photon"][0]["PT"],inputevents[i]["photon"][0]["eta"],inputevents[i]["photon"][0]["phi"], num_photons))

                                    # jet info:
                                    with open(datFolder + 'jet_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["jet"][0]["PT"],inputevents[i]["jet"][0]["eta"],inputevents[i]["jet"][0]["phi"]))

                                    # lepton info
                                    num_leptons = num_electrons + num_muons
                                    if flag == "electron":
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["electron"][0]["PT"],inputevents[i]["electron"][0]["eta"],inputevents[i]["electron"][0]["phi"], num_leptons, num_taus))
                                    else:
                                        with open(datFolder + 'lepton_bu.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["muon"][0]["PT"],inputevents[i]["muon"][0]["eta"],inputevents[i]["muon"][0]["phi"], num_leptons, num_taus))

                                    # MET (eta=0 by def, but I save phi because I have many jets)
                                    with open(datFolder + 'MET_bu.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))

                                    # hadronic info
                                    with open(datFolder + 'hadinfo_bu.dat', 'a') as f:
                                        f.write('%f %f %d\n' % (HT, HT_had, num_jets))

                                    # total events that passed the cuts
                                    Wjets_cut_ev += 1

print('Num initial events: ', Wjets_init_ev)
print('Total pasaron los cortes: ', Wjets_cut_ev)
print('Hay: ', contador_tau, ' taus')
print('Hay: ', counter_bad_jets, ' eventos con al menos un jet malo')
print('Hay: ', contador_btag, ' con btag en leading jet')

('Reading ', '/home/martinrios/Downloads/MG5aMC_LTS_2.9.9/MG5_aMC_v2_9_9/pptowjtolnuj/run100k_1.lhco')
Esta run tuvo 54919 eventos en el lhco
('Reading ', '/home/martinrios/Downloads/MG5aMC_LTS_2.9.9/MG5_aMC_v2_9_9/pptowjtolnuj/run100k_2.lhco')
Esta run tuvo 54896 eventos en el lhco
('Reading ', '/home/martinrios/Downloads/MG5aMC_LTS_2.9.9/MG5_aMC_v2_9_9/pptowjtolnuj/run100k_3.lhco')
Esta run tuvo 54637 eventos en el lhco
('Reading ', '/home/martinrios/Downloads/MG5aMC_LTS_2.9.9/MG5_aMC_v2_9_9/pptowjtolnuj/run100k_4.lhco')
Esta run tuvo 54863 eventos en el lhco
('Reading ', '/home/martinrios/Downloads/MG5aMC_LTS_2.9.9/MG5_aMC_v2_9_9/pptowjtolnuj/run100k_5.lhco')
Esta run tuvo 54743 eventos en el lhco
('Reading ', '/home/martinrios/Downloads/MG5aMC_LTS_2.9.9/MG5_aMC_v2_9_9/pptowjtolnuj/run100k_6.lhco')
Esta run tuvo 54751 eventos en el lhco
('Reading ', '/home/martinrios/Downloads/MG5aMC_LTS_2.9.9/MG5_aMC_v2_9_9/pptowjtolnuj/run100k_7.lhco')
Esta run tuvo 54698 eventos en el lhco
('Read

In [19]:
1050./46783

0.02244405018917128

In [27]:
Wjets_cross = 168.12 #[pb]  # Integrated cross section without matching
# Hay 4 corridas de 100k canceladas (19,33,106,325) y 3 corridas q fueron renombradas como 100k pero tienen 1M (81,82,83)
Wjets_runs  = analyzed_100k_runs - 3
Wjets_NLO   = 1.27 # k-factor [14 TeV]
Wjets_init_sim_eve = (100000 * Wjets_runs) + 3000000 + (1000000 * analyzed_1M_runs)

In [28]:
analyzed_100k_runs

495

In [29]:
analyzed_1M_runs

40

In [30]:
# full
print(Wjets_init_ev)
print(Wjets_cut_ev)

50384374
46783


In [21]:
Wjets_init_ev = 50384374 #40877363 #19022395 # Events after matching
Wjets_cut_ev = 259345 #210604 #98061

In [31]:
# full

cross       = Wjets_cross * 1000 * Wjets_NLO
aceptancia  = Wjets_cut_ev / (1. * Wjets_init_sim_eve)
luminosidad = 100

Wjets_fidcross = cross * aceptancia
Wjets_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', Wjets_fidcross)
print('Events expected: ', Wjets_expected)

('fiducial cross section: ', 108.33785910195228)
('Events expected: ', 10833.785910195227)


In [59]:
# Primeros 325
print(Wjets_init_ev)
print(Wjets_cut_ev)

19022395
98061


In [38]:
# 326-500
print(Wjets_init_ev)
print(Wjets_cut_ev)

9507011
48741


In [65]:
# 40 1M
print(Wjets_init_ev)
print(Wjets_cut_ev)

21309179
109626


In [60]:
# Primeros 325

cross       = Wjets_cross * 1000 * Wjets_NLO
aceptancia  = Wjets_cut_ev / (1. * Wjets_init_sim_eve)
luminosidad = 100

Wjets_fidcross = cross * aceptancia
Wjets_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', Wjets_fidcross)
print('Events expected: ', Wjets_expected)

('fiducial cross section: ', 601.6448119655172)
('Events expected: ', 60164.48119655172)


In [54]:
# 325-500

cross       = Wjets_cross * 1000 * Wjets_NLO
aceptancia  = Wjets_cut_ev / (1. * Wjets_init_sim_eve)
luminosidad = 100

Wjets_fidcross = cross * aceptancia
Wjets_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', Wjets_fidcross)
print('Events expected: ', Wjets_expected)

('fiducial cross section: ', 598.0924073793103)
('Events expected: ', 59809.24073793103)


In [66]:
# 40 1M

cross       = Wjets_cross * 1000 * Wjets_NLO
aceptancia  = Wjets_cut_ev / (1. * Wjets_init_sim_eve)
luminosidad = 100

Wjets_fidcross = cross * aceptancia
Wjets_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', Wjets_fidcross)
print('Events expected: ', Wjets_expected)

('fiducial cross section: ', 600.1669323692307)
('Events expected: ', 60016.69323692307)


In [31]:
cross       = Wjets_cross * 1000 * Wjets_NLO
aceptancia  = Wjets_cut_ev / (1. * Wjets_init_sim_eve)
luminosidad = 100

Wjets_fidcross = cross * aceptancia
Wjets_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', Wjets_fidcross)
print('Events expected: ', Wjets_expected)

('fiducial cross section: ', 592.8308799304813)
('Events expected: ', 59283.087993048124)
